In [1]:
#Import Modules

import os
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string
import re

In [ ]:
#Manually define variables
variable_Extraction_date = "November 03, 2025" #Enter date of data extraction. Format: "Month DD, YYYY"
variable_range_applications_academic_year = "2025-26" #Enter the academic year(s) of your application data. Format: "YYYY-YY" or "YYYY-YY & YYYY-YY"
variable_current_academic_year_file_name = "2025-26" #Enter academic year for output file name. Format: "YYYY-YY"

In [3]:
#Course Code List (must match all courses on template, using course codes in Power Campus)

#List all Business Administration programs on template
program_ID_Business_Administration = [
    'BABUSC',
    'BAACTD',
    'BABUSD',
    'BAESPM',
    'BAHRMD',
    'GSCMD',
    'DIGMD',
    'BAATCO',
    'BABSCO',
    'NAO',
    'ALO',
    'HOSMNG',
    'BAHRMC',
    'OFAC'
]

#List all Education and Academic Upgrading programs on template
program_ID_Education_and_Academic_Upgrading = [
    'EAP',
    'HSAFA',
    'HSAD',
    'HSGD',
    'HSNP',
    'UTELEM',
    'UTSEC'
]

#List all University Studies and Environmental Science programs on template
program_ID_University_Studies_and_Environmental_Science = [
    'ENVTD',
    'ENVNC',
    'UTARTS',
    'UTCOMM',
    'UTENG',
    'UTSCGS',
    'UTSCWK',
    'OPEN',
    'GOVNSC',
    'COMSCC',
    'COMSCD',
    'GNARTC',
    'GNSCIC',
    'ANCPDC',
    'NSPDC',
    'AITS'
]

#List all Language Instruction for Newcomers to Canada programs on template
program_ID_Language_Instruction_for_Newcomers_to_Canada = [
    'LINC13'
]

#List all Childhood Studies and Human Services programs on template
program_ID_Childhood_Studies_and_Human_Services = [
    'AELCC',
    'ECCERT',
    'ECDIP',
    'CYCD',
    'ECEAC',
    'SOCWKD'
]

#List all Nursing and Allied Health programs (excluding 3rd year nursing programs) on template
program_ID_Nursing_and_Allied_Health = [
    'HCAC',
    'PCPC',
    'ACP',
    'PNR',
    'BSCN',
    'IENCN'
]

#List 3rd year nursing programs on template
program_ID_NursingYr3 = [
    'BSCACN',
    'BSCAEE',
    'BSCAFT'
]

#List all Pre-employment programs on template
program_ID_Pre_employment = [
    'PAELEC',
    'PAWELD',
    'PAHET',
    'PAPLUM'
]

#List all Power Engineering programs on template
program_ID_Power_Engineering = [
    'POWCM4',
    'POWCM3',
    'POWCP4',
    'POWCO4',
    'POWCO3',
    'PPET'
]

#Combine all program Lists into 1 List for all programs
program_ID_all = (
    program_ID_Business_Administration +
    program_ID_Education_and_Academic_Upgrading +
    program_ID_University_Studies_and_Environmental_Science +
    program_ID_Language_Instruction_for_Newcomers_to_Canada +
    program_ID_Childhood_Studies_and_Human_Services +
    program_ID_Nursing_and_Allied_Health +
    program_ID_Pre_employment +
    program_ID_Power_Engineering
)

#List all possible Population values (All letters must be capitalized here, regardless of whether it is capitalized or not in your data)

population_values = ['DOMESTIC', 'INTERNATIONAL']

#List all possible Year values

year_values = ['1', '2', '3']


In [4]:
#Search in the folder 'Data Files' for 4 application comparative files, and assign them to 4 variables chronologically

term_order = {"Winter": 1, "Spring": 2, "Summer": 3, "Fall": 4}

def extract_term_and_year(filename):
    match = re.search(r"(Winter|Spring|Summer|Fall)\s*(\d{4})", filename, re.IGNORECASE)
    if match:
        term = match.group(1).capitalize()
        year = int(match.group(2))
        return (year, term_order[term])
    return None

def get_chronological_application_comparative_files(folder_path):
    files = os.listdir(folder_path)
    matched_files = []

    for file in files:
        if not file.lower().endswith(".xlsx"):
            continue
        if re.search(r"applications?\s*comparative", file, re.IGNORECASE):
            term_info = extract_term_and_year(file)
            if term_info:
                matched_files.append((term_info, os.path.join(folder_path, file)))

    if len(matched_files) != 4:
        raise ValueError(f"Expected exactly 4 'Application Comparative' files, found {len(matched_files)}.")

    matched_files.sort(key=lambda x: (x[0][0], x[0][1]))

    return [f[1] for f in matched_files]

data_folder = "Data Files"

(
    term_1_file_application_comparative_filename,
    term_2_file_application_comparative_filename,
    term_3_file_application_comparative_filename,
    term_4_file_application_comparative_filename,
) = get_chronological_application_comparative_files(data_folder)

print("Term 1 Application Comparative:", term_1_file_application_comparative_filename)
print("Term 2 Application Comparative:", term_2_file_application_comparative_filename)
print("Term 3 Application Comparative:", term_3_file_application_comparative_filename)
print("Term 4 Application Comparative:", term_4_file_application_comparative_filename)

Term 1 Application Comparative: Data Files\zTestingData-Application Comparative -Summer 2025 as of June 02, 2025.xlsx
Term 2 Application Comparative: Data Files\zTestingData-Application Comparative -Fall 2025 as of June 02, 2025.xlsx
Term 3 Application Comparative: Data Files\zTestingData-Application Comparative -Winter 2026 as of June 02, 2025.xlsx
Term 4 Application Comparative: Data Files\zTestingData-Application Comparative -Spring 2026 as of June 02, 2025.xlsx


In [5]:
#Search in the folder 'Data Files' for 4 application details files, and assign them to 4 variables chronologically

term_order = {"Winter": 1, "Spring": 2, "Summer": 3, "Fall": 4}

def extract_term_and_year(filename):
    match = re.search(r"(Winter|Spring|Summer|Fall)\s*(\d{4})", filename, re.IGNORECASE)
    if match:
        term = match.group(1).capitalize()
        year = int(match.group(2))
        return (year, term_order[term])
    return None

def get_chronological_applications_reports_details_files(folder_path):
    files = os.listdir(folder_path)
    matched_files = []

    for file in files:
        if not file.lower().endswith(".xlsx"):
            continue
        if re.search(r"applications?\s*reports?\s*details?", file, re.IGNORECASE):
            term_info = extract_term_and_year(file)
            if term_info:
                matched_files.append((term_info, os.path.join(folder_path, file)))

    if len(matched_files) != 4:
        raise ValueError(f"Expected exactly 4 'Application Comparative' files, found {len(matched_files)}.")

    matched_files.sort(key=lambda x: (x[0][0], x[0][1]))

    return [f[1] for f in matched_files]

data_folder = "Data Files"

(
    term_1_file_application_details_filename,
    term_2_file_application_details_filename,
    term_3_file_application_details_filename,
    term_4_file_application_details_filename,
) = get_chronological_applications_reports_details_files(data_folder)

print("Term 1 Applications Reports Details:", term_1_file_application_details_filename)
print("Term 2 Applications Reports Details:", term_2_file_application_details_filename)
print("Term 3 Applications Reports Details:", term_3_file_application_details_filename)
print("Term 4 Applications Reports Details:", term_4_file_application_details_filename)

Term 1 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Summer 2025 as of June 02, 2025.xlsx
Term 2 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Fall 2025 as of June 02, 2025.xlsx
Term 3 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Winter 2026 as of June 02, 2025.xlsx
Term 4 Applications Reports Details: Data Files\zTestingData-Applications Reports Details-IR-New-Spring 2026 as of June 02, 2025.xlsx


In [6]:
#Set filenames as variables

term_1_file_application_comparative = term_1_file_application_comparative_filename
term_1_file_application_details = term_1_file_application_details_filename

term_2_file_application_comparative = term_2_file_application_comparative_filename
term_2_file_application_details = term_2_file_application_details_filename

term_3_file_application_comparative = term_3_file_application_comparative_filename
term_3_file_application_details = term_3_file_application_details_filename

term_4_file_application_comparative = term_4_file_application_comparative_filename
term_4_file_application_details = term_4_file_application_details_filename

In [7]:
#Customized Math Formula Functions

#Function for subtracting current_year_value by previous_year_value, then dividing by previous_year_value, then multiplying by 100, rounding to 1 decimal place
def percent_difference_value_calculator_round_1(current_year_value, previous_year_value):
    try:
        return round((current_year_value - previous_year_value) / previous_year_value * 100, 1)
    except ZeroDivisionError:
        return "N/A"
    except TypeError:
        return "N/A"
    
#Function for dividing numerator by denominator, then multiplying by 100, rounding to 1 decimal place
def division_calculator(numerator, denominator): 
    try:
        return round(numerator / denominator * 100, 1)
    except ZeroDivisionError:
        return "N/A"
    except TypeError:
        return "N/A"
    
#Function for dividing numerator by denominator, calculating a decimal value. Return None if denominator is None. Treat numerator as 0 when it is None. 
def division_calculator_decimal(numerator, denominator): 
    if denominator in [None, 0]:
        return None
    if numerator is None or numerator == 0:
        return None
    try:
        return numerator / denominator
    except TypeError:
        return None


In [8]:
#Application Comparative Term 1
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_1_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_1_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_1_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_1_year_current = int(re.search(r"\d{4}", term_1_term_year_current).group())
term_1_year_minus_one = term_1_year_current - 1
term_1_year_minus_two = term_1_year_current - 2
term_1_year_minus_three = term_1_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_1_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_1_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_1_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_1_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_1_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_1_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_1_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_1_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_1_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_1_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_1_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_1_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_1_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_1_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_1_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_1_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_1_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_1_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_1_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_1_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_1_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_1_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_1_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_one:
            term_1_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_merged_range}")

# Now term_1_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_1_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_1_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_one_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_Unique_merged_range}")

# Now term_1_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_1_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_1_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_one_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_one_Total_Apps_merged_range}")

# Now term_1_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_1_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_1_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_1_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_two:
            term_1_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_merged_range}")

# Now term_1_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_two_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_Unique_merged_range}")

# Now term_1_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_two_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_two_Total_Apps_merged_range}")

# Now term_1_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_1_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_1_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_1_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_1_Report_Summary_last_row + 1
max_search_row = term_1_All_Applicants_first_row - 1
min_search_col_letter = term_1_Report_Summary_first_column  # Column A
max_search_col_letter = term_1_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_1_year_minus_three:
            term_1_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_merged_range}")

# Now term_1_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_1_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_1_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_three_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_1_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_Unique_merged_range}")

# Now term_1_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_1_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_1_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_1_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_1_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_1_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_1_year_minus_three_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_1_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_1_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_1_year_minus_three_Total_Apps_merged_range}")

# Now term_1_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_1_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_1_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_1_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_1_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_1_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_1_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_1_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_1_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_1_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_1_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_1_year_minus_one_Unique_All_Applicants," ",term_1_year_minus_one_Unique_International_Applicants," ",term_1_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_1_year_minus_one_Total_Apps_All_Applicants," ",term_1_year_minus_one_Total_Apps_International_Applicants," ",term_1_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_1_year_minus_two_Unique_All_Applicants," ",term_1_year_minus_two_Unique_International_Applicants," ",term_1_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_1_year_minus_two_Total_Apps_All_Applicants," ",term_1_year_minus_two_Total_Apps_International_Applicants," ",term_1_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_1_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_1_All_Applicants_first_row
max_search_row = term_1_All_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_1_International_Applicants_first_row
max_search_row = term_1_International_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK-------------------------------------------------------------------------------term_1--------------------3
# Define your search boundaries using column letters
min_search_row = term_1_Domestic_Applicants_first_row
max_search_row = term_1_Domestic_Applicants_last_row
min_search_col_letter = term_1_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_1_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_1_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_1_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_1_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_1_year_minus_three_Unique_All_Applicants," ",term_1_year_minus_three_Unique_International_Applicants," ",term_1_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_1_year_minus_three_Total_Apps_All_Applicants," ",term_1_year_minus_three_Total_Apps_International_Applicants," ",term_1_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_1_term_year_current)

if term_match:
    term_1_term = term_match.group(1)
    term_1_year_current = int(term_match.group(2))

    # Subtract one year
    term_1_year_minus_one = term_1_year_current - 1

    # Construct the previous term-year string
    term_1_term_year_minus_one = f"{term_1_term} {term_1_year_minus_one}"

#Variables created
print("Current Term and Year:", term_1_term_year_current)
print("Current Term and Year Minus 1:", term_1_term_year_minus_one)
print("\n")
print(f"{term_1_year_minus_one}, Unique, All Applicants:", term_1_year_minus_one_Unique_All_Applicants)
print(f"{term_1_year_minus_one}, Unique, International Applicants:", term_1_year_minus_one_Unique_International_Applicants)
print(f"{term_1_year_minus_one}, Unique, Domestic Applicants:", term_1_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, All Applicants:", term_1_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, International Applicants:", term_1_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_one}, Total Apps, Domestic Applicants:", term_1_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_1_year_minus_two}, Unique, All Applicants:", term_1_year_minus_two_Unique_All_Applicants)
print(f"{term_1_year_minus_two}, Unique, International Applicants:", term_1_year_minus_two_Unique_International_Applicants)
print(f"{term_1_year_minus_two}, Unique, Domestic Applicants:", term_1_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, All Applicants:", term_1_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, International Applicants:", term_1_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_two}, Total Apps, Domestic Applicants:", term_1_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_1_year_minus_three}, Unique, All Applicants:", term_1_year_minus_three_Unique_All_Applicants)
print(f"{term_1_year_minus_three}, Unique, International Applicants:", term_1_year_minus_three_Unique_International_Applicants)
print(f"{term_1_year_minus_three}, Unique, Domestic Applicants:", term_1_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, All Applicants:", term_1_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, International Applicants:", term_1_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_1_year_minus_three}, Total Apps, Domestic Applicants:", term_1_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Summer 2025
Current Term and Year Minus 1: Summer 2024


2024, Unique, All Applicants: 1425
2024, Unique, International Applicants: 1387
2024, Unique, Domestic Applicants: 38
2024, Total Apps, All Applicants: 1431
2024, Total Apps, International Applicants: 1393
2024, Total Apps, Domestic Applicants: 38


2023, Unique, All Applicants: 39
2023, Unique, International Applicants: 1
2023, Unique, Domestic Applicants: 38
2023, Total Apps, All Applicants: 39
2023, Total Apps, International Applicants: 1
2023, Total Apps, Domestic Applicants: 38


2022, Unique, All Applicants: 42
2022, Unique, International Applicants: 0
2022, Unique, Domestic Applicants: 42
2022, Total Apps, All Applicants: 42
2022, Total Apps, International Applicants: 0
2022, Total Apps, Domestic Applicants: 42


In [9]:
#Application Comparative Term 2
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_2_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_2_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_2_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_2_year_current = int(re.search(r"\d{4}", term_2_term_year_current).group())
term_2_year_minus_one = term_2_year_current - 1
term_2_year_minus_two = term_2_year_current - 2
term_2_year_minus_three = term_2_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_2_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_2_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_2_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_2_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_2_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_2_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_2_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_2_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_2_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_2_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_2_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_2_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_2_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_2_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_2_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_2_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_2_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_2_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_2_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_2_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_2_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_2_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_2_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_one:
            term_2_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_merged_range}")

# Now term_2_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_2_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_2_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_one_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_Unique_merged_range}")

# Now term_2_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_2_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_2_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_one_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_one_Total_Apps_merged_range}")

# Now term_2_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_2_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_2_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_2_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_two:
            term_2_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_merged_range}")

# Now term_2_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_two_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_Unique_merged_range}")

# Now term_2_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_two_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_two_Total_Apps_merged_range}")

# Now term_2_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_2_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_2_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_2_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_2_Report_Summary_last_row + 1
max_search_row = term_2_All_Applicants_first_row - 1
min_search_col_letter = term_2_Report_Summary_first_column  # Column A
max_search_col_letter = term_2_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_2_year_minus_three:
            term_2_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_merged_range}")

# Now term_2_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_2_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_2_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_three_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_2_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_Unique_merged_range}")

# Now term_2_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_2_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_2_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_2_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_2_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_2_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_2_year_minus_three_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_2_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_2_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_2_year_minus_three_Total_Apps_merged_range}")

# Now term_2_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_2_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_2_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_2_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_2_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_2_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_2_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_2_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_2_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_2_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_2_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_2_year_minus_one_Unique_All_Applicants," ",term_2_year_minus_one_Unique_International_Applicants," ",term_2_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_2_year_minus_one_Total_Apps_All_Applicants," ",term_2_year_minus_one_Total_Apps_International_Applicants," ",term_2_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_2_year_minus_two_Unique_All_Applicants," ",term_2_year_minus_two_Unique_International_Applicants," ",term_2_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_2_year_minus_two_Total_Apps_All_Applicants," ",term_2_year_minus_two_Total_Apps_International_Applicants," ",term_2_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_2_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_2_All_Applicants_first_row
max_search_row = term_2_All_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_2_International_Applicants_first_row
max_search_row = term_2_International_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_2_Domestic_Applicants_first_row
max_search_row = term_2_Domestic_Applicants_last_row
min_search_col_letter = term_2_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_2_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_2_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_2_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_2_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_2_year_minus_three_Unique_All_Applicants," ",term_2_year_minus_three_Unique_International_Applicants," ",term_2_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_2_year_minus_three_Total_Apps_All_Applicants," ",term_2_year_minus_three_Total_Apps_International_Applicants," ",term_2_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_2_term_year_current)

if term_match:
    term_2_term = term_match.group(1)
    term_2_year_current = int(term_match.group(2))

    # Subtract one year
    term_2_year_minus_one = term_2_year_current - 1

    # Construct the previous term-year string
    term_2_term_year_minus_one = f"{term_2_term} {term_2_year_minus_one}"

#Variables created
print("Current Term and Year:", term_2_term_year_current)
print("Current Term and Year Minus 1:", term_2_term_year_minus_one)
print("\n")
print(f"{term_2_year_minus_one}, Unique, All Applicants:", term_2_year_minus_one_Unique_All_Applicants)
print(f"{term_2_year_minus_one}, Unique, International Applicants:", term_2_year_minus_one_Unique_International_Applicants)
print(f"{term_2_year_minus_one}, Unique, Domestic Applicants:", term_2_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, All Applicants:", term_2_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, International Applicants:", term_2_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_one}, Total Apps, Domestic Applicants:", term_2_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_2_year_minus_two}, Unique, All Applicants:", term_2_year_minus_two_Unique_All_Applicants)
print(f"{term_2_year_minus_two}, Unique, International Applicants:", term_2_year_minus_two_Unique_International_Applicants)
print(f"{term_2_year_minus_two}, Unique, Domestic Applicants:", term_2_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, All Applicants:", term_2_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, International Applicants:", term_2_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_two}, Total Apps, Domestic Applicants:", term_2_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_2_year_minus_three}, Unique, All Applicants:", term_2_year_minus_three_Unique_All_Applicants)
print(f"{term_2_year_minus_three}, Unique, International Applicants:", term_2_year_minus_three_Unique_International_Applicants)
print(f"{term_2_year_minus_three}, Unique, Domestic Applicants:", term_2_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, All Applicants:", term_2_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, International Applicants:", term_2_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_2_year_minus_three}, Total Apps, Domestic Applicants:", term_2_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Fall 2025
Current Term and Year Minus 1: Fall 2024


2024, Unique, All Applicants: 4369
2024, Unique, International Applicants: 3508
2024, Unique, Domestic Applicants: 861
2024, Total Apps, All Applicants: 4655
2024, Total Apps, International Applicants: 3684
2024, Total Apps, Domestic Applicants: 971


2023, Unique, All Applicants: 2479
2023, Unique, International Applicants: 1632
2023, Unique, Domestic Applicants: 847
2023, Total Apps, All Applicants: 2713
2023, Total Apps, International Applicants: 1755
2023, Total Apps, Domestic Applicants: 958


2022, Unique, All Applicants: 1645
2022, Unique, International Applicants: 829
2022, Unique, Domestic Applicants: 816
2022, Total Apps, All Applicants: 1778
2022, Total Apps, International Applicants: 881
2022, Total Apps, Domestic Applicants: 897


In [10]:
#Application Comparative Term 3
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_3_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_3_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_3_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_3_year_current = int(re.search(r"\d{4}", term_3_term_year_current).group())
term_3_year_minus_one = term_3_year_current - 1
term_3_year_minus_two = term_3_year_current - 2
term_3_year_minus_three = term_3_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_3_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_3_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_3_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_3_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_3_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_3_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_3_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_3_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_3_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_3_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_3_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_3_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_3_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_3_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_3_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_3_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_3_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_3_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_3_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_3_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_3_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_3_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_3_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_one:
            term_3_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_merged_range}")

# Now term_3_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_3_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_3_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_one_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_Unique_merged_range}")

# Now term_3_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_3_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_3_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_one_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_one_Total_Apps_merged_range}")

# Now term_3_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_3_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_3_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_3_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_two:
            term_3_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_merged_range}")

# Now term_3_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_two_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_Unique_merged_range}")

# Now term_3_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_two_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_two_Total_Apps_merged_range}")

# Now term_3_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_3_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_3_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_3_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_3_Report_Summary_last_row + 1
max_search_row = term_3_All_Applicants_first_row - 1
min_search_col_letter = term_3_Report_Summary_first_column  # Column A
max_search_col_letter = term_3_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_3_year_minus_three:
            term_3_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_merged_range}")

# Now term_3_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_3_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_3_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_three_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_3_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_Unique_merged_range}")

# Now term_3_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_3_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_3_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_3_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_3_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_3_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_3_year_minus_three_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_3_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_3_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_3_year_minus_three_Total_Apps_merged_range}")

# Now term_3_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_3_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_3_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_3_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_3_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_3_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_3_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_3_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_3_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_3_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_3_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_3_year_minus_one_Unique_All_Applicants," ",term_3_year_minus_one_Unique_International_Applicants," ",term_3_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_3_year_minus_one_Total_Apps_All_Applicants," ",term_3_year_minus_one_Total_Apps_International_Applicants," ",term_3_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_3_year_minus_two_Unique_All_Applicants," ",term_3_year_minus_two_Unique_International_Applicants," ",term_3_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_3_year_minus_two_Total_Apps_All_Applicants," ",term_3_year_minus_two_Total_Apps_International_Applicants," ",term_3_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_3_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_3_All_Applicants_first_row
max_search_row = term_3_All_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_3_International_Applicants_first_row
max_search_row = term_3_International_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_3_Domestic_Applicants_first_row
max_search_row = term_3_Domestic_Applicants_last_row
min_search_col_letter = term_3_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_3_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_3_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_3_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_3_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_3_year_minus_three_Unique_All_Applicants," ",term_3_year_minus_three_Unique_International_Applicants," ",term_3_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_3_year_minus_three_Total_Apps_All_Applicants," ",term_3_year_minus_three_Total_Apps_International_Applicants," ",term_3_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_3_term_year_current)

if term_match:
    term_3_term = term_match.group(1)
    term_3_year_current = int(term_match.group(2))

    # Subtract one year
    term_3_year_minus_one = term_3_year_current - 1

    # Construct the previous term-year string
    term_3_term_year_minus_one = f"{term_3_term} {term_3_year_minus_one}"

#Variables created
print("Current Term and Year:", term_3_term_year_current)
print("Current Term and Year Minus 1:", term_3_term_year_minus_one)
print("\n")
print(f"{term_3_year_minus_one}, Unique, All Applicants:", term_3_year_minus_one_Unique_All_Applicants)
print(f"{term_3_year_minus_one}, Unique, International Applicants:", term_3_year_minus_one_Unique_International_Applicants)
print(f"{term_3_year_minus_one}, Unique, Domestic Applicants:", term_3_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, All Applicants:", term_3_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, International Applicants:", term_3_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_one}, Total Apps, Domestic Applicants:", term_3_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_3_year_minus_two}, Unique, All Applicants:", term_3_year_minus_two_Unique_All_Applicants)
print(f"{term_3_year_minus_two}, Unique, International Applicants:", term_3_year_minus_two_Unique_International_Applicants)
print(f"{term_3_year_minus_two}, Unique, Domestic Applicants:", term_3_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, All Applicants:", term_3_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, International Applicants:", term_3_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_two}, Total Apps, Domestic Applicants:", term_3_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_3_year_minus_three}, Unique, All Applicants:", term_3_year_minus_three_Unique_All_Applicants)
print(f"{term_3_year_minus_three}, Unique, International Applicants:", term_3_year_minus_three_Unique_International_Applicants)
print(f"{term_3_year_minus_three}, Unique, Domestic Applicants:", term_3_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, All Applicants:", term_3_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, International Applicants:", term_3_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_3_year_minus_three}, Total Apps, Domestic Applicants:", term_3_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Winter 2026
Current Term and Year Minus 1: Winter 2025


2025, Unique, All Applicants: 994
2025, Unique, International Applicants: 972
2025, Unique, Domestic Applicants: 22
2025, Total Apps, All Applicants: 1007
2025, Total Apps, International Applicants: 985
2025, Total Apps, Domestic Applicants: 22


2024, Unique, All Applicants: 326
2024, Unique, International Applicants: 289
2024, Unique, Domestic Applicants: 37
2024, Total Apps, All Applicants: 331
2024, Total Apps, International Applicants: 293
2024, Total Apps, Domestic Applicants: 38


2023, Unique, All Applicants: 115
2023, Unique, International Applicants: 70
2023, Unique, Domestic Applicants: 45
2023, Total Apps, All Applicants: 116
2023, Total Apps, International Applicants: 71
2023, Total Apps, Domestic Applicants: 45


In [11]:
#Application Comparative Term 4
# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your actual file name
ws = wb.active  # Use ws = wb["SheetName"] if known

# Define regex pattern to find the term and year (handles cases like "Spring2024" or "Spring 2024")
pattern = r"\b(Summer|Fall|Winter|Spring)\s*(\d{4})\b"  # Now correctly separates term & year

# Initialize term variable
term_4_term_year_current = None

# Iterate through all cells in the worksheet
for row in ws.iter_rows():
    for cell in row:
        if isinstance(cell.value, str):  # Ensure the cell contains text
            ##print(f"Checking cell: {cell.coordinate}, Value: '{cell.value}'")  # Debugging: print each cell
            
            match = re.search(pattern, cell.value, re.IGNORECASE)  # Case-insensitive search
            if match:
                term_name = match.group(1).capitalize()  # Capitalize first letter (e.g., "Spring")
                term_year = match.group(2)  # Extract the year (e.g., "2024")

                term_4_term_year_current = f"{term_name} {term_year}"  # Format as "Spring 2024"
                ##print(f"Match found: {match.group(0)}")  # Debug print
                break  # Stop searching once found
    if term_4_term_year_current:
        break  # Stop outer loop if term is found

#create variables for term year and years
term_4_year_current = int(re.search(r"\d{4}", term_4_term_year_current).group())
term_4_year_minus_one = term_4_year_current - 1
term_4_year_minus_two = term_4_year_current - 2
term_4_year_minus_three = term_4_year_current - 3

#Extract location of 'Report Summary'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_Report_Summary_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "Report Summary"
    if top_left_cell.value == "Report Summary":
        term_4_Report_Summary_range = merged_range
        ##print(f"Merged cell containing 'Report Summary' spans: {term_4_Report_Summary_range}")

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # "A45:AA45"

# Extract the first part before the number
term_4_Report_Summary_first_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[0]))

##print("First column:", term_4_Report_Summary_first_column)  # Output: "A"

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_Report_Summary_last_column = ''.join(filter(str.isalpha, range_str_Report_Summary.split(':')[1]))

##print("Last column:", term_4_Report_Summary_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Report_Summary.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_Report_Summary_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_Report_Summary_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Report_Summary = str(term_4_Report_Summary_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Report_Summary.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_Report_Summary_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_Report_Summary_last_row)
#Extract location of 'All Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_All_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "All Applicants":
        term_4_All_Applicants_range = merged_range
        ##print(f"Merged cell containing 'All Applicants' spans: {term_4_All_Applicants_range}")

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_All_Applicants_first_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[0]))

##print("First column:", term_4_All_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_All_Applicants_last_column = ''.join(filter(str.isalpha, range_str_All_Applicants.split(':')[1]))

##print("Last column:", term_4_All_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_All_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_All_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_All_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_All_Applicants = str(term_4_All_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_All_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_All_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_All_Applicants_last_row)
#Extract location of 'International Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_International_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "International Applicants":
        term_4_International_Applicants_range = merged_range
        #print(f"Merged cell containing 'International Applicants' spans: {term_4_International_Applicants_range}")

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_International_Applicants_first_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[0]))

##print("First column:", term_4_International_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_International_Applicants_last_column = ''.join(filter(str.isalpha, range_str_International_Applicants.split(':')[1]))

##print("Last column:", term_4_International_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_International_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_International_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_International_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_International_Applicants = str(term_4_International_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_International_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_International_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_International_Applicants_last_row)
#Extract location of 'Domestic Applicants'

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

term_4_Domestic_Applicants_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Check if the top-left cell contains "All Applicants"
    if top_left_cell.value == "Domestic Applicants":
        term_4_Domestic_Applicants_range = merged_range
        #print(f"Merged cell containing 'Domestic Applicants' spans: {term_4_Domestic_Applicants_range}")

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # "A45:AA45"

# Extract the first part before the number
term_4_Domestic_Applicants_first_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[0]))

##print("First column:", term_4_Domestic_Applicants_first_column)  # Output: "A"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_Domestic_Applicants_last_column = ''.join(filter(str.isalpha, range_str_Domestic_Applicants.split(':')[1]))

##print("Last column:", term_4_Domestic_Applicants_last_column)  # Output: "AA"

# Convert merged range to string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_Domestic_Applicants.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_Domestic_Applicants_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_Domestic_Applicants_first_row)  # Output: 45

# Convert the merged range to a string
range_str_Domestic_Applicants = str(term_4_Domestic_Applicants_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_Domestic_Applicants.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_Domestic_Applicants_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_Domestic_Applicants_last_row)
#Extract Location of 2023 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_one:
            term_4_year_minus_one_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_merged_range}")

# Now term_4_year_minus_one_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_one_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_4_year_minus_one_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_4_year_minus_one_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_year_minus_one_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_year_minus_one_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_one_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_one_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_Unique_merged_range}")

# Now term_4_year_minus_one_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_one_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

##print("First column:", term_4_year_minus_one_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

##print("Last column:", term_4_year_minus_one_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_one_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_one_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_one_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_one_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_one_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_one_Total_Apps_merged_range}")

# Now term_4_year_minus_one_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_one_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_one_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[0]))

#print("First column:", term_4_year_minus_one_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_one_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_one.split(':')[1]))

#print("Last column:", term_4_year_minus_one_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_one.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_one_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_one_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_one = str(term_4_year_minus_one_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_one.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_one_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_one_Total_Apps_last_row)
#Extract Location of 2022 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_two:
            term_4_year_minus_two_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_merged_range}")

# Now term_4_year_minus_two_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_two_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_two_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_Unique_merged_range}")

# Now term_4_year_minus_two_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_two_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_two_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_two_Total_Apps_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_two_Total_Apps_merged_range}")

# Now term_4_year_minus_two_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_two_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_two_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[0]))

#print("First column:", term_4_year_minus_two_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_two_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_two.split(':')[1]))

#print("Last column:", term_4_year_minus_two_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_two.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_two_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_two_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_two = str(term_4_year_minus_two_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_two.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_two_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_two_Total_Apps_last_row)
#Extract Location of 2021 everything 

# Define your search boundaries using column letters
min_search_row = term_4_Report_Summary_last_row + 1
max_search_row = term_4_All_Applicants_first_row - 1
min_search_col_letter = term_4_Report_Summary_first_column  # Column A
max_search_col_letter = term_4_Report_Summary_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == term_4_year_minus_three:
            term_4_year_minus_three_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_merged_range}")

# Now term_4_year_minus_three_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_three_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_4_year_minus_three_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_4_year_minus_three_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_three_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_three_last_row)


#Extract Location of 2023 unique

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_three_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_Unique_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Unique":
            term_4_year_minus_three_Unique_merged_range = merged_range
            #print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_Unique_merged_range}")

# Now term_4_year_minus_three_Unique_merged_range contains the merged range with "Domestic Applicants" (if found)
#print("Stored merged range:", term_4_year_minus_three_Unique_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_Unique_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

#print("First column:", term_4_year_minus_three_Unique_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_Unique_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

#print("Last column:", term_4_year_minus_three_Unique_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_Unique_first_row = int(''.join(filter(str.isdigit, first_cell)))

#print("First row:", term_4_year_minus_three_Unique_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_Unique_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_Unique_last_row = int(''.join(filter(str.isdigit, end_cell)))
#print("Last row extracted:", term_4_year_minus_three_Unique_last_row)


#Extract Location of 2023 Total_Apps

# Define your search boundaries using column letters
min_search_row = term_4_year_minus_three_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Load the workbook and select the first sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)
ws = wb.active  # Or use ws = wb["SheetName"]

term_4_year_minus_three_Total_Apps_merged_range = None

# Iterate over merged cell ranges
for merged_range in ws.merged_cells.ranges:
    # Get the top-left cell of the merged range
    top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
    
    # Only consider merged ranges whose top-left cell is within A5:C30
    if (min_search_row <= merged_range.min_row <= max_search_row and
        min_search_col <= merged_range.min_col <= max_search_col):
        
        # Check if the top-left cell contains the desired string
        if top_left_cell.value == "Total Apps":
            term_4_year_minus_three_Total_Apps_merged_range = merged_range
            ##print(f"Merged cell containing '2023' spans: {term_4_year_minus_three_Total_Apps_merged_range}")

# Now term_4_year_minus_three_Total_Apps_merged_range contains the merged range with "Domestic Applicants" (if found)
##print("Stored merged range:", term_4_year_minus_three_Total_Apps_merged_range)

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # "A45:AA45"

# Extract the first part before the number
term_4_year_minus_three_Total_Apps_first_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[0]))

##print("First column:", term_4_year_minus_three_Total_Apps_first_column)  # Output: "A"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the last part after ':'
term_4_year_minus_three_Total_Apps_last_column = ''.join(filter(str.isalpha, range_str_year_minus_three.split(':')[1]))

##print("Last column:", term_4_year_minus_three_Total_Apps_last_column)  # Output: "AA"

# Convert merged range to string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # Example: "A45:AA45"

# Extract the first part before ':'
first_cell = range_str_year_minus_three.split(':')[0]  # "A45"

# Extract only the digits (row number)
term_4_year_minus_three_Total_Apps_first_row = int(''.join(filter(str.isdigit, first_cell)))

##print("First row:", term_4_year_minus_three_Total_Apps_first_row)  # Output: 45

# Convert the merged range to a string
range_str_year_minus_three = str(term_4_year_minus_three_Total_Apps_merged_range)  # e.g., "A53:C54"

# Split on ':' to get the second cell reference
end_cell = range_str_year_minus_three.split(':')[1]  # "C54"

# Extract the digits from the cell reference
term_4_year_minus_three_Total_Apps_last_row = int(''.join(filter(str.isdigit, end_cell)))
##print("Last row extracted:", term_4_year_minus_three_Total_Apps_last_row)
#All 2023 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_one_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_one_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_one_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_one_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_one_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_4_year_minus_one_Unique_All_Applicants," ",term_4_year_minus_one_Unique_International_Applicants," ",term_4_year_minus_one_Unique_Domestic_Applicants)
##print("Extracted Number:", term_4_year_minus_one_Total_Apps_All_Applicants," ",term_4_year_minus_one_Total_Apps_International_Applicants," ",term_4_year_minus_one_Total_Apps_Domestic_Applicants)


#All 2022 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_two_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_two_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_two_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_two_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_two_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

##print("Extracted Number:", term_4_year_minus_two_Unique_All_Applicants," ",term_4_year_minus_two_Unique_International_Applicants," ",term_4_year_minus_two_Unique_Domestic_Applicants)
##print("Extracted Number:", term_4_year_minus_two_Total_Apps_All_Applicants," ",term_4_year_minus_two_Total_Apps_International_Applicants," ",term_4_year_minus_two_Total_Apps_Domestic_Applicants)


#All 2021 numbers

# Load the workbook and select the active sheet
wb = openpyxl.load_workbook(term_4_file_application_comparative, data_only=True)  # Replace with your file name
ws = wb.active  # Use ws = wb["SheetName"] if you know the sheet name

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Unique_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Unique_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Unique_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Unique_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Unique_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found


#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------1
# Define your search boundaries using column letters
min_search_row = term_4_All_Applicants_first_row
max_search_row = term_4_All_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_All_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_All_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_All_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------2
# Define your search boundaries using column letters
min_search_row = term_4_International_Applicants_first_row
max_search_row = term_4_International_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_International_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_International_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_International_Applicants is not None:
        break  # Exit outer loop if a value was found

#BREAKKKKKKKKKK---------------------------------------------------------------------------------------------------3
# Define your search boundaries using column letters
min_search_row = term_4_Domestic_Applicants_first_row
max_search_row = term_4_Domestic_Applicants_last_row
min_search_col_letter = term_4_year_minus_three_Total_Apps_first_column  # Column A
max_search_col_letter = term_4_year_minus_three_Total_Apps_last_column  # Column C

# Convert column letters to numeric indexes
min_search_col = column_index_from_string(min_search_col_letter)
max_search_col = column_index_from_string(max_search_col_letter)

# Initialize variable
term_4_year_minus_three_Total_Apps_Domestic_Applicants = None

# Iterate through the defined cell range
for row in range(min_search_row, max_search_row + 1):
    for col in range(min_search_col, max_search_col + 1):
        cell_value = ws.cell(row=row, column=col).value
        
        # Assuming you're looking for a numeric value
        if isinstance(cell_value, (int, float)):  
            term_4_year_minus_three_Total_Apps_Domestic_Applicants = cell_value
            break  # Stop searching once a value is found

    if term_4_year_minus_three_Total_Apps_Domestic_Applicants is not None:
        break  # Exit outer loop if a value was found

#print("Extracted Number:", term_4_year_minus_three_Unique_All_Applicants," ",term_4_year_minus_three_Unique_International_Applicants," ",term_4_year_minus_three_Unique_Domestic_Applicants)
#print("Extracted Number:", term_4_year_minus_three_Total_Apps_All_Applicants," ",term_4_year_minus_three_Total_Apps_International_Applicants," ",term_4_year_minus_three_Total_Apps_Domestic_Applicants)

# Extract the term and year 
term_match = re.match(r"([A-Za-z]+)\s+(\d{4})", term_4_term_year_current)

if term_match:
    term_4_term = term_match.group(1)
    term_4_year_current = int(term_match.group(2))

    # Subtract one year
    term_4_year_minus_one = term_4_year_current - 1

    # Construct the previous term-year string
    term_4_term_year_minus_one = f"{term_4_term} {term_4_year_minus_one}"

#Variables created
print("Current Term and Year:", term_4_term_year_current)
print("Current Term and Year Minus 1:", term_4_term_year_minus_one)
print("\n")
print(f"{term_4_year_minus_one}, Unique, All Applicants:", term_4_year_minus_one_Unique_All_Applicants)
print(f"{term_4_year_minus_one}, Unique, International Applicants:", term_4_year_minus_one_Unique_International_Applicants)
print(f"{term_4_year_minus_one}, Unique, Domestic Applicants:", term_4_year_minus_one_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, All Applicants:", term_4_year_minus_one_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, International Applicants:", term_4_year_minus_one_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_one}, Total Apps, Domestic Applicants:", term_4_year_minus_one_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_4_year_minus_two}, Unique, All Applicants:", term_4_year_minus_two_Unique_All_Applicants)
print(f"{term_4_year_minus_two}, Unique, International Applicants:", term_4_year_minus_two_Unique_International_Applicants)
print(f"{term_4_year_minus_two}, Unique, Domestic Applicants:", term_4_year_minus_two_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, All Applicants:", term_4_year_minus_two_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, International Applicants:", term_4_year_minus_two_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_two}, Total Apps, Domestic Applicants:", term_4_year_minus_two_Total_Apps_Domestic_Applicants)
print("\n")
print(f"{term_4_year_minus_three}, Unique, All Applicants:", term_4_year_minus_three_Unique_All_Applicants)
print(f"{term_4_year_minus_three}, Unique, International Applicants:", term_4_year_minus_three_Unique_International_Applicants)
print(f"{term_4_year_minus_three}, Unique, Domestic Applicants:", term_4_year_minus_three_Unique_Domestic_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, All Applicants:", term_4_year_minus_three_Total_Apps_All_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, International Applicants:", term_4_year_minus_three_Total_Apps_International_Applicants)
print(f"{term_4_year_minus_three}, Total Apps, Domestic Applicants:", term_4_year_minus_three_Total_Apps_Domestic_Applicants)
      


Current Term and Year: Spring 2026
Current Term and Year Minus 1: Spring 2025


2025, Unique, All Applicants: 2
2025, Unique, International Applicants: 0
2025, Unique, Domestic Applicants: 2
2025, Total Apps, All Applicants: 2
2025, Total Apps, International Applicants: 0
2025, Total Apps, Domestic Applicants: 2


2024, Unique, All Applicants: 3
2024, Unique, International Applicants: 3
2024, Unique, Domestic Applicants: 0
2024, Total Apps, All Applicants: 3
2024, Total Apps, International Applicants: 3
2024, Total Apps, Domestic Applicants: 0


2023, Unique, All Applicants: 2
2023, Unique, International Applicants: 1
2023, Unique, Domestic Applicants: 1
2023, Total Apps, All Applicants: 2
2023, Total Apps, International Applicants: 1
2023, Total Apps, Domestic Applicants: 1


In [12]:
#Application Details Term 1
# Load the workbook
wb = openpyxl.load_workbook(term_1_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_1_details = pd.read_excel(term_1_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_1_details_total_applications = df_term_1_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_1_details_total_unique_applicants = df_term_1_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_1_details_total_applications}")
print(f"Total unique applicants: {term_1_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_1_details_domestic_applications = df_term_1_details[df_term_1_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_1_details_international_applications = df_term_1_details[df_term_1_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_1_details_domestic_applications}")
print(f"Total International applications: {term_1_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_1_details = df_term_1_details[
    df_term_1_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_1_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_1_details_accepted_applications = len(filtered_df_term_1_details)
print(f"Valid application count: {term_1_details_accepted_applications}")

Total applications: 50
Total unique applicants: 50
Total Domestic applications: 24
Total International applications: 26
Valid application count: 48


In [13]:
#Application Details Term 2
# Load the workbook
wb = openpyxl.load_workbook(term_2_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_2_details = pd.read_excel(term_2_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_2_details_total_applications = df_term_2_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_2_details_total_unique_applicants = df_term_2_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_2_details_total_applications}")
print(f"Total unique applicants: {term_2_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_2_details_domestic_applications = df_term_2_details[df_term_2_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_2_details_international_applications = df_term_2_details[df_term_2_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_2_details_domestic_applications}")
print(f"Total International applications: {term_2_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_2_details = df_term_2_details[
    df_term_2_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_2_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_2_details_accepted_applications = len(filtered_df_term_2_details)
print(f"Valid application count: {term_2_details_accepted_applications}")

Total applications: 1297
Total unique applicants: 1297
Total Domestic applications: 856
Total International applications: 441
Valid application count: 489


In [14]:
#Application Details Term 3
# Load the workbook
wb = openpyxl.load_workbook(term_3_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_3_details = pd.read_excel(term_3_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_3_details_total_applications = df_term_3_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_3_details_total_unique_applicants = df_term_3_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_3_details_total_applications}")
print(f"Total unique applicants: {term_3_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_3_details_domestic_applications = df_term_3_details[df_term_3_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_3_details_international_applications = df_term_3_details[df_term_3_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_3_details_domestic_applications}")
print(f"Total International applications: {term_3_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_3_details = df_term_3_details[
    df_term_3_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_3_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_3_details_accepted_applications = len(filtered_df_term_3_details)
print(f"Valid application count: {term_3_details_accepted_applications}")

Total applications: 33
Total unique applicants: 33
Total Domestic applications: 12
Total International applications: 21
Valid application count: 10


In [15]:
#Application Details Term 4
# Load the workbook
wb = openpyxl.load_workbook(term_4_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_4_details = pd.read_excel(term_4_file_application_details, sheet_name=matched_sheet_name)

# Count total applications (assuming no empty cells)
term_4_details_total_applications = df_term_4_details['Applicants'].count()

# Count unique applicants (assuming the column contains identifiers for each applicant)
term_4_details_total_unique_applicants = df_term_4_details['Applicants'].nunique()

# Output the results
print(f"Total applications: {term_4_details_total_applications}")
print(f"Total unique applicants: {term_4_details_total_unique_applicants}")

# Count how many "Domestic" applications
term_4_details_domestic_applications = df_term_4_details[df_term_4_details['Population'] == 'Domestic'].shape[0]

# Count how many "International" applications
term_4_details_international_applications = df_term_4_details[df_term_4_details['Population'] == 'International'].shape[0]

# Output the results
print(f"Total Domestic applications: {term_4_details_domestic_applications}")
print(f"Total International applications: {term_4_details_international_applications}")

column_Accepted_values = ["ACPT", "COND", "ACLN"]
column_Deferred_Application_values = ["ACPT", "COND"]

# Apply the filters
filtered_df_term_4_details = df_term_4_details[
    df_term_4_details["Accepted"].isin(column_Accepted_values) &
    ~df_term_4_details["Deferred Application"].isin(column_Deferred_Application_values)
]

#Count how many Accepted applications
term_4_details_accepted_applications = len(filtered_df_term_4_details)
print(f"Valid application count: {term_4_details_accepted_applications}")

Total applications: 1
Total unique applicants: 1
Total Domestic applications: 1
Total International applications: 0
Valid application count: 0


In [16]:
#Applications Calculations

#Calculate % Change for term 1 Total Applications
term_1_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_total_applications, term_1_year_minus_one_Total_Apps_All_Applicants)}%"
term_1_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_total_applications, term_1_year_minus_one_Total_Apps_All_Applicants)
try:
    term_1_total_applications_percent_change_number_absolute = np.abs(term_1_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 1 Unique Applications
term_1_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_1_details_total_unique_applicants, term_1_year_minus_one_Unique_All_Applicants)}%"
term_1_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_1_details_total_unique_applicants, term_1_year_minus_one_Unique_All_Applicants)
try:
    term_1_total_applications_percent_change_number_unique_absolute = np.abs(term_1_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_1_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 1 International Applications
term_1_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_international_applications, term_1_year_minus_one_Total_Apps_International_Applicants)}%"
term_1_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_international_applications, term_1_year_minus_one_Total_Apps_International_Applicants)
try:
    term_1_international_applications_percent_change_number_absolute = np.abs(term_1_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 1 Domestic Applications
term_1_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_1_details_domestic_applications, term_1_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_1_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_1_details_domestic_applications, term_1_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_1_domestic_applications_percent_change_number_absolute = np.abs(term_1_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_1_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_1}}
try:
    if term_1_total_applications_percent_change_number_unique >= 0:
        higher_lower_1 = "higher"
    else:
        higher_lower_1 = "lower"
except (TypeError, ValueError): 
    higher_lower_1 = "FAILED TO CALCULATE"

#Calculation for {{term_1_acceptance_percentage_number}}

term_1_acceptance_percentage_number = division_calculator(term_1_details_accepted_applications, term_1_details_total_applications)

#Calculation for {{term_1_international_percentage_number}}

term_1_international_percentage_number = division_calculator(term_1_details_international_applications, term_1_details_total_applications)

#Calculate % Change for term 2 Total Applications
term_2_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_total_applications, term_2_year_minus_one_Total_Apps_All_Applicants)}%"
term_2_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_total_applications, term_2_year_minus_one_Total_Apps_All_Applicants)
try:
    term_2_total_applications_percent_change_number_absolute = np.abs(term_2_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 2 Unique Applications
term_2_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_2_details_total_unique_applicants, term_2_year_minus_one_Unique_All_Applicants)}%"
term_2_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_2_details_total_unique_applicants, term_2_year_minus_one_Unique_All_Applicants)
try:
    term_2_total_applications_percent_change_number_unique_absolute = np.abs(term_2_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_2_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 2 International Applications
term_2_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_international_applications, term_2_year_minus_one_Total_Apps_International_Applicants)}%"
term_2_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_international_applications, term_2_year_minus_one_Total_Apps_International_Applicants)
try:
    term_2_international_applications_percent_change_number_absolute = np.abs(term_2_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 2 Domestic Applications
term_2_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_2_details_domestic_applications, term_2_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_2_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_2_details_domestic_applications, term_2_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_2_domestic_applications_percent_change_number_absolute = np.abs(term_2_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_2_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_2}}
try:
    if term_2_total_applications_percent_change_number_unique >= 0:
        higher_lower_2 = "higher"
    else:
        higher_lower_2 = "lower"
except (TypeError, ValueError): 
    higher_lower_2 = "FAILED TO CALCULATE"

#Calculation for {{term_2_acceptance_percentage_number}}

term_2_acceptance_percentage_number = division_calculator(term_2_details_accepted_applications, term_2_details_total_applications)

#Calculation for {{term_2_international_percentage_number}}

term_2_international_percentage_number = division_calculator(term_2_details_international_applications, term_2_details_total_applications)

#Calculate % Change for term 3 Total Applications
term_3_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_total_applications, term_3_year_minus_one_Total_Apps_All_Applicants)}%"
term_3_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_total_applications, term_3_year_minus_one_Total_Apps_All_Applicants)
try:
    term_3_total_applications_percent_change_number_absolute = np.abs(term_3_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 3 Unique Applications
term_3_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_3_details_total_unique_applicants, term_3_year_minus_one_Unique_All_Applicants)}%"
term_3_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_3_details_total_unique_applicants, term_3_year_minus_one_Unique_All_Applicants)
try:
    term_3_total_applications_percent_change_number_unique_absolute = np.abs(term_3_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_3_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 3 International Applications
term_3_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_international_applications, term_3_year_minus_one_Total_Apps_International_Applicants)}%"
term_3_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_international_applications, term_3_year_minus_one_Total_Apps_International_Applicants)
try:
    term_3_international_applications_percent_change_number_absolute = np.abs(term_3_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 3 Domestic Applications
term_3_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_3_details_domestic_applications, term_3_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_3_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_3_details_domestic_applications, term_3_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_3_domestic_applications_percent_change_number_absolute = np.abs(term_3_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_3_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_3}}
try:
    if term_3_total_applications_percent_change_number_unique >= 0:
        higher_lower_3 = "higher"
    else:
        higher_lower_3 = "lower"

except (TypeError, ValueError): 
    higher_lower_3 = "FAILED TO CALCULATE"

#Calculation for {{term_3_acceptance_percentage_number}}

term_3_acceptance_percentage_number = division_calculator(term_3_details_accepted_applications, term_3_details_total_applications)

#Calculation for {{term_3_international_percentage_number}}

term_3_international_percentage_number = division_calculator(term_3_details_international_applications, term_3_details_total_applications)

#Calculate % Change for term 4 Total Applications
term_4_total_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_total_applications, term_4_year_minus_one_Total_Apps_All_Applicants)}%"
term_4_total_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_total_applications, term_4_year_minus_one_Total_Apps_All_Applicants)
try:
    term_4_total_applications_percent_change_number_absolute = np.abs(term_4_total_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_total_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 4 Unique Applications
term_4_total_applications_percent_change_unique = f"{percent_difference_value_calculator_round_1(term_4_details_total_unique_applicants, term_4_year_minus_one_Unique_All_Applicants)}%"
term_4_total_applications_percent_change_number_unique = percent_difference_value_calculator_round_1(term_4_details_total_unique_applicants, term_4_year_minus_one_Unique_All_Applicants)
try:
    term_4_total_applications_percent_change_number_unique_absolute = np.abs(term_4_total_applications_percent_change_number_unique)
except (TypeError, ValueError):
    term_4_total_applications_percent_change_number_unique_absolute = "N/A"

#Calculate % Change for term 4 International Applications
term_4_international_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_international_applications, term_4_year_minus_one_Total_Apps_International_Applicants)}%"
term_4_international_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_international_applications, term_4_year_minus_one_Total_Apps_International_Applicants)
try:
    term_4_international_applications_percent_change_number_absolute = np.abs(term_4_international_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_international_applications_percent_change_number_absolute = "N/A"

#Calculate % Change for term 4 Domestic Applications
term_4_domestic_applications_percent_change = f"{percent_difference_value_calculator_round_1(term_4_details_domestic_applications, term_4_year_minus_one_Total_Apps_Domestic_Applicants)}%"
term_4_domestic_applications_percent_change_number = percent_difference_value_calculator_round_1(term_4_details_domestic_applications, term_4_year_minus_one_Total_Apps_Domestic_Applicants)
try:
    term_4_domestic_applications_percent_change_number_absolute = np.abs(term_4_domestic_applications_percent_change_number)
except (TypeError, ValueError):
    term_4_domestic_applications_percent_change_number_absolute = "N/A"

#Calculation for {{higher_lower_4}}
try:
    if term_4_total_applications_percent_change_number_unique >= 0:
        higher_lower_4 = "higher"
    else:
        higher_lower_4 = "lower"

except (TypeError, ValueError): 
    higher_lower_4 = "FAILED TO CALCULATE"

#Calculation for {{term_4_acceptance_percentage_number}}

term_4_acceptance_percentage_number = division_calculator(term_4_details_accepted_applications, term_4_details_total_applications)

#Calculation for {{term_4_international_percentage_number}}

term_4_international_percentage_number = division_calculator(term_4_details_international_applications, term_4_details_total_applications)


In [17]:
#Program Specific Applications Calculations Term 1

# Load the workbook
wb = openpyxl.load_workbook(term_1_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_1_details = pd.read_excel(term_1_file_application_details, sheet_name=matched_sheet_name)

# Generate Business Administration variables
term_1_variable_dictionary = {}

# Helper to avoid repetitive filtering
def base_filter(df, program, pop, year):
    return df[
        (df['CURRICULUM'].str.replace(' ', '').str.upper() == program) &
        (df['Population'].str.replace(' ', '').str.upper() == pop) &
        (df['Year'].apply(lambda x: str(int(x)) if not pd.isna(x) else '') == year)
    ]

# Loop through each program, population, and year
for program in program_ID_all:
    for pop in population_values:
        for year in year_values:
            key_base = f"term_1_{program}_{pop}_Year_{year}"
            df_filtered = base_filter(df_term_1_details, program, pop, year)

            applications_count = len(df_filtered) if len(df_filtered) > 0 else None
            term_1_variable_dictionary[f"{key_base}_Applications"] = applications_count

            if applications_count is not None:
                # Accepted — always report count, even if 0
                subset = df_filtered[
                    (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
                    ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
                ]
                term_1_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

                # Payment Pending
                subset = df_filtered[
                    ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
                ]
                term_1_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

                # Denied
                subset = df_filtered[
                    df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
                ]
                term_1_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

                # Declined
                subset = df_filtered[
                    df_filtered['Declined'].isin(['CCAN', 'DECL'])
                ]
                term_1_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

                # Pending
                subset = df_filtered[
                    df_filtered['Pending'].isin(['APRO', 'NODE'])
                ]
                term_1_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

                # Waitlisted
                subset = df_filtered[
                    df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
                ]
                term_1_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

                # Enrolled
                subset = df_filtered[
                    df_filtered['Enrollment'].isin(['ENRO'])
                ]
                enrolled_count = len(subset) if len(subset) > 0 else None
                term_1_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

                # Conversion Rate
                accepted_count = term_1_variable_dictionary[f"{key_base}_Accepted"]
                term_1_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
                    division_calculator_decimal(enrolled_count, accepted_count)
                    if enrolled_count is not None and accepted_count
                    else None
                )
            else:
                # If no applications, set all other metrics to None
                term_1_variable_dictionary[f"{key_base}_Accepted"] = None
                term_1_variable_dictionary[f"{key_base}_Payment_Pending"] = None
                term_1_variable_dictionary[f"{key_base}_Denied"] = None
                term_1_variable_dictionary[f"{key_base}_Declined"] = None
                term_1_variable_dictionary[f"{key_base}_Pending"] = None
                term_1_variable_dictionary[f"{key_base}_Waitlisted"] = None
                term_1_variable_dictionary[f"{key_base}_Enrolled"] = None
                term_1_variable_dictionary[f"{key_base}_Conversion_Rate"] = None


In [18]:
#Program Specific Applications Calculations Term 2

# Load the workbook
wb = openpyxl.load_workbook(term_2_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_2_details = pd.read_excel(term_2_file_application_details, sheet_name=matched_sheet_name)

# Generate Business Administration variables
term_2_variable_dictionary = {}

# Helper to avoid repetitive filtering
def base_filter(df, program, pop, year):
    return df[
        (df['CURRICULUM'].str.replace(' ', '').str.upper() == program) &
        (df['Population'].str.replace(' ', '').str.upper() == pop) &
        (df['Year'].apply(lambda x: str(int(x)) if not pd.isna(x) else '') == year)
    ]

# Loop through each program, population, and year
for program in program_ID_all:
    for pop in population_values:
        for year in year_values:
            key_base = f"term_2_{program}_{pop}_Year_{year}"
            df_filtered = base_filter(df_term_2_details, program, pop, year)

            applications_count = len(df_filtered) if len(df_filtered) > 0 else None
            term_2_variable_dictionary[f"{key_base}_Applications"] = applications_count

            if applications_count is not None:
                # Accepted — always report count, even if 0
                subset = df_filtered[
                    (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
                    ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
                ]
                term_2_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

                # Payment Pending
                subset = df_filtered[
                    ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
                ]
                term_2_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

                # Denied
                subset = df_filtered[
                    df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
                ]
                term_2_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

                # Declined
                subset = df_filtered[
                    df_filtered['Declined'].isin(['CCAN', 'DECL'])
                ]
                term_2_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

                # Pending
                subset = df_filtered[
                    df_filtered['Pending'].isin(['APRO', 'NODE'])
                ]
                term_2_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

                # Waitlisted
                subset = df_filtered[
                    df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
                ]
                term_2_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

                # Enrolled
                subset = df_filtered[
                    df_filtered['Enrollment'].isin(['ENRO'])
                ]
                enrolled_count = len(subset) if len(subset) > 0 else None
                term_2_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

                # Conversion Rate
                accepted_count = term_2_variable_dictionary[f"{key_base}_Accepted"]
                term_2_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
                    division_calculator_decimal(enrolled_count, accepted_count)
                    if enrolled_count is not None and accepted_count
                    else None
                )
            else:
                # If no applications, set all other metrics to None
                term_2_variable_dictionary[f"{key_base}_Accepted"] = None
                term_2_variable_dictionary[f"{key_base}_Payment_Pending"] = None
                term_2_variable_dictionary[f"{key_base}_Denied"] = None
                term_2_variable_dictionary[f"{key_base}_Declined"] = None
                term_2_variable_dictionary[f"{key_base}_Pending"] = None
                term_2_variable_dictionary[f"{key_base}_Waitlisted"] = None
                term_2_variable_dictionary[f"{key_base}_Enrolled"] = None
                term_2_variable_dictionary[f"{key_base}_Conversion_Rate"] = None


In [19]:
print(df_term_2_details['Year'].unique())
print(df_term_2_details['Year'].dtype)

[1 2]
int64


In [20]:
#Program Specific Applications Calculations Term 3

# Load the workbook
wb = openpyxl.load_workbook(term_3_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_3_details = pd.read_excel(term_3_file_application_details, sheet_name=matched_sheet_name)

# Generate Business Administration variables
term_3_variable_dictionary = {}

# Helper to avoid repetitive filtering
def base_filter(df, program, pop, year):
    return df[
        (df['CURRICULUM'].str.replace(' ', '').str.upper() == program) &
        (df['Population'].str.replace(' ', '').str.upper() == pop) &
        (df['Year'].apply(lambda x: str(int(x)) if not pd.isna(x) else '') == year)
    ]

# Loop through each program, population, and year
for program in program_ID_all:
    for pop in population_values:
        for year in year_values:
            key_base = f"term_3_{program}_{pop}_Year_{year}"
            df_filtered = base_filter(df_term_3_details, program, pop, year)

            applications_count = len(df_filtered) if len(df_filtered) > 0 else None
            term_3_variable_dictionary[f"{key_base}_Applications"] = applications_count

            if applications_count is not None:
                # Accepted — always report count, even if 0
                subset = df_filtered[
                    (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
                    ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
                ]
                term_3_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

                # Payment Pending
                subset = df_filtered[
                    ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
                ]
                term_3_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

                # Denied
                subset = df_filtered[
                    df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
                ]
                term_3_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

                # Declined
                subset = df_filtered[
                    df_filtered['Declined'].isin(['CCAN', 'DECL'])
                ]
                term_3_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

                # Pending
                subset = df_filtered[
                    df_filtered['Pending'].isin(['APRO', 'NODE'])
                ]
                term_3_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

                # Waitlisted
                subset = df_filtered[
                    df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
                ]
                term_3_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

                # Enrolled
                subset = df_filtered[
                    df_filtered['Enrollment'].isin(['ENRO'])
                ]
                enrolled_count = len(subset) if len(subset) > 0 else None
                term_3_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

                # Conversion Rate
                accepted_count = term_3_variable_dictionary[f"{key_base}_Accepted"]
                term_3_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
                    division_calculator_decimal(enrolled_count, accepted_count)
                    if enrolled_count is not None and accepted_count
                    else None
                )
            else:
                # If no applications, set all other metrics to None
                term_3_variable_dictionary[f"{key_base}_Accepted"] = None
                term_3_variable_dictionary[f"{key_base}_Payment_Pending"] = None
                term_3_variable_dictionary[f"{key_base}_Denied"] = None
                term_3_variable_dictionary[f"{key_base}_Declined"] = None
                term_3_variable_dictionary[f"{key_base}_Pending"] = None
                term_3_variable_dictionary[f"{key_base}_Waitlisted"] = None
                term_3_variable_dictionary[f"{key_base}_Enrolled"] = None
                term_3_variable_dictionary[f"{key_base}_Conversion_Rate"] = None



In [21]:
#Program Specific Applications Calculations Term 4

# Load the workbook
wb = openpyxl.load_workbook(term_4_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_4_details = pd.read_excel(term_4_file_application_details, sheet_name=matched_sheet_name)

# Generate Business Administration variables
term_4_variable_dictionary = {}

# Helper to avoid repetitive filtering
def base_filter(df, program, pop, year):
    return df[
        (df['CURRICULUM'].str.replace(' ', '').str.upper() == program) &
        (df['Population'].str.replace(' ', '').str.upper() == pop) &
        (df['Year'].apply(lambda x: str(int(x)) if not pd.isna(x) else '') == year)
    ]

# Loop through each program, population, and year
for program in program_ID_all:
    for pop in population_values:
        for year in year_values:
            key_base = f"term_4_{program}_{pop}_Year_{year}"
            df_filtered = base_filter(df_term_4_details, program, pop, year)

            applications_count = len(df_filtered) if len(df_filtered) > 0 else None
            term_4_variable_dictionary[f"{key_base}_Applications"] = applications_count

            if applications_count is not None:
                # Accepted — always report count, even if 0
                subset = df_filtered[
                    (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
                    ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
                ]
                term_4_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

                # Payment Pending
                subset = df_filtered[
                    ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
                ]
                term_4_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

                # Denied
                subset = df_filtered[
                    df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
                ]
                term_4_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

                # Declined
                subset = df_filtered[
                    df_filtered['Declined'].isin(['CCAN', 'DECL'])
                ]
                term_4_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

                # Pending
                subset = df_filtered[
                    df_filtered['Pending'].isin(['APRO', 'NODE'])
                ]
                term_4_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

                # Waitlisted
                subset = df_filtered[
                    df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
                ]
                term_4_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

                # Enrolled
                subset = df_filtered[
                    df_filtered['Enrollment'].isin(['ENRO'])
                ]
                enrolled_count = len(subset) if len(subset) > 0 else None
                term_4_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

                # Conversion Rate
                accepted_count = term_4_variable_dictionary[f"{key_base}_Accepted"]
                term_4_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
                    division_calculator_decimal(enrolled_count, accepted_count)
                    if enrolled_count is not None and accepted_count
                    else None
                )
            else:
                # If no applications, set all other metrics to None
                term_4_variable_dictionary[f"{key_base}_Accepted"] = None
                term_4_variable_dictionary[f"{key_base}_Payment_Pending"] = None
                term_4_variable_dictionary[f"{key_base}_Denied"] = None
                term_4_variable_dictionary[f"{key_base}_Declined"] = None
                term_4_variable_dictionary[f"{key_base}_Pending"] = None
                term_4_variable_dictionary[f"{key_base}_Waitlisted"] = None
                term_4_variable_dictionary[f"{key_base}_Enrolled"] = None
                term_4_variable_dictionary[f"{key_base}_Conversion_Rate"] = None



In [22]:
#Special Nursing Year 3 Calculations

#Term 1
# Special case: NursingYr3 Year 3 (multi-program)
term_1_NursingYr3_variable_dictionary = {}

for pop in population_values:
    year = "3"
    key_base = f"term_1_NursingYr3_{pop}_Year_{year}"
    
    df_filtered = df_term_1_details[
        (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_NursingYr3)) &
        (df_term_1_details['Population'].str.replace(' ', '').str.upper() == pop) &
        (df_term_1_details['Year'].astype(str).str.strip() == year)
    ]

    applications_count = len(df_filtered) if len(df_filtered) > 0 else None
    term_1_NursingYr3_variable_dictionary[f"{key_base}_Applications"] = applications_count

    if applications_count is not None:
        # Accepted — always report count, even if 0
        subset = df_filtered[
            (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
            ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
        ]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

        # Payment Pending
        subset = df_filtered[
            ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
        ]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

        # Denied
        subset = df_filtered[
            df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
        ]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

        # Declined
        subset = df_filtered[
            df_filtered['Declined'].isin(['CCAN', 'DECL'])
        ]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

        # Pending
        subset = df_filtered[
            df_filtered['Pending'].isin(['APRO', 'NODE'])
        ]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

        # Waitlisted
        subset = df_filtered[
            df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
        ]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

        # Enrolled
        subset = df_filtered[
            df_filtered['Enrollment'].isin(['ENRO'])
        ]
        enrolled_count = len(subset) if len(subset) > 0 else None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

        # Conversion Rate
        accepted_count = term_1_NursingYr3_variable_dictionary[f"{key_base}_Accepted"]
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
            division_calculator_decimal(enrolled_count, accepted_count)
            if enrolled_count is not None and accepted_count
            else None
        )
    else:
        # If no applications, set all other metrics to None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = None
        term_1_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = None

#Term 2
# Special case: NursingYr3 Year 3 (multi-program)
term_2_NursingYr3_variable_dictionary = {}

for pop in population_values:
    year = "3"
    key_base = f"term_2_NursingYr3_{pop}_Year_{year}"
    
    df_filtered = df_term_2_details[
        (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_NursingYr3)) &
        (df_term_2_details['Population'].str.replace(' ', '').str.upper() == pop) &
        (df_term_2_details['Year'].astype(str).str.strip() == year)
    ]

    applications_count = len(df_filtered) if len(df_filtered) > 0 else None
    term_2_NursingYr3_variable_dictionary[f"{key_base}_Applications"] = applications_count

    if applications_count is not None:
        # Accepted — always report count, even if 0
        subset = df_filtered[
            (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
            ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
        ]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

        # Payment Pending
        subset = df_filtered[
            ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
        ]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

        # Denied
        subset = df_filtered[
            df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
        ]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

        # Declined
        subset = df_filtered[
            df_filtered['Declined'].isin(['CCAN', 'DECL'])
        ]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

        # Pending
        subset = df_filtered[
            df_filtered['Pending'].isin(['APRO', 'NODE'])
        ]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

        # Waitlisted
        subset = df_filtered[
            df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
        ]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

        # Enrolled
        subset = df_filtered[
            df_filtered['Enrollment'].isin(['ENRO'])
        ]
        enrolled_count = len(subset) if len(subset) > 0 else None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

        # Conversion Rate
        accepted_count = term_2_NursingYr3_variable_dictionary[f"{key_base}_Accepted"]
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
            division_calculator_decimal(enrolled_count, accepted_count)
            if enrolled_count is not None and accepted_count
            else None
        )
    else:
        # If no applications, set all other metrics to None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = None
        term_2_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = None

#Term 3
# Special case: NursingYr3 Year 3 (multi-program)
term_3_NursingYr3_variable_dictionary = {}

for pop in population_values:
    year = "3"
    key_base = f"term_3_NursingYr3_{pop}_Year_{year}"
    
    df_filtered = df_term_3_details[
        (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_NursingYr3)) &
        (df_term_3_details['Population'].str.replace(' ', '').str.upper() == pop) &
        (df_term_3_details['Year'].astype(str).str.strip() == year)
    ]

    applications_count = len(df_filtered) if len(df_filtered) > 0 else None
    term_3_NursingYr3_variable_dictionary[f"{key_base}_Applications"] = applications_count

    if applications_count is not None:
        # Accepted — always report count, even if 0
        subset = df_filtered[
            (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
            ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
        ]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

        # Payment Pending
        subset = df_filtered[
            ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
        ]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

        # Denied
        subset = df_filtered[
            df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
        ]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

        # Declined
        subset = df_filtered[
            df_filtered['Declined'].isin(['CCAN', 'DECL'])
        ]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

        # Pending
        subset = df_filtered[
            df_filtered['Pending'].isin(['APRO', 'NODE'])
        ]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

        # Waitlisted
        subset = df_filtered[
            df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
        ]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

        # Enrolled
        subset = df_filtered[
            df_filtered['Enrollment'].isin(['ENRO'])
        ]
        enrolled_count = len(subset) if len(subset) > 0 else None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

        # Conversion Rate
        accepted_count = term_3_NursingYr3_variable_dictionary[f"{key_base}_Accepted"]
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
            division_calculator_decimal(enrolled_count, accepted_count)
            if enrolled_count is not None and accepted_count
            else None
        )
    else:
        # If no applications, set all other metrics to None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = None
        term_3_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = None

#Term 4
# Special case: NursingYr3 Year 3 (multi-program)
term_4_NursingYr3_variable_dictionary = {}

for pop in population_values:
    year = "3"
    key_base = f"term_4_NursingYr3_{pop}_Year_{year}"
    
    df_filtered = df_term_4_details[
        (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_NursingYr3)) &
        (df_term_4_details['Population'].str.replace(' ', '').str.upper() == pop) &
        (df_term_4_details['Year'].astype(str).str.strip() == year)
    ]

    applications_count = len(df_filtered) if len(df_filtered) > 0 else None
    term_4_NursingYr3_variable_dictionary[f"{key_base}_Applications"] = applications_count

    if applications_count is not None:
        # Accepted — always report count, even if 0
        subset = df_filtered[
            (df_filtered['Accepted'].isin(['ACPT', 'ACLN', 'COND'])) &
            ~(df_filtered['Deferred Application'].isin(['ACPT', 'COND']))
        ]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = len(subset)

        # Payment Pending
        subset = df_filtered[
            ~(df_filtered['Payment Pending'].isna() | (df_filtered['Payment Pending'] == ''))
        ]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = len(subset) if len(subset) > 0 else None

        # Denied
        subset = df_filtered[
            df_filtered['Denied'].isin(['APEX', 'DENY', 'NTSL'])
        ]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = len(subset) if len(subset) > 0 else None

        # Declined
        subset = df_filtered[
            df_filtered['Declined'].isin(['CCAN', 'DECL'])
        ]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = len(subset) if len(subset) > 0 else None

        # Pending
        subset = df_filtered[
            df_filtered['Pending'].isin(['APRO', 'NODE'])
        ]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = len(subset) if len(subset) > 0 else None

        # Waitlisted
        subset = df_filtered[
            df_filtered['Waitlisted'].isin(['CDMT', 'RQMT', 'WAIT'])
        ]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = len(subset) if len(subset) > 0 else None

        # Enrolled
        subset = df_filtered[
            df_filtered['Enrollment'].isin(['ENRO'])
        ]
        enrolled_count = len(subset) if len(subset) > 0 else None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = enrolled_count

        # Conversion Rate
        accepted_count = term_4_NursingYr3_variable_dictionary[f"{key_base}_Accepted"]
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = (
            division_calculator_decimal(enrolled_count, accepted_count)
            if enrolled_count is not None and accepted_count
            else None
        )
    else:
        # If no applications, set all other metrics to None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Accepted"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Payment_Pending"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Denied"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Declined"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Pending"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Waitlisted"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Enrolled"] = None
        term_4_NursingYr3_variable_dictionary[f"{key_base}_Conversion_Rate"] = None


In [23]:
#Term 1 Totals Calculations

# Load the workbook
wb = openpyxl.load_workbook(term_1_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_1_details = pd.read_excel(term_1_file_application_details, sheet_name=matched_sheet_name)

#Business Administration----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_Business_Administration_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Business_Administration))
]
term_1_Total_Business_Administration_Applications = len(df_term_1_Total_Business_Administration_Applications)

#Accepted applications
df_term_1_Total_Business_Administration_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Business_Administration_Accepted = len(df_term_1_Total_Business_Administration_Accepted)

#Payment Pending applications
df_term_1_Total_Business_Administration_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Business_Administration_Payment_Pending = len(df_term_1_Total_Business_Administration_Payment_Pending)

#Denied applications
df_term_1_Total_Business_Administration_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Business_Administration_Denied = len(df_term_1_Total_Business_Administration_Denied)

#Declined applications
df_term_1_Total_Business_Administration_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Business_Administration_Declined = len(df_term_1_Total_Business_Administration_Declined)

#Pending applications
df_term_1_Total_Business_Administration_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Business_Administration_Pending = len(df_term_1_Total_Business_Administration_Pending)

#Waitlisted applications
df_term_1_Total_Business_Administration_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Business_Administration_Waitlisted = len(df_term_1_Total_Business_Administration_Waitlisted)

#Enrolled applications
df_term_1_Total_Business_Administration_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Business_Administration_Enrolled = len(df_term_1_Total_Business_Administration_Enrolled)

#Conversion Rate
if term_1_Total_Business_Administration_Accepted == 0:  
    term_1_Total_Business_Administration_Conversion_Rate = 0
else:
    term_1_Total_Business_Administration_Conversion_Rate = term_1_Total_Business_Administration_Enrolled / term_1_Total_Business_Administration_Accepted

#Education and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_Education_and_Academic_Upgrading_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading))
]
term_1_Total_Education_and_Academic_Upgrading_Applications = len(df_term_1_Total_Education_and_Academic_Upgrading_Applications)

#Accepted applications
df_term_1_Total_Education_and_Academic_Upgrading_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Education_and_Academic_Upgrading_Accepted = len(df_term_1_Total_Education_and_Academic_Upgrading_Accepted)

#Payment Pending applications
df_term_1_Total_Education_and_Academic_Upgrading_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Education_and_Academic_Upgrading_Payment_Pending = len(df_term_1_Total_Education_and_Academic_Upgrading_Payment_Pending)

#Denied applications
df_term_1_Total_Education_and_Academic_Upgrading_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Education_and_Academic_Upgrading_Denied = len(df_term_1_Total_Education_and_Academic_Upgrading_Denied)

#Declined applications
df_term_1_Total_Education_and_Academic_Upgrading_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Education_and_Academic_Upgrading_Declined = len(df_term_1_Total_Education_and_Academic_Upgrading_Declined)

#Pending applications
df_term_1_Total_Education_and_Academic_Upgrading_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Education_and_Academic_Upgrading_Pending = len(df_term_1_Total_Education_and_Academic_Upgrading_Pending)

#Waitlisted applications
df_term_1_Total_Education_and_Academic_Upgrading_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Education_and_Academic_Upgrading_Waitlisted = len(df_term_1_Total_Education_and_Academic_Upgrading_Waitlisted)

#Enrolled applications
df_term_1_Total_Education_and_Academic_Upgrading_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Education_and_Academic_Upgrading_Enrolled = len(df_term_1_Total_Education_and_Academic_Upgrading_Enrolled)

#Conversion Rate
if term_1_Total_Education_and_Academic_Upgrading_Accepted == 0:  
    term_1_Total_Education_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_1_Total_Education_and_Academic_Upgrading_Conversion_Rate = term_1_Total_Education_and_Academic_Upgrading_Enrolled / term_1_Total_Education_and_Academic_Upgrading_Accepted

#University Studies and Environmental Science----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_University_Studies_and_Environmental_Science_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science))
]
term_1_Total_University_Studies_and_Environmental_Science_Applications = len(df_term_1_Total_University_Studies_and_Environmental_Science_Applications)

#Accepted applications
df_term_1_Total_University_Studies_and_Environmental_Science_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_University_Studies_and_Environmental_Science_Accepted = len(df_term_1_Total_University_Studies_and_Environmental_Science_Accepted)

#Payment Pending applications
df_term_1_Total_University_Studies_and_Environmental_Science_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_University_Studies_and_Environmental_Science_Payment_Pending = len(df_term_1_Total_University_Studies_and_Environmental_Science_Payment_Pending)

#Denied applications
df_term_1_Total_University_Studies_and_Environmental_Science_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_University_Studies_and_Environmental_Science_Denied = len(df_term_1_Total_University_Studies_and_Environmental_Science_Denied)

#Declined applications
df_term_1_Total_University_Studies_and_Environmental_Science_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_University_Studies_and_Environmental_Science_Declined = len(df_term_1_Total_University_Studies_and_Environmental_Science_Declined)

#Pending applications
df_term_1_Total_University_Studies_and_Environmental_Science_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_University_Studies_and_Environmental_Science_Pending = len(df_term_1_Total_University_Studies_and_Environmental_Science_Pending)

#Waitlisted applications
df_term_1_Total_University_Studies_and_Environmental_Science_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_University_Studies_and_Environmental_Science_Waitlisted = len(df_term_1_Total_University_Studies_and_Environmental_Science_Waitlisted)

#Enrolled applications
df_term_1_Total_University_Studies_and_Environmental_Science_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_University_Studies_and_Environmental_Science_Enrolled = len(df_term_1_Total_University_Studies_and_Environmental_Science_Enrolled)

#Conversion Rate
if term_1_Total_University_Studies_and_Environmental_Science_Accepted == 0:  
    term_1_Total_University_Studies_and_Environmental_Science_Conversion_Rate = 0
else:
    term_1_Total_University_Studies_and_Environmental_Science_Conversion_Rate = term_1_Total_University_Studies_and_Environmental_Science_Enrolled / term_1_Total_University_Studies_and_Environmental_Science_Accepted

#Language Instruction for Newcomers to Canada----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Applications)

#Accepted applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted)

#Payment Pending applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending)

#Denied applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Denied)

#Declined applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Declined)

#Pending applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Pending)

#Waitlisted applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted)

#Enrolled applications
df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = len(df_term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled)

#Conversion Rate
if term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted == 0:  
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = 0
else:
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled / term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted

#Childhood Studies and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services))
]
term_1_Total_Childhood_Studies_and_Human_Services_Applications = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Applications)

#Accepted applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Childhood_Studies_and_Human_Services_Accepted = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Accepted)

#Payment Pending applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Childhood_Studies_and_Human_Services_Payment_Pending = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Payment_Pending)

#Denied applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Childhood_Studies_and_Human_Services_Denied = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Denied)

#Declined applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Childhood_Studies_and_Human_Services_Declined = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Declined)

#Pending applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Childhood_Studies_and_Human_Services_Pending = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Pending)

#Waitlisted applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Childhood_Studies_and_Human_Services_Waitlisted = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Waitlisted)

#Enrolled applications
df_term_1_Total_Childhood_Studies_and_Human_Services_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Childhood_Studies_and_Human_Services_Enrolled = len(df_term_1_Total_Childhood_Studies_and_Human_Services_Enrolled)

#Conversion Rate
if term_1_Total_Childhood_Studies_and_Human_Services_Accepted == 0:  
    term_1_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = 0
else:
    term_1_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = term_1_Total_Childhood_Studies_and_Human_Services_Enrolled / term_1_Total_Childhood_Studies_and_Human_Services_Accepted

#Nursing and Allied Health----------------------------------------------------------------------------------------------------Done
#Combine program_ID_Nursing_and_Allied_Health and program_ID_NursingYr3 for calculation purpose
program_ID_Nursing_and_Allied_Health_including_NursingYr3 = (program_ID_Nursing_and_Allied_Health + program_ID_NursingYr3)

#Total applications
df_term_1_Total_Nursing_and_Allied_Health_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3))
]
term_1_Total_Nursing_and_Allied_Health_Applications = len(df_term_1_Total_Nursing_and_Allied_Health_Applications)

#Accepted applications
df_term_1_Total_Nursing_and_Allied_Health_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Nursing_and_Allied_Health_Accepted = len(df_term_1_Total_Nursing_and_Allied_Health_Accepted)

#Payment Pending applications
df_term_1_Total_Nursing_and_Allied_Health_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Nursing_and_Allied_Health_Payment_Pending = len(df_term_1_Total_Nursing_and_Allied_Health_Payment_Pending)

#Denied applications
df_term_1_Total_Nursing_and_Allied_Health_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Nursing_and_Allied_Health_Denied = len(df_term_1_Total_Nursing_and_Allied_Health_Denied)

#Declined applications
df_term_1_Total_Nursing_and_Allied_Health_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Nursing_and_Allied_Health_Declined = len(df_term_1_Total_Nursing_and_Allied_Health_Declined)

#Pending applications
df_term_1_Total_Nursing_and_Allied_Health_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Nursing_and_Allied_Health_Pending = len(df_term_1_Total_Nursing_and_Allied_Health_Pending)

#Waitlisted applications
df_term_1_Total_Nursing_and_Allied_Health_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Nursing_and_Allied_Health_Waitlisted = len(df_term_1_Total_Nursing_and_Allied_Health_Waitlisted)

#Enrolled applications
df_term_1_Total_Nursing_and_Allied_Health_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Nursing_and_Allied_Health_Enrolled = len(df_term_1_Total_Nursing_and_Allied_Health_Enrolled)

#Conversion Rate
if term_1_Total_Nursing_and_Allied_Health_Accepted == 0:  
    term_1_Total_Nursing_and_Allied_Health_Conversion_Rate = 0
else:
    term_1_Total_Nursing_and_Allied_Health_Conversion_Rate = term_1_Total_Nursing_and_Allied_Health_Enrolled / term_1_Total_Nursing_and_Allied_Health_Accepted

#Pre-employment----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_Pre_employment_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Pre_employment))
]
term_1_Total_Pre_employment_Applications = len(df_term_1_Total_Pre_employment_Applications)

#Accepted applications
df_term_1_Total_Pre_employment_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Pre_employment_Accepted = len(df_term_1_Total_Pre_employment_Accepted)

#Payment Pending applications
df_term_1_Total_Pre_employment_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Pre_employment_Payment_Pending = len(df_term_1_Total_Pre_employment_Payment_Pending)

#Denied applications
df_term_1_Total_Pre_employment_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Pre_employment_Denied = len(df_term_1_Total_Pre_employment_Denied)

#Declined applications
df_term_1_Total_Pre_employment_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Pre_employment_Declined = len(df_term_1_Total_Pre_employment_Declined)

#Pending applications
df_term_1_Total_Pre_employment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Pre_employment_Pending = len(df_term_1_Total_Pre_employment_Pending)

#Waitlisted applications
df_term_1_Total_Pre_employment_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Pre_employment_Waitlisted = len(df_term_1_Total_Pre_employment_Waitlisted)

#Enrolled applications
df_term_1_Total_Pre_employment_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Pre_employment_Enrolled = len(df_term_1_Total_Pre_employment_Enrolled)

#Conversion Rate
if term_1_Total_Pre_employment_Accepted == 0:  
    term_1_Total_Pre_employment_Conversion_Rate = 0
else:
    term_1_Total_Pre_employment_Conversion_Rate = term_1_Total_Pre_employment_Enrolled / term_1_Total_Pre_employment_Accepted

#Power Engineering----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_1_Total_Power_Engineering_Applications = df_term_1_details[
    (df_term_1_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering))
]
term_1_Total_Power_Engineering_Applications = len(df_term_1_Total_Power_Engineering_Applications)

#Accepted applications
df_term_1_Total_Power_Engineering_Accepted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_1_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_1_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_1_Total_Power_Engineering_Accepted = len(df_term_1_Total_Power_Engineering_Accepted)

#Payment Pending applications
df_term_1_Total_Power_Engineering_Payment_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    ~(df_term_1_details['Payment Pending'].isna() | (df_term_1_details['Payment Pending'] == ''))
]
term_1_Total_Power_Engineering_Payment_Pending = len(df_term_1_Total_Power_Engineering_Payment_Pending)

#Denied applications
df_term_1_Total_Power_Engineering_Denied = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_1_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_1_Total_Power_Engineering_Denied = len(df_term_1_Total_Power_Engineering_Denied)

#Declined applications
df_term_1_Total_Power_Engineering_Declined = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_1_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_1_Total_Power_Engineering_Declined = len(df_term_1_Total_Power_Engineering_Declined)

#Pending applications
df_term_1_Total_Power_Engineering_Pending = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_1_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_1_Total_Power_Engineering_Pending = len(df_term_1_Total_Power_Engineering_Pending)

#Waitlisted applications
df_term_1_Total_Power_Engineering_Waitlisted = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_1_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_1_Total_Power_Engineering_Waitlisted = len(df_term_1_Total_Power_Engineering_Waitlisted)

#Enrolled applications
df_term_1_Total_Power_Engineering_Enrolled = df_term_1_details[
    (df_term_1_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_1_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_1_Total_Power_Engineering_Enrolled = len(df_term_1_Total_Power_Engineering_Enrolled)

#Conversion Rate
if term_1_Total_Power_Engineering_Accepted == 0:  
    term_1_Total_Power_Engineering_Conversion_Rate = 0
else:
    term_1_Total_Power_Engineering_Conversion_Rate = term_1_Total_Power_Engineering_Enrolled / term_1_Total_Power_Engineering_Accepted

#School of University Studies, Career Programs and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications = sum([
    term_1_Total_Business_Administration_Applications,
    term_1_Total_Education_and_Academic_Upgrading_Applications,
    term_1_Total_University_Studies_and_Environmental_Science_Applications,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Applications
])

#Accepted applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted = sum([
    term_1_Total_Business_Administration_Accepted,
    term_1_Total_Education_and_Academic_Upgrading_Accepted,
    term_1_Total_University_Studies_and_Environmental_Science_Accepted,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted
])

#Payment Pending applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending = sum([
    term_1_Total_Business_Administration_Payment_Pending,
    term_1_Total_Education_and_Academic_Upgrading_Payment_Pending,
    term_1_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending
])

#Denied applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied = sum([
    term_1_Total_Business_Administration_Denied,
    term_1_Total_Education_and_Academic_Upgrading_Denied,
    term_1_Total_University_Studies_and_Environmental_Science_Denied,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Denied
])

#Declined applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined = sum([
    term_1_Total_Business_Administration_Declined,
    term_1_Total_Education_and_Academic_Upgrading_Declined,
    term_1_Total_University_Studies_and_Environmental_Science_Declined,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Declined
])

#Pending applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending = sum([
    term_1_Total_Business_Administration_Pending,
    term_1_Total_Education_and_Academic_Upgrading_Pending,
    term_1_Total_University_Studies_and_Environmental_Science_Pending,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Pending
])

#Waitlisted applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted = sum([
    term_1_Total_Business_Administration_Waitlisted,
    term_1_Total_Education_and_Academic_Upgrading_Waitlisted,
    term_1_Total_University_Studies_and_Environmental_Science_Waitlisted,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted
])

#Enrolled applications
term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled = sum([
    term_1_Total_Business_Administration_Enrolled,
    term_1_Total_Education_and_Academic_Upgrading_Enrolled,
    term_1_Total_University_Studies_and_Environmental_Science_Enrolled,
    term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled
])

#Conversion Rate
if term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted == 0:  
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled / term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted

#School of Health and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
term_1_Total_School_of_Health_and_Human_Services_Applications = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Applications,
    term_1_Total_Nursing_and_Allied_Health_Applications
])

#Accepted applications
term_1_Total_School_of_Health_and_Human_Services_Accepted = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Accepted,
    term_1_Total_Nursing_and_Allied_Health_Accepted
])

#Payment Pending applications
term_1_Total_School_of_Health_and_Human_Services_Payment_Pending = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    term_1_Total_Nursing_and_Allied_Health_Payment_Pending
])

#Denied applications
term_1_Total_School_of_Health_and_Human_Services_Denied = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Denied,
    term_1_Total_Nursing_and_Allied_Health_Denied
])

#Declined applications
term_1_Total_School_of_Health_and_Human_Services_Declined = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Declined,
    term_1_Total_Nursing_and_Allied_Health_Declined
])

#Pending applications
term_1_Total_School_of_Health_and_Human_Services_Pending = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Pending,
    term_1_Total_Nursing_and_Allied_Health_Pending
])

#Waitlisted applications
term_1_Total_School_of_Health_and_Human_Services_Waitlisted = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    term_1_Total_Nursing_and_Allied_Health_Waitlisted
])

#Enrolled applications
term_1_Total_School_of_Health_and_Human_Services_Enrolled = sum([
    term_1_Total_Childhood_Studies_and_Human_Services_Enrolled,
    term_1_Total_Nursing_and_Allied_Health_Enrolled
])

#Conversion Rate
if term_1_Total_School_of_Health_and_Human_Services_Accepted == 0:  
    term_1_Total_School_of_Health_and_Human_Services_Conversion_Rate = 0
else:
    term_1_Total_School_of_Health_and_Human_Services_Conversion_Rate = term_1_Total_School_of_Health_and_Human_Services_Enrolled / term_1_Total_School_of_Health_and_Human_Services_Accepted

#School of Trades and Heavy Industrial----------------------------------------------------------------------------------------------------Done
#Total applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Applications = sum([
    term_1_Total_Pre_employment_Applications,
    term_1_Total_Power_Engineering_Applications
])

#Accepted applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Accepted = sum([
    term_1_Total_Pre_employment_Accepted,
    term_1_Total_Power_Engineering_Accepted
])

#Payment Pending applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending = sum([
    term_1_Total_Pre_employment_Payment_Pending,
    term_1_Total_Power_Engineering_Payment_Pending
])

#Denied applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Denied = sum([
    term_1_Total_Pre_employment_Denied,
    term_1_Total_Power_Engineering_Denied
])

#Declined applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Declined = sum([
    term_1_Total_Pre_employment_Declined,
    term_1_Total_Power_Engineering_Declined
])

#Pending applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Pending = sum([
    term_1_Total_Pre_employment_Pending,
    term_1_Total_Power_Engineering_Pending
])

#Waitlisted applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted = sum([
    term_1_Total_Pre_employment_Waitlisted,
    term_1_Total_Power_Engineering_Waitlisted
])

#Enrolled applications
term_1_Total_School_of_Trades_and_Heavy_Industrial_Enrolled = sum([
    term_1_Total_Pre_employment_Enrolled,
    term_1_Total_Power_Engineering_Enrolled
])

#Conversion Rate
if term_1_Total_School_of_Trades_and_Heavy_Industrial_Accepted == 0:  
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = 0
else:
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = term_1_Total_School_of_Trades_and_Heavy_Industrial_Enrolled / term_1_Total_School_of_Trades_and_Heavy_Industrial_Accepted

#Keyano College----------------------------------------------------------------------------------------------------Done
#Total applications
term_1_Keyano_College_Total_Applications = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    term_1_Total_School_of_Health_and_Human_Services_Applications,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Applications
])

#Accepted applications
term_1_Keyano_College_Total_Accepted = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    term_1_Total_School_of_Health_and_Human_Services_Accepted,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Accepted
])

#Payment Pending applications
term_1_Keyano_College_Total_Payment_Pending = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    term_1_Total_School_of_Health_and_Human_Services_Payment_Pending,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending
])

#Denied applications
term_1_Keyano_College_Total_Denied = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    term_1_Total_School_of_Health_and_Human_Services_Denied,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Denied
])

#Declined applications
term_1_Keyano_College_Total_Declined = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    term_1_Total_School_of_Health_and_Human_Services_Declined,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Declined
])

#Pending applications
term_1_Keyano_College_Total_Pending = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    term_1_Total_School_of_Health_and_Human_Services_Pending,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Pending
])

#Waitlisted applications
term_1_Keyano_College_Total_Waitlisted = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    term_1_Total_School_of_Health_and_Human_Services_Waitlisted,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted
])

#Enrolled applications
term_1_Keyano_College_Total_Enrolled = sum([
    term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    term_1_Total_School_of_Health_and_Human_Services_Enrolled,
    term_1_Total_School_of_Trades_and_Heavy_Industrial_Enrolled
])

#Conversion Rate
if term_1_Keyano_College_Total_Accepted == 0:  
    term_1_Keyano_College_Total_Conversion_Rate = 0
else:
    term_1_Keyano_College_Total_Conversion_Rate = term_1_Keyano_College_Total_Enrolled / term_1_Keyano_College_Total_Accepted



In [24]:
#Term 2 Totals Calculations

# Load the workbook
wb = openpyxl.load_workbook(term_2_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_2_details = pd.read_excel(term_2_file_application_details, sheet_name=matched_sheet_name)

#Business Administration----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_Business_Administration_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Business_Administration))
]
term_2_Total_Business_Administration_Applications = len(df_term_2_Total_Business_Administration_Applications)

#Accepted applications
df_term_2_Total_Business_Administration_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Business_Administration_Accepted = len(df_term_2_Total_Business_Administration_Accepted)

#Payment Pending applications
df_term_2_Total_Business_Administration_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Business_Administration_Payment_Pending = len(df_term_2_Total_Business_Administration_Payment_Pending)

#Denied applications
df_term_2_Total_Business_Administration_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Business_Administration_Denied = len(df_term_2_Total_Business_Administration_Denied)

#Declined applications
df_term_2_Total_Business_Administration_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Business_Administration_Declined = len(df_term_2_Total_Business_Administration_Declined)

#Pending applications
df_term_2_Total_Business_Administration_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Business_Administration_Pending = len(df_term_2_Total_Business_Administration_Pending)

#Waitlisted applications
df_term_2_Total_Business_Administration_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Business_Administration_Waitlisted = len(df_term_2_Total_Business_Administration_Waitlisted)

#Enrolled applications
df_term_2_Total_Business_Administration_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Business_Administration_Enrolled = len(df_term_2_Total_Business_Administration_Enrolled)

#Conversion Rate
if term_2_Total_Business_Administration_Accepted == 0:  
    term_2_Total_Business_Administration_Conversion_Rate = 0
else:
    term_2_Total_Business_Administration_Conversion_Rate = term_2_Total_Business_Administration_Enrolled / term_2_Total_Business_Administration_Accepted

#Education and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_Education_and_Academic_Upgrading_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading))
]
term_2_Total_Education_and_Academic_Upgrading_Applications = len(df_term_2_Total_Education_and_Academic_Upgrading_Applications)

#Accepted applications
df_term_2_Total_Education_and_Academic_Upgrading_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Education_and_Academic_Upgrading_Accepted = len(df_term_2_Total_Education_and_Academic_Upgrading_Accepted)

#Payment Pending applications
df_term_2_Total_Education_and_Academic_Upgrading_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Education_and_Academic_Upgrading_Payment_Pending = len(df_term_2_Total_Education_and_Academic_Upgrading_Payment_Pending)

#Denied applications
df_term_2_Total_Education_and_Academic_Upgrading_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Education_and_Academic_Upgrading_Denied = len(df_term_2_Total_Education_and_Academic_Upgrading_Denied)

#Declined applications
df_term_2_Total_Education_and_Academic_Upgrading_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Education_and_Academic_Upgrading_Declined = len(df_term_2_Total_Education_and_Academic_Upgrading_Declined)

#Pending applications
df_term_2_Total_Education_and_Academic_Upgrading_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Education_and_Academic_Upgrading_Pending = len(df_term_2_Total_Education_and_Academic_Upgrading_Pending)

#Waitlisted applications
df_term_2_Total_Education_and_Academic_Upgrading_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Education_and_Academic_Upgrading_Waitlisted = len(df_term_2_Total_Education_and_Academic_Upgrading_Waitlisted)

#Enrolled applications
df_term_2_Total_Education_and_Academic_Upgrading_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Education_and_Academic_Upgrading_Enrolled = len(df_term_2_Total_Education_and_Academic_Upgrading_Enrolled)

#Conversion Rate
if term_2_Total_Education_and_Academic_Upgrading_Accepted == 0:  
    term_2_Total_Education_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_2_Total_Education_and_Academic_Upgrading_Conversion_Rate = term_2_Total_Education_and_Academic_Upgrading_Enrolled / term_2_Total_Education_and_Academic_Upgrading_Accepted

#University Studies and Environmental Science----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_University_Studies_and_Environmental_Science_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science))
]
term_2_Total_University_Studies_and_Environmental_Science_Applications = len(df_term_2_Total_University_Studies_and_Environmental_Science_Applications)

#Accepted applications
df_term_2_Total_University_Studies_and_Environmental_Science_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_University_Studies_and_Environmental_Science_Accepted = len(df_term_2_Total_University_Studies_and_Environmental_Science_Accepted)

#Payment Pending applications
df_term_2_Total_University_Studies_and_Environmental_Science_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_University_Studies_and_Environmental_Science_Payment_Pending = len(df_term_2_Total_University_Studies_and_Environmental_Science_Payment_Pending)

#Denied applications
df_term_2_Total_University_Studies_and_Environmental_Science_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_University_Studies_and_Environmental_Science_Denied = len(df_term_2_Total_University_Studies_and_Environmental_Science_Denied)

#Declined applications
df_term_2_Total_University_Studies_and_Environmental_Science_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_University_Studies_and_Environmental_Science_Declined = len(df_term_2_Total_University_Studies_and_Environmental_Science_Declined)

#Pending applications
df_term_2_Total_University_Studies_and_Environmental_Science_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_University_Studies_and_Environmental_Science_Pending = len(df_term_2_Total_University_Studies_and_Environmental_Science_Pending)

#Waitlisted applications
df_term_2_Total_University_Studies_and_Environmental_Science_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_University_Studies_and_Environmental_Science_Waitlisted = len(df_term_2_Total_University_Studies_and_Environmental_Science_Waitlisted)

#Enrolled applications
df_term_2_Total_University_Studies_and_Environmental_Science_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_University_Studies_and_Environmental_Science_Enrolled = len(df_term_2_Total_University_Studies_and_Environmental_Science_Enrolled)

#Conversion Rate
if term_2_Total_University_Studies_and_Environmental_Science_Accepted == 0:  
    term_2_Total_University_Studies_and_Environmental_Science_Conversion_Rate = 0
else:
    term_2_Total_University_Studies_and_Environmental_Science_Conversion_Rate = term_2_Total_University_Studies_and_Environmental_Science_Enrolled / term_2_Total_University_Studies_and_Environmental_Science_Accepted

#Language Instruction for Newcomers to Canada----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Applications)

#Accepted applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted)

#Payment Pending applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending)

#Denied applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Denied)

#Declined applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Declined)

#Pending applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Pending)

#Waitlisted applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted)

#Enrolled applications
df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = len(df_term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled)

#Conversion Rate
if term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted == 0:  
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = 0
else:
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled / term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted

#Childhood Studies and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services))
]
term_2_Total_Childhood_Studies_and_Human_Services_Applications = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Applications)

#Accepted applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Childhood_Studies_and_Human_Services_Accepted = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Accepted)

#Payment Pending applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Childhood_Studies_and_Human_Services_Payment_Pending = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Payment_Pending)

#Denied applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Childhood_Studies_and_Human_Services_Denied = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Denied)

#Declined applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Childhood_Studies_and_Human_Services_Declined = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Declined)

#Pending applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Childhood_Studies_and_Human_Services_Pending = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Pending)

#Waitlisted applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Childhood_Studies_and_Human_Services_Waitlisted = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Waitlisted)

#Enrolled applications
df_term_2_Total_Childhood_Studies_and_Human_Services_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Childhood_Studies_and_Human_Services_Enrolled = len(df_term_2_Total_Childhood_Studies_and_Human_Services_Enrolled)

#Conversion Rate
if term_2_Total_Childhood_Studies_and_Human_Services_Accepted == 0:  
    term_2_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = 0
else:
    term_2_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = term_2_Total_Childhood_Studies_and_Human_Services_Enrolled / term_2_Total_Childhood_Studies_and_Human_Services_Accepted

#Nursing and Allied Health----------------------------------------------------------------------------------------------------Done
#Combine program_ID_Nursing_and_Allied_Health and program_ID_NursingYr3 for calculation purpose
program_ID_Nursing_and_Allied_Health_including_NursingYr3 = (program_ID_Nursing_and_Allied_Health + program_ID_NursingYr3)

#Total applications
df_term_2_Total_Nursing_and_Allied_Health_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3))
]
term_2_Total_Nursing_and_Allied_Health_Applications = len(df_term_2_Total_Nursing_and_Allied_Health_Applications)

#Accepted applications
df_term_2_Total_Nursing_and_Allied_Health_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Nursing_and_Allied_Health_Accepted = len(df_term_2_Total_Nursing_and_Allied_Health_Accepted)

#Payment Pending applications
df_term_2_Total_Nursing_and_Allied_Health_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Nursing_and_Allied_Health_Payment_Pending = len(df_term_2_Total_Nursing_and_Allied_Health_Payment_Pending)

#Denied applications
df_term_2_Total_Nursing_and_Allied_Health_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Nursing_and_Allied_Health_Denied = len(df_term_2_Total_Nursing_and_Allied_Health_Denied)

#Declined applications
df_term_2_Total_Nursing_and_Allied_Health_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Nursing_and_Allied_Health_Declined = len(df_term_2_Total_Nursing_and_Allied_Health_Declined)

#Pending applications
df_term_2_Total_Nursing_and_Allied_Health_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Nursing_and_Allied_Health_Pending = len(df_term_2_Total_Nursing_and_Allied_Health_Pending)

#Waitlisted applications
df_term_2_Total_Nursing_and_Allied_Health_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Nursing_and_Allied_Health_Waitlisted = len(df_term_2_Total_Nursing_and_Allied_Health_Waitlisted)

#Enrolled applications
df_term_2_Total_Nursing_and_Allied_Health_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Nursing_and_Allied_Health_Enrolled = len(df_term_2_Total_Nursing_and_Allied_Health_Enrolled)

#Conversion Rate
if term_2_Total_Nursing_and_Allied_Health_Accepted == 0:  
    term_2_Total_Nursing_and_Allied_Health_Conversion_Rate = 0
else:
    term_2_Total_Nursing_and_Allied_Health_Conversion_Rate = term_2_Total_Nursing_and_Allied_Health_Enrolled / term_2_Total_Nursing_and_Allied_Health_Accepted

#Pre-employment----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_Pre_employment_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Pre_employment))
]
term_2_Total_Pre_employment_Applications = len(df_term_2_Total_Pre_employment_Applications)

#Accepted applications
df_term_2_Total_Pre_employment_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Pre_employment_Accepted = len(df_term_2_Total_Pre_employment_Accepted)

#Payment Pending applications
df_term_2_Total_Pre_employment_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Pre_employment_Payment_Pending = len(df_term_2_Total_Pre_employment_Payment_Pending)

#Denied applications
df_term_2_Total_Pre_employment_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Pre_employment_Denied = len(df_term_2_Total_Pre_employment_Denied)

#Declined applications
df_term_2_Total_Pre_employment_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Pre_employment_Declined = len(df_term_2_Total_Pre_employment_Declined)

#Pending applications
df_term_2_Total_Pre_employment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Pre_employment_Pending = len(df_term_2_Total_Pre_employment_Pending)

#Waitlisted applications
df_term_2_Total_Pre_employment_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Pre_employment_Waitlisted = len(df_term_2_Total_Pre_employment_Waitlisted)

#Enrolled applications
df_term_2_Total_Pre_employment_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Pre_employment_Enrolled = len(df_term_2_Total_Pre_employment_Enrolled)

#Conversion Rate
if term_2_Total_Pre_employment_Accepted == 0:  
    term_2_Total_Pre_employment_Conversion_Rate = 0
else:
    term_2_Total_Pre_employment_Conversion_Rate = term_2_Total_Pre_employment_Enrolled / term_2_Total_Pre_employment_Accepted

#Power Engineering----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_2_Total_Power_Engineering_Applications = df_term_2_details[
    (df_term_2_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering))
]
term_2_Total_Power_Engineering_Applications = len(df_term_2_Total_Power_Engineering_Applications)

#Accepted applications
df_term_2_Total_Power_Engineering_Accepted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_2_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_2_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_2_Total_Power_Engineering_Accepted = len(df_term_2_Total_Power_Engineering_Accepted)

#Payment Pending applications
df_term_2_Total_Power_Engineering_Payment_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    ~(df_term_2_details['Payment Pending'].isna() | (df_term_2_details['Payment Pending'] == ''))
]
term_2_Total_Power_Engineering_Payment_Pending = len(df_term_2_Total_Power_Engineering_Payment_Pending)

#Denied applications
df_term_2_Total_Power_Engineering_Denied = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_2_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_2_Total_Power_Engineering_Denied = len(df_term_2_Total_Power_Engineering_Denied)

#Declined applications
df_term_2_Total_Power_Engineering_Declined = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_2_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_2_Total_Power_Engineering_Declined = len(df_term_2_Total_Power_Engineering_Declined)

#Pending applications
df_term_2_Total_Power_Engineering_Pending = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_2_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_2_Total_Power_Engineering_Pending = len(df_term_2_Total_Power_Engineering_Pending)

#Waitlisted applications
df_term_2_Total_Power_Engineering_Waitlisted = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_2_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_2_Total_Power_Engineering_Waitlisted = len(df_term_2_Total_Power_Engineering_Waitlisted)

#Enrolled applications
df_term_2_Total_Power_Engineering_Enrolled = df_term_2_details[
    (df_term_2_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_2_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_2_Total_Power_Engineering_Enrolled = len(df_term_2_Total_Power_Engineering_Enrolled)

#Conversion Rate
if term_2_Total_Power_Engineering_Accepted == 0:  
    term_2_Total_Power_Engineering_Conversion_Rate = 0
else:
    term_2_Total_Power_Engineering_Conversion_Rate = term_2_Total_Power_Engineering_Enrolled / term_2_Total_Power_Engineering_Accepted

#School of University Studies, Career Programs and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications = sum([
    term_2_Total_Business_Administration_Applications,
    term_2_Total_Education_and_Academic_Upgrading_Applications,
    term_2_Total_University_Studies_and_Environmental_Science_Applications,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Applications
])

#Accepted applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted = sum([
    term_2_Total_Business_Administration_Accepted,
    term_2_Total_Education_and_Academic_Upgrading_Accepted,
    term_2_Total_University_Studies_and_Environmental_Science_Accepted,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted
])

#Payment Pending applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending = sum([
    term_2_Total_Business_Administration_Payment_Pending,
    term_2_Total_Education_and_Academic_Upgrading_Payment_Pending,
    term_2_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending
])

#Denied applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied = sum([
    term_2_Total_Business_Administration_Denied,
    term_2_Total_Education_and_Academic_Upgrading_Denied,
    term_2_Total_University_Studies_and_Environmental_Science_Denied,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Denied
])

#Declined applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined = sum([
    term_2_Total_Business_Administration_Declined,
    term_2_Total_Education_and_Academic_Upgrading_Declined,
    term_2_Total_University_Studies_and_Environmental_Science_Declined,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Declined
])

#Pending applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending = sum([
    term_2_Total_Business_Administration_Pending,
    term_2_Total_Education_and_Academic_Upgrading_Pending,
    term_2_Total_University_Studies_and_Environmental_Science_Pending,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Pending
])

#Waitlisted applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted = sum([
    term_2_Total_Business_Administration_Waitlisted,
    term_2_Total_Education_and_Academic_Upgrading_Waitlisted,
    term_2_Total_University_Studies_and_Environmental_Science_Waitlisted,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted
])

#Enrolled applications
term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled = sum([
    term_2_Total_Business_Administration_Enrolled,
    term_2_Total_Education_and_Academic_Upgrading_Enrolled,
    term_2_Total_University_Studies_and_Environmental_Science_Enrolled,
    term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled
])

#Conversion Rate
if term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted == 0:  
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled / term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted

#School of Health and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
term_2_Total_School_of_Health_and_Human_Services_Applications = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Applications,
    term_2_Total_Nursing_and_Allied_Health_Applications
])

#Accepted applications
term_2_Total_School_of_Health_and_Human_Services_Accepted = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Accepted,
    term_2_Total_Nursing_and_Allied_Health_Accepted
])

#Payment Pending applications
term_2_Total_School_of_Health_and_Human_Services_Payment_Pending = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    term_2_Total_Nursing_and_Allied_Health_Payment_Pending
])

#Denied applications
term_2_Total_School_of_Health_and_Human_Services_Denied = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Denied,
    term_2_Total_Nursing_and_Allied_Health_Denied
])

#Declined applications
term_2_Total_School_of_Health_and_Human_Services_Declined = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Declined,
    term_2_Total_Nursing_and_Allied_Health_Declined
])

#Pending applications
term_2_Total_School_of_Health_and_Human_Services_Pending = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Pending,
    term_2_Total_Nursing_and_Allied_Health_Pending
])

#Waitlisted applications
term_2_Total_School_of_Health_and_Human_Services_Waitlisted = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    term_2_Total_Nursing_and_Allied_Health_Waitlisted
])

#Enrolled applications
term_2_Total_School_of_Health_and_Human_Services_Enrolled = sum([
    term_2_Total_Childhood_Studies_and_Human_Services_Enrolled,
    term_2_Total_Nursing_and_Allied_Health_Enrolled
])

#Conversion Rate
if term_2_Total_School_of_Health_and_Human_Services_Accepted == 0:  
    term_2_Total_School_of_Health_and_Human_Services_Conversion_Rate = 0
else:
    term_2_Total_School_of_Health_and_Human_Services_Conversion_Rate = term_2_Total_School_of_Health_and_Human_Services_Enrolled / term_2_Total_School_of_Health_and_Human_Services_Accepted

#School of Trades and Heavy Industrial----------------------------------------------------------------------------------------------------Done
#Total applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Applications = sum([
    term_2_Total_Pre_employment_Applications,
    term_2_Total_Power_Engineering_Applications
])

#Accepted applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Accepted = sum([
    term_2_Total_Pre_employment_Accepted,
    term_2_Total_Power_Engineering_Accepted
])

#Payment Pending applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending = sum([
    term_2_Total_Pre_employment_Payment_Pending,
    term_2_Total_Power_Engineering_Payment_Pending
])

#Denied applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Denied = sum([
    term_2_Total_Pre_employment_Denied,
    term_2_Total_Power_Engineering_Denied
])

#Declined applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Declined = sum([
    term_2_Total_Pre_employment_Declined,
    term_2_Total_Power_Engineering_Declined
])

#Pending applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Pending = sum([
    term_2_Total_Pre_employment_Pending,
    term_2_Total_Power_Engineering_Pending
])

#Waitlisted applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted = sum([
    term_2_Total_Pre_employment_Waitlisted,
    term_2_Total_Power_Engineering_Waitlisted
])

#Enrolled applications
term_2_Total_School_of_Trades_and_Heavy_Industrial_Enrolled = sum([
    term_2_Total_Pre_employment_Enrolled,
    term_2_Total_Power_Engineering_Enrolled
])

#Conversion Rate
if term_2_Total_School_of_Trades_and_Heavy_Industrial_Accepted == 0:  
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = 0
else:
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = term_2_Total_School_of_Trades_and_Heavy_Industrial_Enrolled / term_2_Total_School_of_Trades_and_Heavy_Industrial_Accepted

#Keyano College----------------------------------------------------------------------------------------------------Done
#Total applications
term_2_Keyano_College_Total_Applications = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    term_2_Total_School_of_Health_and_Human_Services_Applications,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Applications
])

#Accepted applications
term_2_Keyano_College_Total_Accepted = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    term_2_Total_School_of_Health_and_Human_Services_Accepted,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Accepted
])

#Payment Pending applications
term_2_Keyano_College_Total_Payment_Pending = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    term_2_Total_School_of_Health_and_Human_Services_Payment_Pending,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending
])

#Denied applications
term_2_Keyano_College_Total_Denied = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    term_2_Total_School_of_Health_and_Human_Services_Denied,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Denied
])

#Declined applications
term_2_Keyano_College_Total_Declined = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    term_2_Total_School_of_Health_and_Human_Services_Declined,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Declined
])

#Pending applications
term_2_Keyano_College_Total_Pending = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    term_2_Total_School_of_Health_and_Human_Services_Pending,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Pending
])

#Waitlisted applications
term_2_Keyano_College_Total_Waitlisted = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    term_2_Total_School_of_Health_and_Human_Services_Waitlisted,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted
])

#Enrolled applications
term_2_Keyano_College_Total_Enrolled = sum([
    term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    term_2_Total_School_of_Health_and_Human_Services_Enrolled,
    term_2_Total_School_of_Trades_and_Heavy_Industrial_Enrolled
])

#Conversion Rate
if term_2_Keyano_College_Total_Accepted == 0:  
    term_2_Keyano_College_Total_Conversion_Rate = 0
else:
    term_2_Keyano_College_Total_Conversion_Rate = term_2_Keyano_College_Total_Enrolled / term_2_Keyano_College_Total_Accepted



In [25]:
#Term 3 Totals Calculations

# Load the workbook
wb = openpyxl.load_workbook(term_3_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_3_details = pd.read_excel(term_3_file_application_details, sheet_name=matched_sheet_name)

#Business Administration----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_Business_Administration_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Business_Administration))
]
term_3_Total_Business_Administration_Applications = len(df_term_3_Total_Business_Administration_Applications)

#Accepted applications
df_term_3_Total_Business_Administration_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Business_Administration_Accepted = len(df_term_3_Total_Business_Administration_Accepted)

#Payment Pending applications
df_term_3_Total_Business_Administration_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Business_Administration_Payment_Pending = len(df_term_3_Total_Business_Administration_Payment_Pending)

#Denied applications
df_term_3_Total_Business_Administration_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Business_Administration_Denied = len(df_term_3_Total_Business_Administration_Denied)

#Declined applications
df_term_3_Total_Business_Administration_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Business_Administration_Declined = len(df_term_3_Total_Business_Administration_Declined)

#Pending applications
df_term_3_Total_Business_Administration_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Business_Administration_Pending = len(df_term_3_Total_Business_Administration_Pending)

#Waitlisted applications
df_term_3_Total_Business_Administration_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Business_Administration_Waitlisted = len(df_term_3_Total_Business_Administration_Waitlisted)

#Enrolled applications
df_term_3_Total_Business_Administration_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Business_Administration_Enrolled = len(df_term_3_Total_Business_Administration_Enrolled)

#Conversion Rate
if term_3_Total_Business_Administration_Accepted == 0:  
    term_3_Total_Business_Administration_Conversion_Rate = 0
else:
    term_3_Total_Business_Administration_Conversion_Rate = term_3_Total_Business_Administration_Enrolled / term_3_Total_Business_Administration_Accepted

#Education and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_Education_and_Academic_Upgrading_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading))
]
term_3_Total_Education_and_Academic_Upgrading_Applications = len(df_term_3_Total_Education_and_Academic_Upgrading_Applications)

#Accepted applications
df_term_3_Total_Education_and_Academic_Upgrading_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Education_and_Academic_Upgrading_Accepted = len(df_term_3_Total_Education_and_Academic_Upgrading_Accepted)

#Payment Pending applications
df_term_3_Total_Education_and_Academic_Upgrading_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Education_and_Academic_Upgrading_Payment_Pending = len(df_term_3_Total_Education_and_Academic_Upgrading_Payment_Pending)

#Denied applications
df_term_3_Total_Education_and_Academic_Upgrading_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Education_and_Academic_Upgrading_Denied = len(df_term_3_Total_Education_and_Academic_Upgrading_Denied)

#Declined applications
df_term_3_Total_Education_and_Academic_Upgrading_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Education_and_Academic_Upgrading_Declined = len(df_term_3_Total_Education_and_Academic_Upgrading_Declined)

#Pending applications
df_term_3_Total_Education_and_Academic_Upgrading_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Education_and_Academic_Upgrading_Pending = len(df_term_3_Total_Education_and_Academic_Upgrading_Pending)

#Waitlisted applications
df_term_3_Total_Education_and_Academic_Upgrading_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Education_and_Academic_Upgrading_Waitlisted = len(df_term_3_Total_Education_and_Academic_Upgrading_Waitlisted)

#Enrolled applications
df_term_3_Total_Education_and_Academic_Upgrading_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Education_and_Academic_Upgrading_Enrolled = len(df_term_3_Total_Education_and_Academic_Upgrading_Enrolled)

#Conversion Rate
if term_3_Total_Education_and_Academic_Upgrading_Accepted == 0:  
    term_3_Total_Education_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_3_Total_Education_and_Academic_Upgrading_Conversion_Rate = term_3_Total_Education_and_Academic_Upgrading_Enrolled / term_3_Total_Education_and_Academic_Upgrading_Accepted

#University Studies and Environmental Science----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_University_Studies_and_Environmental_Science_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science))
]
term_3_Total_University_Studies_and_Environmental_Science_Applications = len(df_term_3_Total_University_Studies_and_Environmental_Science_Applications)

#Accepted applications
df_term_3_Total_University_Studies_and_Environmental_Science_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_University_Studies_and_Environmental_Science_Accepted = len(df_term_3_Total_University_Studies_and_Environmental_Science_Accepted)

#Payment Pending applications
df_term_3_Total_University_Studies_and_Environmental_Science_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_University_Studies_and_Environmental_Science_Payment_Pending = len(df_term_3_Total_University_Studies_and_Environmental_Science_Payment_Pending)

#Denied applications
df_term_3_Total_University_Studies_and_Environmental_Science_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_University_Studies_and_Environmental_Science_Denied = len(df_term_3_Total_University_Studies_and_Environmental_Science_Denied)

#Declined applications
df_term_3_Total_University_Studies_and_Environmental_Science_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_University_Studies_and_Environmental_Science_Declined = len(df_term_3_Total_University_Studies_and_Environmental_Science_Declined)

#Pending applications
df_term_3_Total_University_Studies_and_Environmental_Science_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_University_Studies_and_Environmental_Science_Pending = len(df_term_3_Total_University_Studies_and_Environmental_Science_Pending)

#Waitlisted applications
df_term_3_Total_University_Studies_and_Environmental_Science_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_University_Studies_and_Environmental_Science_Waitlisted = len(df_term_3_Total_University_Studies_and_Environmental_Science_Waitlisted)

#Enrolled applications
df_term_3_Total_University_Studies_and_Environmental_Science_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_University_Studies_and_Environmental_Science_Enrolled = len(df_term_3_Total_University_Studies_and_Environmental_Science_Enrolled)

#Conversion Rate
if term_3_Total_University_Studies_and_Environmental_Science_Accepted == 0:  
    term_3_Total_University_Studies_and_Environmental_Science_Conversion_Rate = 0
else:
    term_3_Total_University_Studies_and_Environmental_Science_Conversion_Rate = term_3_Total_University_Studies_and_Environmental_Science_Enrolled / term_3_Total_University_Studies_and_Environmental_Science_Accepted

#Language Instruction for Newcomers to Canada----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Applications)

#Accepted applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted)

#Payment Pending applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending)

#Denied applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Denied)

#Declined applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Declined)

#Pending applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Pending)

#Waitlisted applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted)

#Enrolled applications
df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = len(df_term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled)

#Conversion Rate
if term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted == 0:  
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = 0
else:
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled / term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted

#Childhood Studies and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services))
]
term_3_Total_Childhood_Studies_and_Human_Services_Applications = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Applications)

#Accepted applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Childhood_Studies_and_Human_Services_Accepted = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Accepted)

#Payment Pending applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Childhood_Studies_and_Human_Services_Payment_Pending = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Payment_Pending)

#Denied applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Childhood_Studies_and_Human_Services_Denied = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Denied)

#Declined applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Childhood_Studies_and_Human_Services_Declined = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Declined)

#Pending applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Childhood_Studies_and_Human_Services_Pending = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Pending)

#Waitlisted applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Childhood_Studies_and_Human_Services_Waitlisted = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Waitlisted)

#Enrolled applications
df_term_3_Total_Childhood_Studies_and_Human_Services_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Childhood_Studies_and_Human_Services_Enrolled = len(df_term_3_Total_Childhood_Studies_and_Human_Services_Enrolled)

#Conversion Rate
if term_3_Total_Childhood_Studies_and_Human_Services_Accepted == 0:  
    term_3_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = 0
else:
    term_3_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = term_3_Total_Childhood_Studies_and_Human_Services_Enrolled / term_3_Total_Childhood_Studies_and_Human_Services_Accepted

#Nursing and Allied Health----------------------------------------------------------------------------------------------------Done
#Combine program_ID_Nursing_and_Allied_Health and program_ID_NursingYr3 for calculation purpose
program_ID_Nursing_and_Allied_Health_including_NursingYr3 = (program_ID_Nursing_and_Allied_Health + program_ID_NursingYr3)

#Total applications
df_term_3_Total_Nursing_and_Allied_Health_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3))
]
term_3_Total_Nursing_and_Allied_Health_Applications = len(df_term_3_Total_Nursing_and_Allied_Health_Applications)

#Accepted applications
df_term_3_Total_Nursing_and_Allied_Health_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Nursing_and_Allied_Health_Accepted = len(df_term_3_Total_Nursing_and_Allied_Health_Accepted)

#Payment Pending applications
df_term_3_Total_Nursing_and_Allied_Health_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Nursing_and_Allied_Health_Payment_Pending = len(df_term_3_Total_Nursing_and_Allied_Health_Payment_Pending)

#Denied applications
df_term_3_Total_Nursing_and_Allied_Health_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Nursing_and_Allied_Health_Denied = len(df_term_3_Total_Nursing_and_Allied_Health_Denied)

#Declined applications
df_term_3_Total_Nursing_and_Allied_Health_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Nursing_and_Allied_Health_Declined = len(df_term_3_Total_Nursing_and_Allied_Health_Declined)

#Pending applications
df_term_3_Total_Nursing_and_Allied_Health_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Nursing_and_Allied_Health_Pending = len(df_term_3_Total_Nursing_and_Allied_Health_Pending)

#Waitlisted applications
df_term_3_Total_Nursing_and_Allied_Health_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Nursing_and_Allied_Health_Waitlisted = len(df_term_3_Total_Nursing_and_Allied_Health_Waitlisted)

#Enrolled applications
df_term_3_Total_Nursing_and_Allied_Health_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Nursing_and_Allied_Health_Enrolled = len(df_term_3_Total_Nursing_and_Allied_Health_Enrolled)

#Conversion Rate
if term_3_Total_Nursing_and_Allied_Health_Accepted == 0:  
    term_3_Total_Nursing_and_Allied_Health_Conversion_Rate = 0
else:
    term_3_Total_Nursing_and_Allied_Health_Conversion_Rate = term_3_Total_Nursing_and_Allied_Health_Enrolled / term_3_Total_Nursing_and_Allied_Health_Accepted

#Pre-employment----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_Pre_employment_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Pre_employment))
]
term_3_Total_Pre_employment_Applications = len(df_term_3_Total_Pre_employment_Applications)

#Accepted applications
df_term_3_Total_Pre_employment_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Pre_employment_Accepted = len(df_term_3_Total_Pre_employment_Accepted)

#Payment Pending applications
df_term_3_Total_Pre_employment_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Pre_employment_Payment_Pending = len(df_term_3_Total_Pre_employment_Payment_Pending)

#Denied applications
df_term_3_Total_Pre_employment_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Pre_employment_Denied = len(df_term_3_Total_Pre_employment_Denied)

#Declined applications
df_term_3_Total_Pre_employment_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Pre_employment_Declined = len(df_term_3_Total_Pre_employment_Declined)

#Pending applications
df_term_3_Total_Pre_employment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Pre_employment_Pending = len(df_term_3_Total_Pre_employment_Pending)

#Waitlisted applications
df_term_3_Total_Pre_employment_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Pre_employment_Waitlisted = len(df_term_3_Total_Pre_employment_Waitlisted)

#Enrolled applications
df_term_3_Total_Pre_employment_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Pre_employment_Enrolled = len(df_term_3_Total_Pre_employment_Enrolled)

#Conversion Rate
if term_3_Total_Pre_employment_Accepted == 0:  
    term_3_Total_Pre_employment_Conversion_Rate = 0
else:
    term_3_Total_Pre_employment_Conversion_Rate = term_3_Total_Pre_employment_Enrolled / term_3_Total_Pre_employment_Accepted

#Power Engineering----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_3_Total_Power_Engineering_Applications = df_term_3_details[
    (df_term_3_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering))
]
term_3_Total_Power_Engineering_Applications = len(df_term_3_Total_Power_Engineering_Applications)

#Accepted applications
df_term_3_Total_Power_Engineering_Accepted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_3_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_3_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_3_Total_Power_Engineering_Accepted = len(df_term_3_Total_Power_Engineering_Accepted)

#Payment Pending applications
df_term_3_Total_Power_Engineering_Payment_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    ~(df_term_3_details['Payment Pending'].isna() | (df_term_3_details['Payment Pending'] == ''))
]
term_3_Total_Power_Engineering_Payment_Pending = len(df_term_3_Total_Power_Engineering_Payment_Pending)

#Denied applications
df_term_3_Total_Power_Engineering_Denied = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_3_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_3_Total_Power_Engineering_Denied = len(df_term_3_Total_Power_Engineering_Denied)

#Declined applications
df_term_3_Total_Power_Engineering_Declined = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_3_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_3_Total_Power_Engineering_Declined = len(df_term_3_Total_Power_Engineering_Declined)

#Pending applications
df_term_3_Total_Power_Engineering_Pending = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_3_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_3_Total_Power_Engineering_Pending = len(df_term_3_Total_Power_Engineering_Pending)

#Waitlisted applications
df_term_3_Total_Power_Engineering_Waitlisted = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_3_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_3_Total_Power_Engineering_Waitlisted = len(df_term_3_Total_Power_Engineering_Waitlisted)

#Enrolled applications
df_term_3_Total_Power_Engineering_Enrolled = df_term_3_details[
    (df_term_3_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_3_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_3_Total_Power_Engineering_Enrolled = len(df_term_3_Total_Power_Engineering_Enrolled)

#Conversion Rate
if term_3_Total_Power_Engineering_Accepted == 0:  
    term_3_Total_Power_Engineering_Conversion_Rate = 0
else:
    term_3_Total_Power_Engineering_Conversion_Rate = term_3_Total_Power_Engineering_Enrolled / term_3_Total_Power_Engineering_Accepted

#School of University Studies, Career Programs and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications = sum([
    term_3_Total_Business_Administration_Applications,
    term_3_Total_Education_and_Academic_Upgrading_Applications,
    term_3_Total_University_Studies_and_Environmental_Science_Applications,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Applications
])

#Accepted applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted = sum([
    term_3_Total_Business_Administration_Accepted,
    term_3_Total_Education_and_Academic_Upgrading_Accepted,
    term_3_Total_University_Studies_and_Environmental_Science_Accepted,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted
])

#Payment Pending applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending = sum([
    term_3_Total_Business_Administration_Payment_Pending,
    term_3_Total_Education_and_Academic_Upgrading_Payment_Pending,
    term_3_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending
])

#Denied applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied = sum([
    term_3_Total_Business_Administration_Denied,
    term_3_Total_Education_and_Academic_Upgrading_Denied,
    term_3_Total_University_Studies_and_Environmental_Science_Denied,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Denied
])

#Declined applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined = sum([
    term_3_Total_Business_Administration_Declined,
    term_3_Total_Education_and_Academic_Upgrading_Declined,
    term_3_Total_University_Studies_and_Environmental_Science_Declined,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Declined
])

#Pending applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending = sum([
    term_3_Total_Business_Administration_Pending,
    term_3_Total_Education_and_Academic_Upgrading_Pending,
    term_3_Total_University_Studies_and_Environmental_Science_Pending,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Pending
])

#Waitlisted applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted = sum([
    term_3_Total_Business_Administration_Waitlisted,
    term_3_Total_Education_and_Academic_Upgrading_Waitlisted,
    term_3_Total_University_Studies_and_Environmental_Science_Waitlisted,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted
])

#Enrolled applications
term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled = sum([
    term_3_Total_Business_Administration_Enrolled,
    term_3_Total_Education_and_Academic_Upgrading_Enrolled,
    term_3_Total_University_Studies_and_Environmental_Science_Enrolled,
    term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled
])

#Conversion Rate
if term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted == 0:  
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled / term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted

#School of Health and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
term_3_Total_School_of_Health_and_Human_Services_Applications = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Applications,
    term_3_Total_Nursing_and_Allied_Health_Applications
])

#Accepted applications
term_3_Total_School_of_Health_and_Human_Services_Accepted = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Accepted,
    term_3_Total_Nursing_and_Allied_Health_Accepted
])

#Payment Pending applications
term_3_Total_School_of_Health_and_Human_Services_Payment_Pending = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    term_3_Total_Nursing_and_Allied_Health_Payment_Pending
])

#Denied applications
term_3_Total_School_of_Health_and_Human_Services_Denied = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Denied,
    term_3_Total_Nursing_and_Allied_Health_Denied
])

#Declined applications
term_3_Total_School_of_Health_and_Human_Services_Declined = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Declined,
    term_3_Total_Nursing_and_Allied_Health_Declined
])

#Pending applications
term_3_Total_School_of_Health_and_Human_Services_Pending = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Pending,
    term_3_Total_Nursing_and_Allied_Health_Pending
])

#Waitlisted applications
term_3_Total_School_of_Health_and_Human_Services_Waitlisted = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    term_3_Total_Nursing_and_Allied_Health_Waitlisted
])

#Enrolled applications
term_3_Total_School_of_Health_and_Human_Services_Enrolled = sum([
    term_3_Total_Childhood_Studies_and_Human_Services_Enrolled,
    term_3_Total_Nursing_and_Allied_Health_Enrolled
])

#Conversion Rate
if term_3_Total_School_of_Health_and_Human_Services_Accepted == 0:  
    term_3_Total_School_of_Health_and_Human_Services_Conversion_Rate = 0
else:
    term_3_Total_School_of_Health_and_Human_Services_Conversion_Rate = term_3_Total_School_of_Health_and_Human_Services_Enrolled / term_3_Total_School_of_Health_and_Human_Services_Accepted

#School of Trades and Heavy Industrial----------------------------------------------------------------------------------------------------Done
#Total applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Applications = sum([
    term_3_Total_Pre_employment_Applications,
    term_3_Total_Power_Engineering_Applications
])

#Accepted applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Accepted = sum([
    term_3_Total_Pre_employment_Accepted,
    term_3_Total_Power_Engineering_Accepted
])

#Payment Pending applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending = sum([
    term_3_Total_Pre_employment_Payment_Pending,
    term_3_Total_Power_Engineering_Payment_Pending
])

#Denied applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Denied = sum([
    term_3_Total_Pre_employment_Denied,
    term_3_Total_Power_Engineering_Denied
])

#Declined applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Declined = sum([
    term_3_Total_Pre_employment_Declined,
    term_3_Total_Power_Engineering_Declined
])

#Pending applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Pending = sum([
    term_3_Total_Pre_employment_Pending,
    term_3_Total_Power_Engineering_Pending
])

#Waitlisted applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted = sum([
    term_3_Total_Pre_employment_Waitlisted,
    term_3_Total_Power_Engineering_Waitlisted
])

#Enrolled applications
term_3_Total_School_of_Trades_and_Heavy_Industrial_Enrolled = sum([
    term_3_Total_Pre_employment_Enrolled,
    term_3_Total_Power_Engineering_Enrolled
])

#Conversion Rate
if term_3_Total_School_of_Trades_and_Heavy_Industrial_Accepted == 0:  
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = 0
else:
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = term_3_Total_School_of_Trades_and_Heavy_Industrial_Enrolled / term_3_Total_School_of_Trades_and_Heavy_Industrial_Accepted

#Keyano College----------------------------------------------------------------------------------------------------Done
#Total applications
term_3_Keyano_College_Total_Applications = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    term_3_Total_School_of_Health_and_Human_Services_Applications,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Applications
])

#Accepted applications
term_3_Keyano_College_Total_Accepted = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    term_3_Total_School_of_Health_and_Human_Services_Accepted,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Accepted
])

#Payment Pending applications
term_3_Keyano_College_Total_Payment_Pending = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    term_3_Total_School_of_Health_and_Human_Services_Payment_Pending,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending
])

#Denied applications
term_3_Keyano_College_Total_Denied = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    term_3_Total_School_of_Health_and_Human_Services_Denied,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Denied
])

#Declined applications
term_3_Keyano_College_Total_Declined = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    term_3_Total_School_of_Health_and_Human_Services_Declined,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Declined
])

#Pending applications
term_3_Keyano_College_Total_Pending = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    term_3_Total_School_of_Health_and_Human_Services_Pending,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Pending
])

#Waitlisted applications
term_3_Keyano_College_Total_Waitlisted = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    term_3_Total_School_of_Health_and_Human_Services_Waitlisted,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted
])

#Enrolled applications
term_3_Keyano_College_Total_Enrolled = sum([
    term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    term_3_Total_School_of_Health_and_Human_Services_Enrolled,
    term_3_Total_School_of_Trades_and_Heavy_Industrial_Enrolled
])

#Conversion Rate
if term_3_Keyano_College_Total_Accepted == 0:  
    term_3_Keyano_College_Total_Conversion_Rate = 0
else:
    term_3_Keyano_College_Total_Conversion_Rate = term_3_Keyano_College_Total_Enrolled / term_3_Keyano_College_Total_Accepted



In [26]:
#Term 4 Totals Calculations

# Load the workbook
wb = openpyxl.load_workbook(term_4_file_application_details, data_only=True)

# Try to match any sheet that resembles "Clean Data"
matched_sheet_name = None
for sheet_name in wb.sheetnames:
    normalized = sheet_name.lower().replace(" ", "")
    if normalized == "cleandata":
        matched_sheet_name = sheet_name
        break

if matched_sheet_name is None:
    raise ValueError(f"No sheet matching 'Clean Data' found. Available sheets: {wb.sheetnames}")

# Load the matched sheet into a pandas DataFrame
df_term_4_details = pd.read_excel(term_4_file_application_details, sheet_name=matched_sheet_name)

#Business Administration----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_Business_Administration_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Business_Administration))
]
term_4_Total_Business_Administration_Applications = len(df_term_4_Total_Business_Administration_Applications)

#Accepted applications
df_term_4_Total_Business_Administration_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Business_Administration_Accepted = len(df_term_4_Total_Business_Administration_Accepted)

#Payment Pending applications
df_term_4_Total_Business_Administration_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Business_Administration_Payment_Pending = len(df_term_4_Total_Business_Administration_Payment_Pending)

#Denied applications
df_term_4_Total_Business_Administration_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Business_Administration_Denied = len(df_term_4_Total_Business_Administration_Denied)

#Declined applications
df_term_4_Total_Business_Administration_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Business_Administration_Declined = len(df_term_4_Total_Business_Administration_Declined)

#Pending applications
df_term_4_Total_Business_Administration_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Business_Administration_Pending = len(df_term_4_Total_Business_Administration_Pending)

#Waitlisted applications
df_term_4_Total_Business_Administration_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Business_Administration_Waitlisted = len(df_term_4_Total_Business_Administration_Waitlisted)

#Enrolled applications
df_term_4_Total_Business_Administration_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Business_Administration)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Business_Administration_Enrolled = len(df_term_4_Total_Business_Administration_Enrolled)

#Conversion Rate
if term_4_Total_Business_Administration_Accepted == 0:  
    term_4_Total_Business_Administration_Conversion_Rate = 0
else:
    term_4_Total_Business_Administration_Conversion_Rate = term_4_Total_Business_Administration_Enrolled / term_4_Total_Business_Administration_Accepted

#Education and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_Education_and_Academic_Upgrading_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading))
]
term_4_Total_Education_and_Academic_Upgrading_Applications = len(df_term_4_Total_Education_and_Academic_Upgrading_Applications)

#Accepted applications
df_term_4_Total_Education_and_Academic_Upgrading_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Education_and_Academic_Upgrading_Accepted = len(df_term_4_Total_Education_and_Academic_Upgrading_Accepted)

#Payment Pending applications
df_term_4_Total_Education_and_Academic_Upgrading_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Education_and_Academic_Upgrading_Payment_Pending = len(df_term_4_Total_Education_and_Academic_Upgrading_Payment_Pending)

#Denied applications
df_term_4_Total_Education_and_Academic_Upgrading_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Education_and_Academic_Upgrading_Denied = len(df_term_4_Total_Education_and_Academic_Upgrading_Denied)

#Declined applications
df_term_4_Total_Education_and_Academic_Upgrading_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Education_and_Academic_Upgrading_Declined = len(df_term_4_Total_Education_and_Academic_Upgrading_Declined)

#Pending applications
df_term_4_Total_Education_and_Academic_Upgrading_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Education_and_Academic_Upgrading_Pending = len(df_term_4_Total_Education_and_Academic_Upgrading_Pending)

#Waitlisted applications
df_term_4_Total_Education_and_Academic_Upgrading_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Education_and_Academic_Upgrading_Waitlisted = len(df_term_4_Total_Education_and_Academic_Upgrading_Waitlisted)

#Enrolled applications
df_term_4_Total_Education_and_Academic_Upgrading_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Education_and_Academic_Upgrading)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Education_and_Academic_Upgrading_Enrolled = len(df_term_4_Total_Education_and_Academic_Upgrading_Enrolled)

#Conversion Rate
if term_4_Total_Education_and_Academic_Upgrading_Accepted == 0:  
    term_4_Total_Education_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_4_Total_Education_and_Academic_Upgrading_Conversion_Rate = term_4_Total_Education_and_Academic_Upgrading_Enrolled / term_4_Total_Education_and_Academic_Upgrading_Accepted

#University Studies and Environmental Science----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_University_Studies_and_Environmental_Science_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science))
]
term_4_Total_University_Studies_and_Environmental_Science_Applications = len(df_term_4_Total_University_Studies_and_Environmental_Science_Applications)

#Accepted applications
df_term_4_Total_University_Studies_and_Environmental_Science_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_University_Studies_and_Environmental_Science_Accepted = len(df_term_4_Total_University_Studies_and_Environmental_Science_Accepted)

#Payment Pending applications
df_term_4_Total_University_Studies_and_Environmental_Science_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_University_Studies_and_Environmental_Science_Payment_Pending = len(df_term_4_Total_University_Studies_and_Environmental_Science_Payment_Pending)

#Denied applications
df_term_4_Total_University_Studies_and_Environmental_Science_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_University_Studies_and_Environmental_Science_Denied = len(df_term_4_Total_University_Studies_and_Environmental_Science_Denied)

#Declined applications
df_term_4_Total_University_Studies_and_Environmental_Science_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_University_Studies_and_Environmental_Science_Declined = len(df_term_4_Total_University_Studies_and_Environmental_Science_Declined)

#Pending applications
df_term_4_Total_University_Studies_and_Environmental_Science_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_University_Studies_and_Environmental_Science_Pending = len(df_term_4_Total_University_Studies_and_Environmental_Science_Pending)

#Waitlisted applications
df_term_4_Total_University_Studies_and_Environmental_Science_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_University_Studies_and_Environmental_Science_Waitlisted = len(df_term_4_Total_University_Studies_and_Environmental_Science_Waitlisted)

#Enrolled applications
df_term_4_Total_University_Studies_and_Environmental_Science_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_University_Studies_and_Environmental_Science)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_University_Studies_and_Environmental_Science_Enrolled = len(df_term_4_Total_University_Studies_and_Environmental_Science_Enrolled)

#Conversion Rate
if term_4_Total_University_Studies_and_Environmental_Science_Accepted == 0:  
    term_4_Total_University_Studies_and_Environmental_Science_Conversion_Rate = 0
else:
    term_4_Total_University_Studies_and_Environmental_Science_Conversion_Rate = term_4_Total_University_Studies_and_Environmental_Science_Enrolled / term_4_Total_University_Studies_and_Environmental_Science_Accepted

#Language Instruction for Newcomers to Canada----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Applications = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Applications)

#Accepted applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted)

#Payment Pending applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending)

#Denied applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Denied = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Denied)

#Declined applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Declined = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Declined)

#Pending applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Pending = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Pending)

#Waitlisted applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted)

#Enrolled applications
df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Language_Instruction_for_Newcomers_to_Canada)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled = len(df_term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled)

#Conversion Rate
if term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted == 0:  
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = 0
else:
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate = term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled / term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted

#Childhood Studies and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services))
]
term_4_Total_Childhood_Studies_and_Human_Services_Applications = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Applications)

#Accepted applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Childhood_Studies_and_Human_Services_Accepted = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Accepted)

#Payment Pending applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Childhood_Studies_and_Human_Services_Payment_Pending = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Payment_Pending)

#Denied applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Childhood_Studies_and_Human_Services_Denied = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Denied)

#Declined applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Childhood_Studies_and_Human_Services_Declined = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Declined)

#Pending applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Childhood_Studies_and_Human_Services_Pending = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Pending)

#Waitlisted applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Childhood_Studies_and_Human_Services_Waitlisted = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Waitlisted)

#Enrolled applications
df_term_4_Total_Childhood_Studies_and_Human_Services_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Childhood_Studies_and_Human_Services)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Childhood_Studies_and_Human_Services_Enrolled = len(df_term_4_Total_Childhood_Studies_and_Human_Services_Enrolled)

#Conversion Rate
if term_4_Total_Childhood_Studies_and_Human_Services_Accepted == 0:  
    term_4_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = 0
else:
    term_4_Total_Childhood_Studies_and_Human_Services_Conversion_Rate = term_4_Total_Childhood_Studies_and_Human_Services_Enrolled / term_4_Total_Childhood_Studies_and_Human_Services_Accepted

#Nursing and Allied Health----------------------------------------------------------------------------------------------------Done
#Combine program_ID_Nursing_and_Allied_Health and program_ID_NursingYr3 for calculation purpose
program_ID_Nursing_and_Allied_Health_including_NursingYr3 = (program_ID_Nursing_and_Allied_Health + program_ID_NursingYr3)

#Total applications
df_term_4_Total_Nursing_and_Allied_Health_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3))
]
term_4_Total_Nursing_and_Allied_Health_Applications = len(df_term_4_Total_Nursing_and_Allied_Health_Applications)

#Accepted applications
df_term_4_Total_Nursing_and_Allied_Health_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Nursing_and_Allied_Health_Accepted = len(df_term_4_Total_Nursing_and_Allied_Health_Accepted)

#Payment Pending applications
df_term_4_Total_Nursing_and_Allied_Health_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Nursing_and_Allied_Health_Payment_Pending = len(df_term_4_Total_Nursing_and_Allied_Health_Payment_Pending)

#Denied applications
df_term_4_Total_Nursing_and_Allied_Health_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Nursing_and_Allied_Health_Denied = len(df_term_4_Total_Nursing_and_Allied_Health_Denied)

#Declined applications
df_term_4_Total_Nursing_and_Allied_Health_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Nursing_and_Allied_Health_Declined = len(df_term_4_Total_Nursing_and_Allied_Health_Declined)

#Pending applications
df_term_4_Total_Nursing_and_Allied_Health_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Nursing_and_Allied_Health_Pending = len(df_term_4_Total_Nursing_and_Allied_Health_Pending)

#Waitlisted applications
df_term_4_Total_Nursing_and_Allied_Health_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Nursing_and_Allied_Health_Waitlisted = len(df_term_4_Total_Nursing_and_Allied_Health_Waitlisted)

#Enrolled applications
df_term_4_Total_Nursing_and_Allied_Health_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Nursing_and_Allied_Health_including_NursingYr3)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Nursing_and_Allied_Health_Enrolled = len(df_term_4_Total_Nursing_and_Allied_Health_Enrolled)

#Conversion Rate
if term_4_Total_Nursing_and_Allied_Health_Accepted == 0:  
    term_4_Total_Nursing_and_Allied_Health_Conversion_Rate = 0
else:
    term_4_Total_Nursing_and_Allied_Health_Conversion_Rate = term_4_Total_Nursing_and_Allied_Health_Enrolled / term_4_Total_Nursing_and_Allied_Health_Accepted

#Pre-employment----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_Pre_employment_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Pre_employment))
]
term_4_Total_Pre_employment_Applications = len(df_term_4_Total_Pre_employment_Applications)

#Accepted applications
df_term_4_Total_Pre_employment_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Pre_employment_Accepted = len(df_term_4_Total_Pre_employment_Accepted)

#Payment Pending applications
df_term_4_Total_Pre_employment_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Pre_employment_Payment_Pending = len(df_term_4_Total_Pre_employment_Payment_Pending)

#Denied applications
df_term_4_Total_Pre_employment_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Pre_employment_Denied = len(df_term_4_Total_Pre_employment_Denied)

#Declined applications
df_term_4_Total_Pre_employment_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Pre_employment_Declined = len(df_term_4_Total_Pre_employment_Declined)

#Pending applications
df_term_4_Total_Pre_employment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Pre_employment_Pending = len(df_term_4_Total_Pre_employment_Pending)

#Waitlisted applications
df_term_4_Total_Pre_employment_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Pre_employment_Waitlisted = len(df_term_4_Total_Pre_employment_Waitlisted)

#Enrolled applications
df_term_4_Total_Pre_employment_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Pre_employment)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Pre_employment_Enrolled = len(df_term_4_Total_Pre_employment_Enrolled)

#Conversion Rate
if term_4_Total_Pre_employment_Accepted == 0:  
    term_4_Total_Pre_employment_Conversion_Rate = 0
else:
    term_4_Total_Pre_employment_Conversion_Rate = term_4_Total_Pre_employment_Enrolled / term_4_Total_Pre_employment_Accepted

#Power Engineering----------------------------------------------------------------------------------------------------Done
#Total applications
df_term_4_Total_Power_Engineering_Applications = df_term_4_details[
    (df_term_4_details['CURRICULUM'].str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering))
]
term_4_Total_Power_Engineering_Applications = len(df_term_4_Total_Power_Engineering_Applications)

#Accepted applications
df_term_4_Total_Power_Engineering_Accepted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_4_details['Accepted'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'ACLN', 'COND'])) &
    ~(df_term_4_details['Deferred Application'].fillna('').str.replace(' ', '').str.upper().isin(['ACPT', 'COND']))
]
term_4_Total_Power_Engineering_Accepted = len(df_term_4_Total_Power_Engineering_Accepted)

#Payment Pending applications
df_term_4_Total_Power_Engineering_Payment_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    ~(df_term_4_details['Payment Pending'].isna() | (df_term_4_details['Payment Pending'] == ''))
]
term_4_Total_Power_Engineering_Payment_Pending = len(df_term_4_Total_Power_Engineering_Payment_Pending)

#Denied applications
df_term_4_Total_Power_Engineering_Denied = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_4_details['Denied'].fillna('').str.replace(' ', '').str.upper().isin(['APEX', 'DENY', 'NTSL']))
]
term_4_Total_Power_Engineering_Denied = len(df_term_4_Total_Power_Engineering_Denied)

#Declined applications
df_term_4_Total_Power_Engineering_Declined = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_4_details['Declined'].fillna('').str.replace(' ', '').str.upper().isin(['CCAN', 'DECL']))
]
term_4_Total_Power_Engineering_Declined = len(df_term_4_Total_Power_Engineering_Declined)

#Pending applications
df_term_4_Total_Power_Engineering_Pending = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_4_details['Pending'].fillna('').str.replace(' ', '').str.upper().isin(['APRO', 'NODE']))
]
term_4_Total_Power_Engineering_Pending = len(df_term_4_Total_Power_Engineering_Pending)

#Waitlisted applications
df_term_4_Total_Power_Engineering_Waitlisted = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_4_details['Waitlisted'].fillna('').str.replace(' ', '').str.upper().isin(['CDMT', 'RQMT', 'WAIT']))
]
term_4_Total_Power_Engineering_Waitlisted = len(df_term_4_Total_Power_Engineering_Waitlisted)

#Enrolled applications
df_term_4_Total_Power_Engineering_Enrolled = df_term_4_details[
    (df_term_4_details['CURRICULUM'].fillna('').str.replace(' ', '').str.upper().isin(program_ID_Power_Engineering)) &
    (df_term_4_details['Enrollment'].fillna('').str.replace(' ', '').str.upper().isin(['ENRO']))
]
term_4_Total_Power_Engineering_Enrolled = len(df_term_4_Total_Power_Engineering_Enrolled)

#Conversion Rate
if term_4_Total_Power_Engineering_Accepted == 0:  
    term_4_Total_Power_Engineering_Conversion_Rate = 0
else:
    term_4_Total_Power_Engineering_Conversion_Rate = term_4_Total_Power_Engineering_Enrolled / term_4_Total_Power_Engineering_Accepted

#School of University Studies, Career Programs and Academic Upgrading----------------------------------------------------------------------------------------------------Done
#Total applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications = sum([
    term_4_Total_Business_Administration_Applications,
    term_4_Total_Education_and_Academic_Upgrading_Applications,
    term_4_Total_University_Studies_and_Environmental_Science_Applications,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Applications
])

#Accepted applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted = sum([
    term_4_Total_Business_Administration_Accepted,
    term_4_Total_Education_and_Academic_Upgrading_Accepted,
    term_4_Total_University_Studies_and_Environmental_Science_Accepted,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted
])

#Payment Pending applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending = sum([
    term_4_Total_Business_Administration_Payment_Pending,
    term_4_Total_Education_and_Academic_Upgrading_Payment_Pending,
    term_4_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending
])

#Denied applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied = sum([
    term_4_Total_Business_Administration_Denied,
    term_4_Total_Education_and_Academic_Upgrading_Denied,
    term_4_Total_University_Studies_and_Environmental_Science_Denied,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Denied
])

#Declined applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined = sum([
    term_4_Total_Business_Administration_Declined,
    term_4_Total_Education_and_Academic_Upgrading_Declined,
    term_4_Total_University_Studies_and_Environmental_Science_Declined,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Declined
])

#Pending applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending = sum([
    term_4_Total_Business_Administration_Pending,
    term_4_Total_Education_and_Academic_Upgrading_Pending,
    term_4_Total_University_Studies_and_Environmental_Science_Pending,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Pending
])

#Waitlisted applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted = sum([
    term_4_Total_Business_Administration_Waitlisted,
    term_4_Total_Education_and_Academic_Upgrading_Waitlisted,
    term_4_Total_University_Studies_and_Environmental_Science_Waitlisted,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted
])

#Enrolled applications
term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled = sum([
    term_4_Total_Business_Administration_Enrolled,
    term_4_Total_Education_and_Academic_Upgrading_Enrolled,
    term_4_Total_University_Studies_and_Environmental_Science_Enrolled,
    term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled
])

#Conversion Rate
if term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted == 0:  
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = 0
else:
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate = term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled / term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted

#School of Health and Human Services----------------------------------------------------------------------------------------------------Done
#Total applications
term_4_Total_School_of_Health_and_Human_Services_Applications = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Applications,
    term_4_Total_Nursing_and_Allied_Health_Applications
])

#Accepted applications
term_4_Total_School_of_Health_and_Human_Services_Accepted = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Accepted,
    term_4_Total_Nursing_and_Allied_Health_Accepted
])

#Payment Pending applications
term_4_Total_School_of_Health_and_Human_Services_Payment_Pending = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    term_4_Total_Nursing_and_Allied_Health_Payment_Pending
])

#Denied applications
term_4_Total_School_of_Health_and_Human_Services_Denied = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Denied,
    term_4_Total_Nursing_and_Allied_Health_Denied
])

#Declined applications
term_4_Total_School_of_Health_and_Human_Services_Declined = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Declined,
    term_4_Total_Nursing_and_Allied_Health_Declined
])

#Pending applications
term_4_Total_School_of_Health_and_Human_Services_Pending = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Pending,
    term_4_Total_Nursing_and_Allied_Health_Pending
])

#Waitlisted applications
term_4_Total_School_of_Health_and_Human_Services_Waitlisted = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    term_4_Total_Nursing_and_Allied_Health_Waitlisted
])

#Enrolled applications
term_4_Total_School_of_Health_and_Human_Services_Enrolled = sum([
    term_4_Total_Childhood_Studies_and_Human_Services_Enrolled,
    term_4_Total_Nursing_and_Allied_Health_Enrolled
])

#Conversion Rate
if term_4_Total_School_of_Health_and_Human_Services_Accepted == 0:  
    term_4_Total_School_of_Health_and_Human_Services_Conversion_Rate = 0
else:
    term_4_Total_School_of_Health_and_Human_Services_Conversion_Rate = term_4_Total_School_of_Health_and_Human_Services_Enrolled / term_4_Total_School_of_Health_and_Human_Services_Accepted

#School of Trades and Heavy Industrial----------------------------------------------------------------------------------------------------Done
#Total applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Applications = sum([
    term_4_Total_Pre_employment_Applications,
    term_4_Total_Power_Engineering_Applications
])

#Accepted applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Accepted = sum([
    term_4_Total_Pre_employment_Accepted,
    term_4_Total_Power_Engineering_Accepted
])

#Payment Pending applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending = sum([
    term_4_Total_Pre_employment_Payment_Pending,
    term_4_Total_Power_Engineering_Payment_Pending
])

#Denied applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Denied = sum([
    term_4_Total_Pre_employment_Denied,
    term_4_Total_Power_Engineering_Denied
])

#Declined applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Declined = sum([
    term_4_Total_Pre_employment_Declined,
    term_4_Total_Power_Engineering_Declined
])

#Pending applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Pending = sum([
    term_4_Total_Pre_employment_Pending,
    term_4_Total_Power_Engineering_Pending
])

#Waitlisted applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted = sum([
    term_4_Total_Pre_employment_Waitlisted,
    term_4_Total_Power_Engineering_Waitlisted
])

#Enrolled applications
term_4_Total_School_of_Trades_and_Heavy_Industrial_Enrolled = sum([
    term_4_Total_Pre_employment_Enrolled,
    term_4_Total_Power_Engineering_Enrolled
])

#Conversion Rate
if term_4_Total_School_of_Trades_and_Heavy_Industrial_Accepted == 0:  
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = 0
else:
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate = term_4_Total_School_of_Trades_and_Heavy_Industrial_Enrolled / term_4_Total_School_of_Trades_and_Heavy_Industrial_Accepted

#Keyano College----------------------------------------------------------------------------------------------------Done
#Total applications
term_4_Keyano_College_Total_Applications = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    term_4_Total_School_of_Health_and_Human_Services_Applications,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Applications
])

#Accepted applications
term_4_Keyano_College_Total_Accepted = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    term_4_Total_School_of_Health_and_Human_Services_Accepted,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Accepted
])

#Payment Pending applications
term_4_Keyano_College_Total_Payment_Pending = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    term_4_Total_School_of_Health_and_Human_Services_Payment_Pending,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending
])

#Denied applications
term_4_Keyano_College_Total_Denied = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    term_4_Total_School_of_Health_and_Human_Services_Denied,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Denied
])

#Declined applications
term_4_Keyano_College_Total_Declined = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    term_4_Total_School_of_Health_and_Human_Services_Declined,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Declined
])

#Pending applications
term_4_Keyano_College_Total_Pending = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    term_4_Total_School_of_Health_and_Human_Services_Pending,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Pending
])

#Waitlisted applications
term_4_Keyano_College_Total_Waitlisted = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    term_4_Total_School_of_Health_and_Human_Services_Waitlisted,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted
])

#Enrolled applications
term_4_Keyano_College_Total_Enrolled = sum([
    term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    term_4_Total_School_of_Health_and_Human_Services_Enrolled,
    term_4_Total_School_of_Trades_and_Heavy_Industrial_Enrolled
])

#Conversion Rate
if term_4_Keyano_College_Total_Accepted == 0:  
    term_4_Keyano_College_Total_Conversion_Rate = 0
else:
    term_4_Keyano_College_Total_Conversion_Rate = term_4_Keyano_College_Total_Enrolled / term_4_Keyano_College_Total_Accepted



In [27]:
# Fill in Applications KPI Excel Document
# Load Excel template
template_path = "KC YYYY-YY Applications KPI_TEMPLATE.xlsx"
wb = openpyxl.load_workbook(template_path)

# Map
variable_map = {
    #Date of data extraction
    "Extraction_date": variable_Extraction_date,

    #Term and Year variables
    "Term1Yr": term_1_term_year_current,
    "Term2Yr": term_2_term_year_current,
    "Term3Yr": term_3_term_year_current,
    "Term4Yr": term_4_term_year_current,

    #App Comparative Report Summary sheet
    "variable_range_applications_academic_year": variable_range_applications_academic_year,
    "term_1_term_year_current": term_1_term_year_current,
    "term_2_term_year_current": term_2_term_year_current,
    "term_3_term_year_current": term_3_term_year_current,
    "term_4_term_year_current": term_4_term_year_current,
    "AppTerm1Yr1": term_1_year_current,
    "AppTerm1Yr2": term_1_year_minus_one,
    "AppTerm1Yr3": term_1_year_minus_two,
    "AppTerm1Yr4": term_1_year_minus_three,
    "AppTerm2Yr1": term_2_year_current,
    "AppTerm2Yr2": term_2_year_minus_one,
    "AppTerm2Yr3": term_2_year_minus_two,
    "AppTerm2Yr4": term_2_year_minus_three,
    "AppTerm3Yr1": term_3_year_current,
    "AppTerm3Yr2": term_3_year_minus_one,
    "AppTerm3Yr3": term_3_year_minus_two,
    "AppTerm3Yr4": term_3_year_minus_three,
    "AppTerm4Yr1": term_4_year_current,
    "AppTerm4Yr2": term_4_year_minus_one,
    "AppTerm4Yr3": term_4_year_minus_two,
    "AppTerm4Yr4": term_4_year_minus_three,
    "App1": term_1_details_total_unique_applicants,
    "App2": term_1_details_total_applications,
    "App3": term_1_Keyano_College_Total_Pending,
    "App4": term_1_year_minus_one_Unique_All_Applicants,
    "App5": term_1_year_minus_one_Total_Apps_All_Applicants,
    "App6": term_1_year_minus_two_Unique_All_Applicants,
    "App7": term_1_year_minus_two_Total_Apps_All_Applicants,
    "App8": term_1_year_minus_three_Unique_All_Applicants,
    "App9": term_1_year_minus_three_Total_Apps_All_Applicants,
    "term_1_details_total_unique_applicants": term_1_details_total_unique_applicants,
    "term_1_total_applications_percent_change_number_unique_absolute": term_1_total_applications_percent_change_number_unique_absolute,
    "higher_lower_1": higher_lower_1,
    "term_1_year_minus_one": term_1_year_minus_one,
    "term_1_details_international_applications": term_1_details_international_applications,
    "term_1_international_percentage_number": term_1_international_percentage_number,
    "App10": term_2_details_total_unique_applicants,
    "App11": term_2_details_total_applications,
    "App12": term_2_Keyano_College_Total_Pending,
    "App13": term_2_year_minus_one_Unique_All_Applicants,
    "App14": term_2_year_minus_one_Total_Apps_All_Applicants,
    "App15": term_2_year_minus_two_Unique_All_Applicants,
    "App16": term_2_year_minus_two_Total_Apps_All_Applicants,
    "App17": term_2_year_minus_three_Unique_All_Applicants,
    "App18": term_2_year_minus_three_Total_Apps_All_Applicants,
    "term_2_details_total_unique_applicants": term_2_details_total_unique_applicants,
    "term_2_total_applications_percent_change_number_unique_absolute": term_2_total_applications_percent_change_number_unique_absolute,
    "higher_lower_2": higher_lower_2,
    "term_2_year_minus_one": term_2_year_minus_one,
    "term_2_details_international_applications": term_2_details_international_applications,
    "term_2_international_percentage_number": term_2_international_percentage_number,
    "App19": term_3_details_total_unique_applicants,
    "App20": term_3_details_total_applications,
    "App21": term_3_Keyano_College_Total_Pending,
    "App22": term_3_year_minus_one_Unique_All_Applicants,
    "App23": term_3_year_minus_one_Total_Apps_All_Applicants,
    "App24": term_3_year_minus_two_Unique_All_Applicants,
    "App25": term_3_year_minus_two_Total_Apps_All_Applicants,
    "App26": term_3_year_minus_three_Unique_All_Applicants,
    "App27": term_3_year_minus_three_Total_Apps_All_Applicants,
    "term_3_details_total_unique_applicants": term_3_details_total_unique_applicants,
    "term_3_total_applications_percent_change_number_unique_absolute": term_3_total_applications_percent_change_number_unique_absolute,
    "higher_lower_3": higher_lower_3,
    "term_3_year_minus_one": term_3_year_minus_one,
    "term_3_details_international_applications": term_3_details_international_applications,
    "term_3_international_percentage_number": term_3_international_percentage_number,
    "App28": term_4_details_total_unique_applicants,
    "App29": term_4_details_total_applications,
    "App30": term_4_Keyano_College_Total_Pending,
    "App31": term_4_year_minus_one_Unique_All_Applicants,
    "App32": term_4_year_minus_one_Total_Apps_All_Applicants,
    "App33": term_4_year_minus_two_Unique_All_Applicants,
    "App34": term_4_year_minus_two_Total_Apps_All_Applicants,
    "App35": term_4_year_minus_three_Unique_All_Applicants,
    "App36": term_4_year_minus_three_Total_Apps_All_Applicants,
    "term_4_details_total_unique_applicants": term_4_details_total_unique_applicants,
    "term_4_total_applications_percent_change_number_unique_absolute": term_4_total_applications_percent_change_number_unique_absolute,
    "higher_lower_4": higher_lower_4,
    "term_4_year_minus_one": term_4_year_minus_one,
    "term_4_details_international_applications": term_4_details_international_applications,
    "term_4_international_percentage_number": term_4_international_percentage_number,
    
    #Term1Yr YYYY Apps By School sheet
    "term_1_Total_Business_Administration_Applications": term_1_Total_Business_Administration_Applications,
    "term_1_Total_Business_Administration_Accepted": term_1_Total_Business_Administration_Accepted,
    "term_1_Total_Business_Administration_Payment_Pending": term_1_Total_Business_Administration_Payment_Pending,
    "term_1_Total_Business_Administration_Denied": term_1_Total_Business_Administration_Denied,
    "term_1_Total_Business_Administration_Declined": term_1_Total_Business_Administration_Declined,
    "term_1_Total_Business_Administration_Pending": term_1_Total_Business_Administration_Pending,
    "term_1_Total_Business_Administration_Waitlisted": term_1_Total_Business_Administration_Waitlisted,
    "term_1_Total_Business_Administration_Enrolled": term_1_Total_Business_Administration_Enrolled,
    "term_1_Total_Business_Administration_Conversion_Rate": term_1_Total_Business_Administration_Conversion_Rate,
    "term_1_Total_Education_and_Academic_Upgrading_Applications": term_1_Total_Education_and_Academic_Upgrading_Applications,
    "term_1_Total_Education_and_Academic_Upgrading_Accepted": term_1_Total_Education_and_Academic_Upgrading_Accepted,
    "term_1_Total_Education_and_Academic_Upgrading_Payment_Pending": term_1_Total_Education_and_Academic_Upgrading_Payment_Pending,
    "term_1_Total_Education_and_Academic_Upgrading_Denied": term_1_Total_Education_and_Academic_Upgrading_Denied,
    "term_1_Total_Education_and_Academic_Upgrading_Declined": term_1_Total_Education_and_Academic_Upgrading_Declined,
    "term_1_Total_Education_and_Academic_Upgrading_Pending": term_1_Total_Education_and_Academic_Upgrading_Pending,
    "term_1_Total_Education_and_Academic_Upgrading_Waitlisted": term_1_Total_Education_and_Academic_Upgrading_Waitlisted,
    "term_1_Total_Education_and_Academic_Upgrading_Enrolled": term_1_Total_Education_and_Academic_Upgrading_Enrolled,
    "term_1_Total_Education_and_Academic_Upgrading_Conversion_Rate": term_1_Total_Education_and_Academic_Upgrading_Conversion_Rate,
    "term_1_Total_University_Studies_and_Environmental_Science_Applications": term_1_Total_University_Studies_and_Environmental_Science_Applications,
    "term_1_Total_University_Studies_and_Environmental_Science_Accepted": term_1_Total_University_Studies_and_Environmental_Science_Accepted,
    "term_1_Total_University_Studies_and_Environmental_Science_Payment_Pending": term_1_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    "term_1_Total_University_Studies_and_Environmental_Science_Denied": term_1_Total_University_Studies_and_Environmental_Science_Denied,
    "term_1_Total_University_Studies_and_Environmental_Science_Declined": term_1_Total_University_Studies_and_Environmental_Science_Declined,
    "term_1_Total_University_Studies_and_Environmental_Science_Pending": term_1_Total_University_Studies_and_Environmental_Science_Pending,
    "term_1_Total_University_Studies_and_Environmental_Science_Waitlisted": term_1_Total_University_Studies_and_Environmental_Science_Waitlisted,
    "term_1_Total_University_Studies_and_Environmental_Science_Enrolled": term_1_Total_University_Studies_and_Environmental_Science_Enrolled,
    "term_1_Total_University_Studies_and_Environmental_Science_Conversion_Rate": term_1_Total_University_Studies_and_Environmental_Science_Conversion_Rate,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Applications": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Applications,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Denied": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Denied,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Declined": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Declined,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Pending": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Pending,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled,
    "term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate": term_1_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    "term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate": term_1_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate,
    "term_1_Total_Childhood_Studies_and_Human_Services_Applications": term_1_Total_Childhood_Studies_and_Human_Services_Applications,
    "term_1_Total_Childhood_Studies_and_Human_Services_Accepted": term_1_Total_Childhood_Studies_and_Human_Services_Accepted,
    "term_1_Total_Childhood_Studies_and_Human_Services_Payment_Pending": term_1_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    "term_1_Total_Childhood_Studies_and_Human_Services_Denied": term_1_Total_Childhood_Studies_and_Human_Services_Denied,
    "term_1_Total_Childhood_Studies_and_Human_Services_Declined": term_1_Total_Childhood_Studies_and_Human_Services_Declined,
    "term_1_Total_Childhood_Studies_and_Human_Services_Pending": term_1_Total_Childhood_Studies_and_Human_Services_Pending,
    "term_1_Total_Childhood_Studies_and_Human_Services_Waitlisted": term_1_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    "term_1_Total_Childhood_Studies_and_Human_Services_Enrolled": term_1_Total_Childhood_Studies_and_Human_Services_Enrolled,
    "term_1_Total_Childhood_Studies_and_Human_Services_Conversion_Rate": term_1_Total_Childhood_Studies_and_Human_Services_Conversion_Rate,
    "term_1_Total_Nursing_and_Allied_Health_Applications": term_1_Total_Nursing_and_Allied_Health_Applications,
    "term_1_Total_Nursing_and_Allied_Health_Accepted": term_1_Total_Nursing_and_Allied_Health_Accepted,
    "term_1_Total_Nursing_and_Allied_Health_Payment_Pending": term_1_Total_Nursing_and_Allied_Health_Payment_Pending,
    "term_1_Total_Nursing_and_Allied_Health_Denied": term_1_Total_Nursing_and_Allied_Health_Denied,
    "term_1_Total_Nursing_and_Allied_Health_Declined": term_1_Total_Nursing_and_Allied_Health_Declined,
    "term_1_Total_Nursing_and_Allied_Health_Pending": term_1_Total_Nursing_and_Allied_Health_Pending,
    "term_1_Total_Nursing_and_Allied_Health_Waitlisted": term_1_Total_Nursing_and_Allied_Health_Waitlisted,
    "term_1_Total_Nursing_and_Allied_Health_Enrolled": term_1_Total_Nursing_and_Allied_Health_Enrolled,
    "term_1_Total_Nursing_and_Allied_Health_Conversion_Rate": term_1_Total_Nursing_and_Allied_Health_Conversion_Rate,
    "term_1_Total_School_of_Health_and_Human_Services_Applications": term_1_Total_School_of_Health_and_Human_Services_Applications,
    "term_1_Total_School_of_Health_and_Human_Services_Accepted": term_1_Total_School_of_Health_and_Human_Services_Accepted,
    "term_1_Total_School_of_Health_and_Human_Services_Payment_Pending": term_1_Total_School_of_Health_and_Human_Services_Payment_Pending,
    "term_1_Total_School_of_Health_and_Human_Services_Denied": term_1_Total_School_of_Health_and_Human_Services_Denied,
    "term_1_Total_School_of_Health_and_Human_Services_Declined": term_1_Total_School_of_Health_and_Human_Services_Declined,
    "term_1_Total_School_of_Health_and_Human_Services_Pending": term_1_Total_School_of_Health_and_Human_Services_Pending,
    "term_1_Total_School_of_Health_and_Human_Services_Waitlisted": term_1_Total_School_of_Health_and_Human_Services_Waitlisted,
    "term_1_Total_School_of_Health_and_Human_Services_Enrolled": term_1_Total_School_of_Health_and_Human_Services_Enrolled,
    "term_1_Total_School_of_Health_and_Human_Services_Conversion_Rate": term_1_Total_School_of_Health_and_Human_Services_Conversion_Rate,
    "term_1_Total_Pre_employment_Applications": term_1_Total_Pre_employment_Applications,
    "term_1_Total_Pre_employment_Accepted": term_1_Total_Pre_employment_Accepted,
    "term_1_Total_Pre_employment_Payment_Pending": term_1_Total_Pre_employment_Payment_Pending,
    "term_1_Total_Pre_employment_Denied": term_1_Total_Pre_employment_Denied,
    "term_1_Total_Pre_employment_Declined": term_1_Total_Pre_employment_Declined,
    "term_1_Total_Pre_employment_Pending": term_1_Total_Pre_employment_Pending,
    "term_1_Total_Pre_employment_Waitlisted": term_1_Total_Pre_employment_Waitlisted,
    "term_1_Total_Pre_employment_Enrolled": term_1_Total_Pre_employment_Enrolled,
    "term_1_Total_Pre_employment_Conversion_Rate": term_1_Total_Pre_employment_Conversion_Rate,
    "term_1_Total_Power_Engineering_Applications": term_1_Total_Power_Engineering_Applications,
    "term_1_Total_Power_Engineering_Accepted": term_1_Total_Power_Engineering_Accepted,
    "term_1_Total_Power_Engineering_Payment_Pending": term_1_Total_Power_Engineering_Payment_Pending,
    "term_1_Total_Power_Engineering_Denied": term_1_Total_Power_Engineering_Denied,
    "term_1_Total_Power_Engineering_Declined": term_1_Total_Power_Engineering_Declined,
    "term_1_Total_Power_Engineering_Pending": term_1_Total_Power_Engineering_Pending,
    "term_1_Total_Power_Engineering_Waitlisted": term_1_Total_Power_Engineering_Waitlisted,
    "term_1_Total_Power_Engineering_Enrolled": term_1_Total_Power_Engineering_Enrolled,
    "term_1_Total_Power_Engineering_Conversion_Rate": term_1_Total_Power_Engineering_Conversion_Rate,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Applications": term_1_Total_School_of_Trades_and_Heavy_Industrial_Applications,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Accepted": term_1_Total_School_of_Trades_and_Heavy_Industrial_Accepted,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending": term_1_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Denied": term_1_Total_School_of_Trades_and_Heavy_Industrial_Denied,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Declined": term_1_Total_School_of_Trades_and_Heavy_Industrial_Declined,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Pending": term_1_Total_School_of_Trades_and_Heavy_Industrial_Pending,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted": term_1_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Enrolled": term_1_Total_School_of_Trades_and_Heavy_Industrial_Enrolled,
    "term_1_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate": term_1_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate,
    "term_1_Keyano_College_Total_Applications": term_1_Keyano_College_Total_Applications,
    "term_1_Keyano_College_Total_Accepted": term_1_Keyano_College_Total_Accepted,
    "term_1_Keyano_College_Total_Payment_Pending": term_1_Keyano_College_Total_Payment_Pending,
    "term_1_Keyano_College_Total_Denied": term_1_Keyano_College_Total_Denied,
    "term_1_Keyano_College_Total_Declined": term_1_Keyano_College_Total_Declined,
    "term_1_Keyano_College_Total_Pending": term_1_Keyano_College_Total_Pending,
    "term_1_Keyano_College_Total_Waitlisted": term_1_Keyano_College_Total_Waitlisted,
    "term_1_Keyano_College_Total_Enrolled": term_1_Keyano_College_Total_Enrolled,
    "term_1_Keyano_College_Total_Conversion_Rate": term_1_Keyano_College_Total_Conversion_Rate,
    "term_1_BABUSC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Applications'],
    "term_1_BABUSC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Accepted'],
    "term_1_BABUSC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BABUSC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Denied'],
    "term_1_BABUSC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Declined'],
    "term_1_BABUSC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Pending'],
    "term_1_BABUSC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BABUSC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Enrolled'],
    "term_1_BABUSC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BABUSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Applications'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Denied'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Declined'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Pending'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BAACTD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Applications'],
    "term_1_BAACTD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Accepted'],
    "term_1_BAACTD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BAACTD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Denied'],
    "term_1_BAACTD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Declined'],
    "term_1_BAACTD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Pending'],
    "term_1_BAACTD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BAACTD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Enrolled'],
    "term_1_BAACTD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Applications'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Denied'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Declined'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Pending'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BAACTD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Applications'],
    "term_1_BAACTD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Accepted'],
    "term_1_BAACTD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BAACTD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Denied'],
    "term_1_BAACTD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Declined'],
    "term_1_BAACTD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Pending'],
    "term_1_BAACTD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BAACTD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Enrolled'],
    "term_1_BAACTD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAACTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Applications'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Denied'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Declined'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Pending'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BABUSD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Applications'],
    "term_1_BABUSD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Accepted'],
    "term_1_BABUSD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BABUSD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Denied'],
    "term_1_BABUSD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Declined'],
    "term_1_BABUSD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Pending'],
    "term_1_BABUSD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BABUSD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Enrolled'],
    "term_1_BABUSD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Applications'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Denied'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Declined'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Pending'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BABUSD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Applications'],
    "term_1_BABUSD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Accepted'],
    "term_1_BABUSD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BABUSD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Denied'],
    "term_1_BABUSD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Declined'],
    "term_1_BABUSD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Pending'],
    "term_1_BABUSD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BABUSD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Enrolled'],
    "term_1_BABUSD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BABUSD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Applications'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Denied'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Declined'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Pending'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BAESPM_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Applications'],
    "term_1_BAESPM_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Accepted'],
    "term_1_BAESPM_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BAESPM_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Denied'],
    "term_1_BAESPM_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Declined'],
    "term_1_BAESPM_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Pending'],
    "term_1_BAESPM_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BAESPM_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Enrolled'],
    "term_1_BAESPM_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Applications'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Denied'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Declined'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Pending'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BAESPM_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Applications'],
    "term_1_BAESPM_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Accepted'],
    "term_1_BAESPM_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BAESPM_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Denied'],
    "term_1_BAESPM_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Declined'],
    "term_1_BAESPM_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Pending'],
    "term_1_BAESPM_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BAESPM_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Enrolled'],
    "term_1_BAESPM_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAESPM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Applications'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Denied'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Declined'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Pending'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Applications'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Accepted'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Denied'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Declined'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Pending'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Enrolled'],
    "term_1_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Applications'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Denied'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Declined'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Pending'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Applications'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Accepted'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Denied'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Declined'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Pending'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Enrolled'],
    "term_1_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAHRMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Applications'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Denied'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Declined'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Pending'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_GSCMD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Applications'],
    "term_1_GSCMD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Accepted'],
    "term_1_GSCMD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_GSCMD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Denied'],
    "term_1_GSCMD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Declined'],
    "term_1_GSCMD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Pending'],
    "term_1_GSCMD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_GSCMD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Enrolled'],
    "term_1_GSCMD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Applications'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Denied'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Declined'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Pending'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_GSCMD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Applications'],
    "term_1_GSCMD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Accepted'],
    "term_1_GSCMD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_GSCMD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Denied'],
    "term_1_GSCMD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Declined'],
    "term_1_GSCMD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Pending'],
    "term_1_GSCMD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_GSCMD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Enrolled'],
    "term_1_GSCMD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_GSCMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Applications'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Denied'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Declined'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Pending'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_DIGMD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Applications'],
    "term_1_DIGMD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Accepted'],
    "term_1_DIGMD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_DIGMD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Denied'],
    "term_1_DIGMD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Declined'],
    "term_1_DIGMD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Pending'],
    "term_1_DIGMD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_DIGMD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Enrolled'],
    "term_1_DIGMD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Applications'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Denied'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Declined'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Pending'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_DIGMD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Applications'],
    "term_1_DIGMD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Accepted'],
    "term_1_DIGMD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_DIGMD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Denied'],
    "term_1_DIGMD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Declined'],
    "term_1_DIGMD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Pending'],
    "term_1_DIGMD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_DIGMD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Enrolled'],
    "term_1_DIGMD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_DIGMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Applications'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Denied'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Declined'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Pending'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BAATCO_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Applications'],
    "term_1_BAATCO_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Accepted'],
    "term_1_BAATCO_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BAATCO_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Denied'],
    "term_1_BAATCO_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Declined'],
    "term_1_BAATCO_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Pending'],
    "term_1_BAATCO_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BAATCO_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Enrolled'],
    "term_1_BAATCO_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Applications'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Denied'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Declined'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Pending'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BAATCO_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Applications'],
    "term_1_BAATCO_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Accepted'],
    "term_1_BAATCO_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BAATCO_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Denied'],
    "term_1_BAATCO_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Declined'],
    "term_1_BAATCO_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Pending'],
    "term_1_BAATCO_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BAATCO_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Enrolled'],
    "term_1_BAATCO_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAATCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Applications'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Denied'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Declined'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Pending'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BABSCO_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Applications'],
    "term_1_BABSCO_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Accepted'],
    "term_1_BABSCO_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BABSCO_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Denied'],
    "term_1_BABSCO_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Declined'],
    "term_1_BABSCO_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Pending'],
    "term_1_BABSCO_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BABSCO_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Enrolled'],
    "term_1_BABSCO_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Applications'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Denied'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Declined'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Pending'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BABSCO_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Applications'],
    "term_1_BABSCO_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Accepted'],
    "term_1_BABSCO_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BABSCO_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Denied'],
    "term_1_BABSCO_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Declined'],
    "term_1_BABSCO_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Pending'],
    "term_1_BABSCO_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BABSCO_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Enrolled'],
    "term_1_BABSCO_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BABSCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Applications'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Denied'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Declined'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Pending'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_NAO_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Applications'],
    "term_1_NAO_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Accepted'],
    "term_1_NAO_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_NAO_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Denied'],
    "term_1_NAO_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Declined'],
    "term_1_NAO_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Pending'],
    "term_1_NAO_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Waitlisted'],
    "term_1_NAO_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Enrolled'],
    "term_1_NAO_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_NAO_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Applications'],
    "term_1_NAO_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Accepted'],
    "term_1_NAO_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_NAO_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Denied'],
    "term_1_NAO_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Declined'],
    "term_1_NAO_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Pending'],
    "term_1_NAO_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_NAO_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_NAO_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_NAO_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Applications'],
    "term_1_NAO_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Accepted'],
    "term_1_NAO_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_NAO_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Denied'],
    "term_1_NAO_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Declined'],
    "term_1_NAO_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Pending'],
    "term_1_NAO_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Waitlisted'],
    "term_1_NAO_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Enrolled'],
    "term_1_NAO_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_NAO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_NAO_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Applications'],
    "term_1_NAO_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Accepted'],
    "term_1_NAO_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_NAO_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Denied'],
    "term_1_NAO_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Declined'],
    "term_1_NAO_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Pending'],
    "term_1_NAO_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_NAO_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_NAO_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_NAO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_ALO_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Applications'],
    "term_1_ALO_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Accepted'],
    "term_1_ALO_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ALO_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Denied'],
    "term_1_ALO_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Declined'],
    "term_1_ALO_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Pending'],
    "term_1_ALO_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ALO_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Enrolled'],
    "term_1_ALO_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ALO_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Applications'],
    "term_1_ALO_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ALO_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ALO_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Denied'],
    "term_1_ALO_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Declined'],
    "term_1_ALO_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Pending'],
    "term_1_ALO_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ALO_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ALO_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ALO_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Applications'],
    "term_1_ALO_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Accepted'],
    "term_1_ALO_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_ALO_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Denied'],
    "term_1_ALO_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Declined'],
    "term_1_ALO_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Pending'],
    "term_1_ALO_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Waitlisted'],
    "term_1_ALO_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Enrolled'],
    "term_1_ALO_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ALO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_ALO_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Applications'],
    "term_1_ALO_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Accepted'],
    "term_1_ALO_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_ALO_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Denied'],
    "term_1_ALO_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Declined'],
    "term_1_ALO_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Pending'],
    "term_1_ALO_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_ALO_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_ALO_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ALO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Applications'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Accepted'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Denied'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Declined'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Pending'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Waitlisted'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Enrolled'],
    "term_1_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Applications'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Accepted'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Denied'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Declined'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Pending'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Applications'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Accepted'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Denied'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Declined'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Pending'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Waitlisted'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Enrolled'],
    "term_1_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_HOSMNG_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Applications'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Accepted'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Denied'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Declined'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Pending'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Applications'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Accepted'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Denied'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Declined'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Pending'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Enrolled'],
    "term_1_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAHRMC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Applications'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Denied'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Declined'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Pending'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_OFAC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Applications'],
    "term_1_OFAC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Accepted'],
    "term_1_OFAC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_OFAC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Denied'],
    "term_1_OFAC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Declined'],
    "term_1_OFAC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Pending'],
    "term_1_OFAC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_OFAC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Enrolled'],
    "term_1_OFAC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_OFAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Applications'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Denied'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Declined'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Pending'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_OFAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_EAP_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Applications'],
    "term_1_EAP_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Accepted'],
    "term_1_EAP_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_EAP_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Denied'],
    "term_1_EAP_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Declined'],
    "term_1_EAP_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Pending'],
    "term_1_EAP_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Waitlisted'],
    "term_1_EAP_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Enrolled'],
    "term_1_EAP_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_EAP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_EAP_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Applications'],
    "term_1_EAP_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Accepted'],
    "term_1_EAP_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_EAP_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Denied'],
    "term_1_EAP_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Declined'],
    "term_1_EAP_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Pending'],
    "term_1_EAP_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_EAP_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_EAP_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_EAP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_HSAFA_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Applications'],
    "term_1_HSAFA_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Accepted'],
    "term_1_HSAFA_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_HSAFA_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Denied'],
    "term_1_HSAFA_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Declined'],
    "term_1_HSAFA_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Pending'],
    "term_1_HSAFA_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Waitlisted'],
    "term_1_HSAFA_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Enrolled'],
    "term_1_HSAFA_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSAFA_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Applications'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Accepted'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Denied'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Declined'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Pending'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_HSAD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Applications'],
    "term_1_HSAD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Accepted'],
    "term_1_HSAD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_HSAD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Denied'],
    "term_1_HSAD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Declined'],
    "term_1_HSAD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Pending'],
    "term_1_HSAD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_HSAD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Enrolled'],
    "term_1_HSAD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSAD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Applications'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Denied'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Declined'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Pending'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSAD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_HSGD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Applications'],
    "term_1_HSGD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Accepted'],
    "term_1_HSGD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_HSGD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Denied'],
    "term_1_HSGD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Declined'],
    "term_1_HSGD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Pending'],
    "term_1_HSGD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_HSGD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Enrolled'],
    "term_1_HSGD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSGD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Applications'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Denied'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Declined'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Pending'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSGD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_HSNP_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Applications'],
    "term_1_HSNP_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Accepted'],
    "term_1_HSNP_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_HSNP_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Denied'],
    "term_1_HSNP_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Declined'],
    "term_1_HSNP_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Pending'],
    "term_1_HSNP_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Waitlisted'],
    "term_1_HSNP_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Enrolled'],
    "term_1_HSNP_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSNP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Applications'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Accepted'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Denied'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Declined'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Pending'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HSNP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTELEM_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Applications'],
    "term_1_UTELEM_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Accepted'],
    "term_1_UTELEM_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTELEM_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Denied'],
    "term_1_UTELEM_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Declined'],
    "term_1_UTELEM_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Pending'],
    "term_1_UTELEM_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTELEM_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTELEM_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTELEM_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Applications'],
    "term_1_UTELEM_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Accepted'],
    "term_1_UTELEM_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_UTELEM_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Denied'],
    "term_1_UTELEM_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Declined'],
    "term_1_UTELEM_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Pending'],
    "term_1_UTELEM_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Waitlisted'],
    "term_1_UTELEM_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Enrolled'],
    "term_1_UTELEM_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTELEM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Applications'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Accepted'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Denied'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Declined'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Pending'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_UTSEC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Applications'],
    "term_1_UTSEC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Accepted'],
    "term_1_UTSEC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTSEC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Denied'],
    "term_1_UTSEC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Declined'],
    "term_1_UTSEC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Pending'],
    "term_1_UTSEC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTSEC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTSEC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTSEC_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Applications'],
    "term_1_UTSEC_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Accepted'],
    "term_1_UTSEC_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_UTSEC_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Denied'],
    "term_1_UTSEC_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Declined'],
    "term_1_UTSEC_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Pending'],
    "term_1_UTSEC_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Waitlisted'],
    "term_1_UTSEC_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Enrolled'],
    "term_1_UTSEC_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTSEC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Applications'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Accepted'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Denied'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Declined'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Pending'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_ENVTD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Applications'],
    "term_1_ENVTD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Accepted'],
    "term_1_ENVTD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ENVTD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Denied'],
    "term_1_ENVTD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Declined'],
    "term_1_ENVTD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Pending'],
    "term_1_ENVTD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ENVTD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Enrolled'],
    "term_1_ENVTD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Applications'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Denied'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Declined'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Pending'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ENVTD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Applications'],
    "term_1_ENVTD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Accepted'],
    "term_1_ENVTD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_ENVTD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Denied'],
    "term_1_ENVTD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Declined'],
    "term_1_ENVTD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Pending'],
    "term_1_ENVTD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_ENVTD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Enrolled'],
    "term_1_ENVTD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ENVTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Applications'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Denied'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Declined'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Pending'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_ENVNC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Applications'],
    "term_1_ENVNC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Accepted'],
    "term_1_ENVNC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ENVNC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Denied'],
    "term_1_ENVNC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Declined'],
    "term_1_ENVNC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Pending'],
    "term_1_ENVNC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ENVNC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Enrolled'],
    "term_1_ENVNC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Applications'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Denied'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Declined'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Pending'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ENVNC_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Applications'],
    "term_1_ENVNC_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Accepted'],
    "term_1_ENVNC_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_ENVNC_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Denied'],
    "term_1_ENVNC_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Declined'],
    "term_1_ENVNC_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Pending'],
    "term_1_ENVNC_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Waitlisted'],
    "term_1_ENVNC_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Enrolled'],
    "term_1_ENVNC_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ENVNC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Applications'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Accepted'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Denied'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Declined'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Pending'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_UTARTS_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Applications'],
    "term_1_UTARTS_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Accepted'],
    "term_1_UTARTS_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTARTS_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Denied'],
    "term_1_UTARTS_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Declined'],
    "term_1_UTARTS_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Pending'],
    "term_1_UTARTS_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTARTS_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTARTS_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTARTS_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Applications'],
    "term_1_UTARTS_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Accepted'],
    "term_1_UTARTS_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_UTARTS_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Denied'],
    "term_1_UTARTS_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Declined'],
    "term_1_UTARTS_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Pending'],
    "term_1_UTARTS_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Waitlisted'],
    "term_1_UTARTS_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Enrolled'],
    "term_1_UTARTS_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTARTS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Applications'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Accepted'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Denied'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Declined'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Pending'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Applications'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Accepted'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Denied'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Declined'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Pending'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Applications'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Accepted'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Denied'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Declined'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Pending'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Waitlisted'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Enrolled'],
    "term_1_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTCOMM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Applications'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Accepted'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Denied'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Declined'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Pending'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_UTENG_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Applications'],
    "term_1_UTENG_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Accepted'],
    "term_1_UTENG_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTENG_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Denied'],
    "term_1_UTENG_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Declined'],
    "term_1_UTENG_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Pending'],
    "term_1_UTENG_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTENG_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTENG_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTENG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTENG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Applications'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Accepted'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Denied'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Declined'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Pending'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Applications'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Accepted'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Denied'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Declined'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Pending'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Waitlisted'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Enrolled'],
    "term_1_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCGS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Applications'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Accepted'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Denied'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Declined'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Pending'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Applications'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Accepted'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Denied'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Declined'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Pending'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Waitlisted'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Enrolled'],
    "term_1_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Applications'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Accepted'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Denied'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Declined'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Pending'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Applications'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Accepted'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Denied'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Declined'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Pending'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Waitlisted'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Enrolled'],
    "term_1_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCWK_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Applications'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Accepted'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Denied'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Declined'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Pending'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_OPEN_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Applications'],
    "term_1_OPEN_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Accepted'],
    "term_1_OPEN_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_OPEN_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Denied'],
    "term_1_OPEN_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Declined'],
    "term_1_OPEN_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Pending'],
    "term_1_OPEN_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Waitlisted'],
    "term_1_OPEN_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Enrolled'],
    "term_1_OPEN_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_OPEN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Applications'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Accepted'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Denied'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Declined'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Pending'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_OPEN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Applications'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Accepted'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Denied'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Declined'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Pending'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Enrolled'],
    "term_1_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Applications'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Denied'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Declined'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Pending'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Applications'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Accepted'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Denied'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Declined'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Pending'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Waitlisted'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Enrolled'],
    "term_1_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_GOVNSC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Applications'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Accepted'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Denied'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Declined'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Pending'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_COMSCC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Applications'],
    "term_1_COMSCC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Accepted'],
    "term_1_COMSCC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_COMSCC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Denied'],
    "term_1_COMSCC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Declined'],
    "term_1_COMSCC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Pending'],
    "term_1_COMSCC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_COMSCC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Enrolled'],
    "term_1_COMSCC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_COMSCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Applications'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Denied'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Declined'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Pending'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_COMSCD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Applications'],
    "term_1_COMSCD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Accepted'],
    "term_1_COMSCD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_COMSCD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Denied'],
    "term_1_COMSCD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Declined'],
    "term_1_COMSCD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Pending'],
    "term_1_COMSCD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_COMSCD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Enrolled'],
    "term_1_COMSCD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Applications'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Denied'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Declined'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Pending'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_COMSCD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Applications'],
    "term_1_COMSCD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Accepted'],
    "term_1_COMSCD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_COMSCD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Denied'],
    "term_1_COMSCD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Declined'],
    "term_1_COMSCD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Pending'],
    "term_1_COMSCD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_COMSCD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Enrolled'],
    "term_1_COMSCD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_COMSCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Applications'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Denied'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Declined'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Pending'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_GNARTC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Applications'],
    "term_1_GNARTC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Accepted'],
    "term_1_GNARTC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_GNARTC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Denied'],
    "term_1_GNARTC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Declined'],
    "term_1_GNARTC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Pending'],
    "term_1_GNARTC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_GNARTC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Enrolled'],
    "term_1_GNARTC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GNARTC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Applications'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Denied'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Declined'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Pending'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Applications'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Accepted'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Denied'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Declined'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Pending'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Enrolled'],
    "term_1_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GNSCIC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Applications'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Denied'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Declined'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Pending'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Applications'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Accepted'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Denied'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Declined'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Pending'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Enrolled'],
    "term_1_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ANCPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Applications'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Denied'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Declined'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Pending'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_NSPDC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Applications'],
    "term_1_NSPDC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Accepted'],
    "term_1_NSPDC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_NSPDC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Denied'],
    "term_1_NSPDC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Declined'],
    "term_1_NSPDC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Pending'],
    "term_1_NSPDC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_NSPDC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Enrolled'],
    "term_1_NSPDC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_NSPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Applications'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Denied'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Declined'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Pending'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_AITS_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Applications'],
    "term_1_AITS_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Accepted'],
    "term_1_AITS_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_AITS_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Denied'],
    "term_1_AITS_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Declined'],
    "term_1_AITS_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Pending'],
    "term_1_AITS_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Waitlisted'],
    "term_1_AITS_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Enrolled'],
    "term_1_AITS_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_AITS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_AITS_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Applications'],
    "term_1_AITS_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Accepted'],
    "term_1_AITS_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_AITS_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Denied'],
    "term_1_AITS_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Declined'],
    "term_1_AITS_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Pending'],
    "term_1_AITS_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_AITS_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_AITS_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_AITS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_LINC13_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Applications'],
    "term_1_LINC13_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Accepted'],
    "term_1_LINC13_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_LINC13_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Denied'],
    "term_1_LINC13_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Declined'],
    "term_1_LINC13_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Pending'],
    "term_1_LINC13_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Waitlisted'],
    "term_1_LINC13_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Enrolled'],
    "term_1_LINC13_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_LINC13_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Applications'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Accepted'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Denied'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Declined'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Pending'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_LINC13_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_AELCC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Applications'],
    "term_1_AELCC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Accepted'],
    "term_1_AELCC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_AELCC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Denied'],
    "term_1_AELCC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Declined'],
    "term_1_AELCC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Pending'],
    "term_1_AELCC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_AELCC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Enrolled'],
    "term_1_AELCC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_AELCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Applications'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Denied'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Declined'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Pending'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_AELCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ECCERT_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Applications'],
    "term_1_ECCERT_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Accepted'],
    "term_1_ECCERT_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ECCERT_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Denied'],
    "term_1_ECCERT_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Declined'],
    "term_1_ECCERT_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Pending'],
    "term_1_ECCERT_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ECCERT_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Enrolled'],
    "term_1_ECCERT_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ECCERT_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Applications'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Denied'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Declined'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Pending'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ECDIP_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Applications'],
    "term_1_ECDIP_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Accepted'],
    "term_1_ECDIP_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ECDIP_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Denied'],
    "term_1_ECDIP_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Declined'],
    "term_1_ECDIP_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Pending'],
    "term_1_ECDIP_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ECDIP_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Enrolled'],
    "term_1_ECDIP_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Applications'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Denied'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Declined'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Pending'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ECDIP_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Applications'],
    "term_1_ECDIP_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Accepted'],
    "term_1_ECDIP_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_ECDIP_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Denied'],
    "term_1_ECDIP_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Declined'],
    "term_1_ECDIP_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Pending'],
    "term_1_ECDIP_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Waitlisted'],
    "term_1_ECDIP_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Enrolled'],
    "term_1_ECDIP_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ECDIP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Applications'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Accepted'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Denied'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Declined'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Pending'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_CYCD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Applications'],
    "term_1_CYCD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Accepted'],
    "term_1_CYCD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_CYCD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Denied'],
    "term_1_CYCD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Declined'],
    "term_1_CYCD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Pending'],
    "term_1_CYCD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_CYCD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Enrolled'],
    "term_1_CYCD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Applications'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Denied'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Declined'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Pending'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_CYCD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Applications'],
    "term_1_CYCD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Accepted'],
    "term_1_CYCD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_CYCD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Denied'],
    "term_1_CYCD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Declined'],
    "term_1_CYCD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Pending'],
    "term_1_CYCD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_CYCD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Enrolled'],
    "term_1_CYCD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_CYCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Applications'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Denied'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Declined'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Pending'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_CYCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_ECEAC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Applications'],
    "term_1_ECEAC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Accepted'],
    "term_1_ECEAC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ECEAC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Denied'],
    "term_1_ECEAC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Declined'],
    "term_1_ECEAC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Pending'],
    "term_1_ECEAC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ECEAC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Enrolled'],
    "term_1_ECEAC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ECEAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Applications'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Denied'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Declined'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Pending'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Applications'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Accepted'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Denied'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Declined'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Pending'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Enrolled'],
    "term_1_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Applications'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Denied'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Declined'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Pending'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Applications'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Accepted'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Denied'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Declined'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Pending'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Waitlisted'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Enrolled'],
    "term_1_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_SOCWKD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Applications'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Accepted'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Denied'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Declined'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Pending'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_HCAC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Applications'],
    "term_1_HCAC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Accepted'],
    "term_1_HCAC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_HCAC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Denied'],
    "term_1_HCAC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Declined'],
    "term_1_HCAC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Pending'],
    "term_1_HCAC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_HCAC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Enrolled'],
    "term_1_HCAC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HCAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Applications'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Denied'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Declined'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Pending'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_HCAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PCPC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Applications'],
    "term_1_PCPC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Accepted'],
    "term_1_PCPC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PCPC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Denied'],
    "term_1_PCPC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Declined'],
    "term_1_PCPC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Pending'],
    "term_1_PCPC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PCPC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Enrolled'],
    "term_1_PCPC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PCPC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Applications'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Denied'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Declined'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Pending'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PCPC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ACP_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Applications'],
    "term_1_ACP_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Accepted'],
    "term_1_ACP_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_ACP_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Denied'],
    "term_1_ACP_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Declined'],
    "term_1_ACP_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Pending'],
    "term_1_ACP_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Waitlisted'],
    "term_1_ACP_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Enrolled'],
    "term_1_ACP_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_ACP_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Applications'],
    "term_1_ACP_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Accepted'],
    "term_1_ACP_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_ACP_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Denied'],
    "term_1_ACP_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Declined'],
    "term_1_ACP_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Pending'],
    "term_1_ACP_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_ACP_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_ACP_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_ACP_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Applications'],
    "term_1_ACP_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Accepted'],
    "term_1_ACP_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_ACP_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Denied'],
    "term_1_ACP_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Declined'],
    "term_1_ACP_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Pending'],
    "term_1_ACP_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Waitlisted'],
    "term_1_ACP_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Enrolled'],
    "term_1_ACP_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ACP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_ACP_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Applications'],
    "term_1_ACP_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Accepted'],
    "term_1_ACP_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_ACP_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Denied'],
    "term_1_ACP_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Declined'],
    "term_1_ACP_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Pending'],
    "term_1_ACP_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_ACP_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_ACP_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_ACP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_PNR_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Applications'],
    "term_1_PNR_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Accepted'],
    "term_1_PNR_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PNR_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Denied'],
    "term_1_PNR_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Declined'],
    "term_1_PNR_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Pending'],
    "term_1_PNR_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PNR_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Enrolled'],
    "term_1_PNR_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PNR_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Applications'],
    "term_1_PNR_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PNR_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PNR_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Denied'],
    "term_1_PNR_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Declined'],
    "term_1_PNR_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Pending'],
    "term_1_PNR_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PNR_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PNR_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PNR_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Applications'],
    "term_1_PNR_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Accepted'],
    "term_1_PNR_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_PNR_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Denied'],
    "term_1_PNR_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Declined'],
    "term_1_PNR_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Pending'],
    "term_1_PNR_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Waitlisted'],
    "term_1_PNR_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Enrolled'],
    "term_1_PNR_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_PNR_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_PNR_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Applications'],
    "term_1_PNR_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Accepted'],
    "term_1_PNR_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_PNR_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Denied'],
    "term_1_PNR_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Declined'],
    "term_1_PNR_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Pending'],
    "term_1_PNR_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_PNR_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_PNR_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_PNR_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_BSCN_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Applications'],
    "term_1_BSCN_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Accepted'],
    "term_1_BSCN_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_BSCN_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Denied'],
    "term_1_BSCN_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Declined'],
    "term_1_BSCN_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Pending'],
    "term_1_BSCN_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Waitlisted'],
    "term_1_BSCN_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Enrolled'],
    "term_1_BSCN_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Applications'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Accepted'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Denied'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Declined'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Pending'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_BSCN_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Applications'],
    "term_1_BSCN_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Accepted'],
    "term_1_BSCN_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_BSCN_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Denied'],
    "term_1_BSCN_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Declined'],
    "term_1_BSCN_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Pending'],
    "term_1_BSCN_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Waitlisted'],
    "term_1_BSCN_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Enrolled'],
    "term_1_BSCN_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BSCN_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Applications'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Accepted'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Denied'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Declined'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Pending'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_BSCN_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Applications": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Applications'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Accepted": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Accepted'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Payment_Pending": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Payment_Pending'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Denied": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Denied'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Declined": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Declined'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Pending": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Pending'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Waitlisted": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Waitlisted'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Enrolled": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Enrolled'],
    "term_1_NursingYr3_DOMESTIC_Year_3_Conversion_Rate": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_DOMESTIC_Year_3_Conversion_Rate'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Applications": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Applications'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Accepted": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Accepted'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Denied": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Denied'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Declined": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Declined'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Pending": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Pending'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Waitlisted": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Waitlisted'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Enrolled": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Enrolled'],
    "term_1_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate": term_1_NursingYr3_variable_dictionary['term_1_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate'],
    "term_1_IENCN_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Applications'],
    "term_1_IENCN_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Accepted'],
    "term_1_IENCN_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_IENCN_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Denied'],
    "term_1_IENCN_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Declined'],
    "term_1_IENCN_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Pending'],
    "term_1_IENCN_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Waitlisted'],
    "term_1_IENCN_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Enrolled'],
    "term_1_IENCN_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_IENCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Applications'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Accepted'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Denied'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Declined'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Pending'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_IENCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PAELEC_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Applications'],
    "term_1_PAELEC_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Accepted'],
    "term_1_PAELEC_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PAELEC_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Denied'],
    "term_1_PAELEC_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Declined'],
    "term_1_PAELEC_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Pending'],
    "term_1_PAELEC_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PAELEC_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Enrolled'],
    "term_1_PAELEC_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAELEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Applications'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Denied'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Declined'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Pending'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PAWELD_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Applications'],
    "term_1_PAWELD_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Accepted'],
    "term_1_PAWELD_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PAWELD_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Denied'],
    "term_1_PAWELD_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Declined'],
    "term_1_PAWELD_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Pending'],
    "term_1_PAWELD_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PAWELD_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Enrolled'],
    "term_1_PAWELD_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAWELD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Applications'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Denied'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Declined'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Pending'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PAHET_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Applications'],
    "term_1_PAHET_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Accepted'],
    "term_1_PAHET_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PAHET_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Denied'],
    "term_1_PAHET_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Declined'],
    "term_1_PAHET_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Pending'],
    "term_1_PAHET_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PAHET_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Enrolled'],
    "term_1_PAHET_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAHET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Applications'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Denied'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Declined'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Pending'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAHET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Applications'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Accepted'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Denied'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Declined'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Pending'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Enrolled'],
    "term_1_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAPLUM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Applications'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Denied'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Declined'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Pending'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_POWCM4_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Applications'],
    "term_1_POWCM4_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Accepted'],
    "term_1_POWCM4_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_POWCM4_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Denied'],
    "term_1_POWCM4_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Declined'],
    "term_1_POWCM4_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Pending'],
    "term_1_POWCM4_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Waitlisted'],
    "term_1_POWCM4_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Enrolled'],
    "term_1_POWCM4_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCM4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Applications'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Accepted'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Denied'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Declined'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Pending'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_POWCM3_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Applications'],
    "term_1_POWCM3_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Accepted'],
    "term_1_POWCM3_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_POWCM3_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Denied'],
    "term_1_POWCM3_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Declined'],
    "term_1_POWCM3_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Pending'],
    "term_1_POWCM3_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Waitlisted'],
    "term_1_POWCM3_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Enrolled'],
    "term_1_POWCM3_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCM3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Applications'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Accepted'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Denied'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Declined'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Pending'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_POWCP4_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Applications'],
    "term_1_POWCP4_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Accepted'],
    "term_1_POWCP4_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_POWCP4_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Denied'],
    "term_1_POWCP4_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Declined'],
    "term_1_POWCP4_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Pending'],
    "term_1_POWCP4_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Waitlisted'],
    "term_1_POWCP4_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Enrolled'],
    "term_1_POWCP4_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCP4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Applications'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Accepted'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Denied'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Declined'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Pending'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_POWCO4_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Applications'],
    "term_1_POWCO4_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Accepted'],
    "term_1_POWCO4_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_POWCO4_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Denied'],
    "term_1_POWCO4_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Declined'],
    "term_1_POWCO4_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Pending'],
    "term_1_POWCO4_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Waitlisted'],
    "term_1_POWCO4_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Enrolled'],
    "term_1_POWCO4_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCO4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Applications'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Accepted'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Denied'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Declined'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Pending'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_POWCO3_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Applications'],
    "term_1_POWCO3_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Accepted'],
    "term_1_POWCO3_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_POWCO3_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Denied'],
    "term_1_POWCO3_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Declined'],
    "term_1_POWCO3_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Pending'],
    "term_1_POWCO3_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Waitlisted'],
    "term_1_POWCO3_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Enrolled'],
    "term_1_POWCO3_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCO3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Applications'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Accepted'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Denied'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Declined'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Pending'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PPET_DOMESTIC_Year_1_Applications": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Applications'],
    "term_1_PPET_DOMESTIC_Year_1_Accepted": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Accepted'],
    "term_1_PPET_DOMESTIC_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Payment_Pending'],
    "term_1_PPET_DOMESTIC_Year_1_Denied": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Denied'],
    "term_1_PPET_DOMESTIC_Year_1_Declined": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Declined'],
    "term_1_PPET_DOMESTIC_Year_1_Pending": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Pending'],
    "term_1_PPET_DOMESTIC_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Waitlisted'],
    "term_1_PPET_DOMESTIC_Year_1_Enrolled": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Enrolled'],
    "term_1_PPET_DOMESTIC_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_1_PPET_INTERNATIONAL_Year_1_Applications": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Applications'],
    "term_1_PPET_INTERNATIONAL_Year_1_Accepted": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Accepted'],
    "term_1_PPET_INTERNATIONAL_Year_1_Payment_Pending": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_1_PPET_INTERNATIONAL_Year_1_Denied": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Denied'],
    "term_1_PPET_INTERNATIONAL_Year_1_Declined": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Declined'],
    "term_1_PPET_INTERNATIONAL_Year_1_Pending": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Pending'],
    "term_1_PPET_INTERNATIONAL_Year_1_Waitlisted": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_1_PPET_INTERNATIONAL_Year_1_Enrolled": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Enrolled'],
    "term_1_PPET_INTERNATIONAL_Year_1_Conversion_Rate": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_1_PPET_DOMESTIC_Year_2_Applications": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Applications'],
    "term_1_PPET_DOMESTIC_Year_2_Accepted": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Accepted'],
    "term_1_PPET_DOMESTIC_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Payment_Pending'],
    "term_1_PPET_DOMESTIC_Year_2_Denied": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Denied'],
    "term_1_PPET_DOMESTIC_Year_2_Declined": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Declined'],
    "term_1_PPET_DOMESTIC_Year_2_Pending": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Pending'],
    "term_1_PPET_DOMESTIC_Year_2_Waitlisted": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Waitlisted'],
    "term_1_PPET_DOMESTIC_Year_2_Enrolled": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Enrolled'],
    "term_1_PPET_DOMESTIC_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_PPET_DOMESTIC_Year_2_Conversion_Rate'],
    "term_1_PPET_INTERNATIONAL_Year_2_Applications": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Applications'],
    "term_1_PPET_INTERNATIONAL_Year_2_Accepted": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Accepted'],
    "term_1_PPET_INTERNATIONAL_Year_2_Payment_Pending": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_1_PPET_INTERNATIONAL_Year_2_Denied": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Denied'],
    "term_1_PPET_INTERNATIONAL_Year_2_Declined": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Declined'],
    "term_1_PPET_INTERNATIONAL_Year_2_Pending": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Pending'],
    "term_1_PPET_INTERNATIONAL_Year_2_Waitlisted": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Waitlisted'],
    "term_1_PPET_INTERNATIONAL_Year_2_Enrolled": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Enrolled'],
    "term_1_PPET_INTERNATIONAL_Year_2_Conversion_Rate": term_1_variable_dictionary['term_1_PPET_INTERNATIONAL_Year_2_Conversion_Rate'],

    #Term2Yr YYYY Apps By School sheet
    "term_2_Total_Business_Administration_Applications": term_2_Total_Business_Administration_Applications,
    "term_2_Total_Business_Administration_Accepted": term_2_Total_Business_Administration_Accepted,
    "term_2_Total_Business_Administration_Payment_Pending": term_2_Total_Business_Administration_Payment_Pending,
    "term_2_Total_Business_Administration_Denied": term_2_Total_Business_Administration_Denied,
    "term_2_Total_Business_Administration_Declined": term_2_Total_Business_Administration_Declined,
    "term_2_Total_Business_Administration_Pending": term_2_Total_Business_Administration_Pending,
    "term_2_Total_Business_Administration_Waitlisted": term_2_Total_Business_Administration_Waitlisted,
    "term_2_Total_Business_Administration_Enrolled": term_2_Total_Business_Administration_Enrolled,
    "term_2_Total_Business_Administration_Conversion_Rate": term_2_Total_Business_Administration_Conversion_Rate,
    "term_2_Total_Education_and_Academic_Upgrading_Applications": term_2_Total_Education_and_Academic_Upgrading_Applications,
    "term_2_Total_Education_and_Academic_Upgrading_Accepted": term_2_Total_Education_and_Academic_Upgrading_Accepted,
    "term_2_Total_Education_and_Academic_Upgrading_Payment_Pending": term_2_Total_Education_and_Academic_Upgrading_Payment_Pending,
    "term_2_Total_Education_and_Academic_Upgrading_Denied": term_2_Total_Education_and_Academic_Upgrading_Denied,
    "term_2_Total_Education_and_Academic_Upgrading_Declined": term_2_Total_Education_and_Academic_Upgrading_Declined,
    "term_2_Total_Education_and_Academic_Upgrading_Pending": term_2_Total_Education_and_Academic_Upgrading_Pending,
    "term_2_Total_Education_and_Academic_Upgrading_Waitlisted": term_2_Total_Education_and_Academic_Upgrading_Waitlisted,
    "term_2_Total_Education_and_Academic_Upgrading_Enrolled": term_2_Total_Education_and_Academic_Upgrading_Enrolled,
    "term_2_Total_Education_and_Academic_Upgrading_Conversion_Rate": term_2_Total_Education_and_Academic_Upgrading_Conversion_Rate,
    "term_2_Total_University_Studies_and_Environmental_Science_Applications": term_2_Total_University_Studies_and_Environmental_Science_Applications,
    "term_2_Total_University_Studies_and_Environmental_Science_Accepted": term_2_Total_University_Studies_and_Environmental_Science_Accepted,
    "term_2_Total_University_Studies_and_Environmental_Science_Payment_Pending": term_2_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    "term_2_Total_University_Studies_and_Environmental_Science_Denied": term_2_Total_University_Studies_and_Environmental_Science_Denied,
    "term_2_Total_University_Studies_and_Environmental_Science_Declined": term_2_Total_University_Studies_and_Environmental_Science_Declined,
    "term_2_Total_University_Studies_and_Environmental_Science_Pending": term_2_Total_University_Studies_and_Environmental_Science_Pending,
    "term_2_Total_University_Studies_and_Environmental_Science_Waitlisted": term_2_Total_University_Studies_and_Environmental_Science_Waitlisted,
    "term_2_Total_University_Studies_and_Environmental_Science_Enrolled": term_2_Total_University_Studies_and_Environmental_Science_Enrolled,
    "term_2_Total_University_Studies_and_Environmental_Science_Conversion_Rate": term_2_Total_University_Studies_and_Environmental_Science_Conversion_Rate,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Applications": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Applications,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Denied": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Denied,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Declined": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Declined,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Pending": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Pending,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled,
    "term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate": term_2_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    "term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate": term_2_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate,
    "term_2_Total_Childhood_Studies_and_Human_Services_Applications": term_2_Total_Childhood_Studies_and_Human_Services_Applications,
    "term_2_Total_Childhood_Studies_and_Human_Services_Accepted": term_2_Total_Childhood_Studies_and_Human_Services_Accepted,
    "term_2_Total_Childhood_Studies_and_Human_Services_Payment_Pending": term_2_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    "term_2_Total_Childhood_Studies_and_Human_Services_Denied": term_2_Total_Childhood_Studies_and_Human_Services_Denied,
    "term_2_Total_Childhood_Studies_and_Human_Services_Declined": term_2_Total_Childhood_Studies_and_Human_Services_Declined,
    "term_2_Total_Childhood_Studies_and_Human_Services_Pending": term_2_Total_Childhood_Studies_and_Human_Services_Pending,
    "term_2_Total_Childhood_Studies_and_Human_Services_Waitlisted": term_2_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    "term_2_Total_Childhood_Studies_and_Human_Services_Enrolled": term_2_Total_Childhood_Studies_and_Human_Services_Enrolled,
    "term_2_Total_Childhood_Studies_and_Human_Services_Conversion_Rate": term_2_Total_Childhood_Studies_and_Human_Services_Conversion_Rate,
    "term_2_Total_Nursing_and_Allied_Health_Applications": term_2_Total_Nursing_and_Allied_Health_Applications,
    "term_2_Total_Nursing_and_Allied_Health_Accepted": term_2_Total_Nursing_and_Allied_Health_Accepted,
    "term_2_Total_Nursing_and_Allied_Health_Payment_Pending": term_2_Total_Nursing_and_Allied_Health_Payment_Pending,
    "term_2_Total_Nursing_and_Allied_Health_Denied": term_2_Total_Nursing_and_Allied_Health_Denied,
    "term_2_Total_Nursing_and_Allied_Health_Declined": term_2_Total_Nursing_and_Allied_Health_Declined,
    "term_2_Total_Nursing_and_Allied_Health_Pending": term_2_Total_Nursing_and_Allied_Health_Pending,
    "term_2_Total_Nursing_and_Allied_Health_Waitlisted": term_2_Total_Nursing_and_Allied_Health_Waitlisted,
    "term_2_Total_Nursing_and_Allied_Health_Enrolled": term_2_Total_Nursing_and_Allied_Health_Enrolled,
    "term_2_Total_Nursing_and_Allied_Health_Conversion_Rate": term_2_Total_Nursing_and_Allied_Health_Conversion_Rate,
    "term_2_Total_School_of_Health_and_Human_Services_Applications": term_2_Total_School_of_Health_and_Human_Services_Applications,
    "term_2_Total_School_of_Health_and_Human_Services_Accepted": term_2_Total_School_of_Health_and_Human_Services_Accepted,
    "term_2_Total_School_of_Health_and_Human_Services_Payment_Pending": term_2_Total_School_of_Health_and_Human_Services_Payment_Pending,
    "term_2_Total_School_of_Health_and_Human_Services_Denied": term_2_Total_School_of_Health_and_Human_Services_Denied,
    "term_2_Total_School_of_Health_and_Human_Services_Declined": term_2_Total_School_of_Health_and_Human_Services_Declined,
    "term_2_Total_School_of_Health_and_Human_Services_Pending": term_2_Total_School_of_Health_and_Human_Services_Pending,
    "term_2_Total_School_of_Health_and_Human_Services_Waitlisted": term_2_Total_School_of_Health_and_Human_Services_Waitlisted,
    "term_2_Total_School_of_Health_and_Human_Services_Enrolled": term_2_Total_School_of_Health_and_Human_Services_Enrolled,
    "term_2_Total_School_of_Health_and_Human_Services_Conversion_Rate": term_2_Total_School_of_Health_and_Human_Services_Conversion_Rate,
    "term_2_Total_Pre_employment_Applications": term_2_Total_Pre_employment_Applications,
    "term_2_Total_Pre_employment_Accepted": term_2_Total_Pre_employment_Accepted,
    "term_2_Total_Pre_employment_Payment_Pending": term_2_Total_Pre_employment_Payment_Pending,
    "term_2_Total_Pre_employment_Denied": term_2_Total_Pre_employment_Denied,
    "term_2_Total_Pre_employment_Declined": term_2_Total_Pre_employment_Declined,
    "term_2_Total_Pre_employment_Pending": term_2_Total_Pre_employment_Pending,
    "term_2_Total_Pre_employment_Waitlisted": term_2_Total_Pre_employment_Waitlisted,
    "term_2_Total_Pre_employment_Enrolled": term_2_Total_Pre_employment_Enrolled,
    "term_2_Total_Pre_employment_Conversion_Rate": term_2_Total_Pre_employment_Conversion_Rate,
    "term_2_Total_Power_Engineering_Applications": term_2_Total_Power_Engineering_Applications,
    "term_2_Total_Power_Engineering_Accepted": term_2_Total_Power_Engineering_Accepted,
    "term_2_Total_Power_Engineering_Payment_Pending": term_2_Total_Power_Engineering_Payment_Pending,
    "term_2_Total_Power_Engineering_Denied": term_2_Total_Power_Engineering_Denied,
    "term_2_Total_Power_Engineering_Declined": term_2_Total_Power_Engineering_Declined,
    "term_2_Total_Power_Engineering_Pending": term_2_Total_Power_Engineering_Pending,
    "term_2_Total_Power_Engineering_Waitlisted": term_2_Total_Power_Engineering_Waitlisted,
    "term_2_Total_Power_Engineering_Enrolled": term_2_Total_Power_Engineering_Enrolled,
    "term_2_Total_Power_Engineering_Conversion_Rate": term_2_Total_Power_Engineering_Conversion_Rate,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Applications": term_2_Total_School_of_Trades_and_Heavy_Industrial_Applications,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Accepted": term_2_Total_School_of_Trades_and_Heavy_Industrial_Accepted,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending": term_2_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Denied": term_2_Total_School_of_Trades_and_Heavy_Industrial_Denied,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Declined": term_2_Total_School_of_Trades_and_Heavy_Industrial_Declined,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Pending": term_2_Total_School_of_Trades_and_Heavy_Industrial_Pending,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted": term_2_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Enrolled": term_2_Total_School_of_Trades_and_Heavy_Industrial_Enrolled,
    "term_2_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate": term_2_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate,
    "term_2_Keyano_College_Total_Applications": term_2_Keyano_College_Total_Applications,
    "term_2_Keyano_College_Total_Accepted": term_2_Keyano_College_Total_Accepted,
    "term_2_Keyano_College_Total_Payment_Pending": term_2_Keyano_College_Total_Payment_Pending,
    "term_2_Keyano_College_Total_Denied": term_2_Keyano_College_Total_Denied,
    "term_2_Keyano_College_Total_Declined": term_2_Keyano_College_Total_Declined,
    "term_2_Keyano_College_Total_Pending": term_2_Keyano_College_Total_Pending,
    "term_2_Keyano_College_Total_Waitlisted": term_2_Keyano_College_Total_Waitlisted,
    "term_2_Keyano_College_Total_Enrolled": term_2_Keyano_College_Total_Enrolled,
    "term_2_Keyano_College_Total_Conversion_Rate": term_2_Keyano_College_Total_Conversion_Rate,
    "term_2_BABUSC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Applications'],
    "term_2_BABUSC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Accepted'],
    "term_2_BABUSC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BABUSC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Denied'],
    "term_2_BABUSC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Declined'],
    "term_2_BABUSC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Pending'],
    "term_2_BABUSC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BABUSC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Enrolled'],
    "term_2_BABUSC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BABUSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Applications'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Denied'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Declined'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Pending'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BAACTD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Applications'],
    "term_2_BAACTD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Accepted'],
    "term_2_BAACTD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BAACTD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Denied'],
    "term_2_BAACTD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Declined'],
    "term_2_BAACTD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Pending'],
    "term_2_BAACTD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BAACTD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Enrolled'],
    "term_2_BAACTD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Applications'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Denied'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Declined'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Pending'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BAACTD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Applications'],
    "term_2_BAACTD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Accepted'],
    "term_2_BAACTD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BAACTD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Denied'],
    "term_2_BAACTD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Declined'],
    "term_2_BAACTD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Pending'],
    "term_2_BAACTD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BAACTD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Enrolled'],
    "term_2_BAACTD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAACTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Applications'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Denied'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Declined'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Pending'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BABUSD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Applications'],
    "term_2_BABUSD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Accepted'],
    "term_2_BABUSD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BABUSD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Denied'],
    "term_2_BABUSD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Declined'],
    "term_2_BABUSD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Pending'],
    "term_2_BABUSD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BABUSD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Enrolled'],
    "term_2_BABUSD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Applications'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Denied'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Declined'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Pending'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BABUSD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Applications'],
    "term_2_BABUSD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Accepted'],
    "term_2_BABUSD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BABUSD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Denied'],
    "term_2_BABUSD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Declined'],
    "term_2_BABUSD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Pending'],
    "term_2_BABUSD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BABUSD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Enrolled'],
    "term_2_BABUSD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BABUSD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Applications'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Denied'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Declined'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Pending'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BAESPM_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Applications'],
    "term_2_BAESPM_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Accepted'],
    "term_2_BAESPM_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BAESPM_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Denied'],
    "term_2_BAESPM_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Declined'],
    "term_2_BAESPM_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Pending'],
    "term_2_BAESPM_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BAESPM_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Enrolled'],
    "term_2_BAESPM_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Applications'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Denied'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Declined'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Pending'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BAESPM_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Applications'],
    "term_2_BAESPM_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Accepted'],
    "term_2_BAESPM_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BAESPM_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Denied'],
    "term_2_BAESPM_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Declined'],
    "term_2_BAESPM_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Pending'],
    "term_2_BAESPM_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BAESPM_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Enrolled'],
    "term_2_BAESPM_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAESPM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Applications'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Denied'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Declined'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Pending'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Applications'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Accepted'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Denied'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Declined'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Pending'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Enrolled'],
    "term_2_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Applications'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Denied'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Declined'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Pending'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Applications'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Accepted'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Denied'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Declined'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Pending'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Enrolled'],
    "term_2_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAHRMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Applications'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Denied'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Declined'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Pending'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_GSCMD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Applications'],
    "term_2_GSCMD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Accepted'],
    "term_2_GSCMD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_GSCMD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Denied'],
    "term_2_GSCMD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Declined'],
    "term_2_GSCMD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Pending'],
    "term_2_GSCMD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_GSCMD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Enrolled'],
    "term_2_GSCMD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Applications'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Denied'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Declined'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Pending'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_GSCMD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Applications'],
    "term_2_GSCMD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Accepted'],
    "term_2_GSCMD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_GSCMD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Denied'],
    "term_2_GSCMD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Declined'],
    "term_2_GSCMD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Pending'],
    "term_2_GSCMD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_GSCMD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Enrolled'],
    "term_2_GSCMD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_GSCMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Applications'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Denied'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Declined'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Pending'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_DIGMD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Applications'],
    "term_2_DIGMD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Accepted'],
    "term_2_DIGMD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_DIGMD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Denied'],
    "term_2_DIGMD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Declined'],
    "term_2_DIGMD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Pending'],
    "term_2_DIGMD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_DIGMD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Enrolled'],
    "term_2_DIGMD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Applications'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Denied'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Declined'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Pending'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_DIGMD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Applications'],
    "term_2_DIGMD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Accepted'],
    "term_2_DIGMD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_DIGMD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Denied'],
    "term_2_DIGMD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Declined'],
    "term_2_DIGMD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Pending'],
    "term_2_DIGMD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_DIGMD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Enrolled'],
    "term_2_DIGMD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_DIGMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Applications'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Denied'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Declined'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Pending'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BAATCO_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Applications'],
    "term_2_BAATCO_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Accepted'],
    "term_2_BAATCO_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BAATCO_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Denied'],
    "term_2_BAATCO_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Declined'],
    "term_2_BAATCO_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Pending'],
    "term_2_BAATCO_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BAATCO_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Enrolled'],
    "term_2_BAATCO_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Applications'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Denied'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Declined'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Pending'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BAATCO_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Applications'],
    "term_2_BAATCO_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Accepted'],
    "term_2_BAATCO_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BAATCO_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Denied'],
    "term_2_BAATCO_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Declined'],
    "term_2_BAATCO_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Pending'],
    "term_2_BAATCO_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BAATCO_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Enrolled'],
    "term_2_BAATCO_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAATCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Applications'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Denied'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Declined'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Pending'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BABSCO_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Applications'],
    "term_2_BABSCO_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Accepted'],
    "term_2_BABSCO_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BABSCO_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Denied'],
    "term_2_BABSCO_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Declined'],
    "term_2_BABSCO_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Pending'],
    "term_2_BABSCO_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BABSCO_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Enrolled'],
    "term_2_BABSCO_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Applications'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Denied'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Declined'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Pending'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BABSCO_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Applications'],
    "term_2_BABSCO_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Accepted'],
    "term_2_BABSCO_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BABSCO_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Denied'],
    "term_2_BABSCO_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Declined'],
    "term_2_BABSCO_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Pending'],
    "term_2_BABSCO_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BABSCO_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Enrolled'],
    "term_2_BABSCO_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BABSCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Applications'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Denied'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Declined'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Pending'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_NAO_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Applications'],
    "term_2_NAO_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Accepted'],
    "term_2_NAO_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_NAO_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Denied'],
    "term_2_NAO_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Declined'],
    "term_2_NAO_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Pending'],
    "term_2_NAO_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Waitlisted'],
    "term_2_NAO_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Enrolled'],
    "term_2_NAO_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_NAO_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Applications'],
    "term_2_NAO_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Accepted'],
    "term_2_NAO_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_NAO_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Denied'],
    "term_2_NAO_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Declined'],
    "term_2_NAO_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Pending'],
    "term_2_NAO_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_NAO_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_NAO_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_NAO_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Applications'],
    "term_2_NAO_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Accepted'],
    "term_2_NAO_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_NAO_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Denied'],
    "term_2_NAO_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Declined'],
    "term_2_NAO_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Pending'],
    "term_2_NAO_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Waitlisted'],
    "term_2_NAO_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Enrolled'],
    "term_2_NAO_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_NAO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_NAO_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Applications'],
    "term_2_NAO_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Accepted'],
    "term_2_NAO_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_NAO_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Denied'],
    "term_2_NAO_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Declined'],
    "term_2_NAO_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Pending'],
    "term_2_NAO_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_NAO_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_NAO_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_NAO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_ALO_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Applications'],
    "term_2_ALO_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Accepted'],
    "term_2_ALO_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ALO_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Denied'],
    "term_2_ALO_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Declined'],
    "term_2_ALO_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Pending'],
    "term_2_ALO_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ALO_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Enrolled'],
    "term_2_ALO_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ALO_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Applications'],
    "term_2_ALO_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ALO_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ALO_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Denied'],
    "term_2_ALO_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Declined'],
    "term_2_ALO_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Pending'],
    "term_2_ALO_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ALO_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ALO_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ALO_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Applications'],
    "term_2_ALO_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Accepted'],
    "term_2_ALO_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_ALO_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Denied'],
    "term_2_ALO_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Declined'],
    "term_2_ALO_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Pending'],
    "term_2_ALO_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Waitlisted'],
    "term_2_ALO_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Enrolled'],
    "term_2_ALO_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ALO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_ALO_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Applications'],
    "term_2_ALO_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Accepted'],
    "term_2_ALO_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_ALO_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Denied'],
    "term_2_ALO_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Declined'],
    "term_2_ALO_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Pending'],
    "term_2_ALO_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_ALO_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_ALO_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ALO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Applications'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Accepted'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Denied'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Declined'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Pending'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Waitlisted'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Enrolled'],
    "term_2_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Applications'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Accepted'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Denied'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Declined'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Pending'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Applications'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Accepted'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Denied'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Declined'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Pending'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Waitlisted'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Enrolled'],
    "term_2_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_HOSMNG_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Applications'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Accepted'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Denied'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Declined'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Pending'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Applications'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Accepted'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Denied'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Declined'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Pending'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Enrolled'],
    "term_2_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAHRMC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Applications'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Denied'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Declined'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Pending'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_OFAC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Applications'],
    "term_2_OFAC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Accepted'],
    "term_2_OFAC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_OFAC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Denied'],
    "term_2_OFAC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Declined'],
    "term_2_OFAC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Pending'],
    "term_2_OFAC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_OFAC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Enrolled'],
    "term_2_OFAC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_OFAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Applications'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Denied'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Declined'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Pending'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_OFAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_EAP_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Applications'],
    "term_2_EAP_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Accepted'],
    "term_2_EAP_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_EAP_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Denied'],
    "term_2_EAP_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Declined'],
    "term_2_EAP_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Pending'],
    "term_2_EAP_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Waitlisted'],
    "term_2_EAP_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Enrolled'],
    "term_2_EAP_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_EAP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_EAP_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Applications'],
    "term_2_EAP_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Accepted'],
    "term_2_EAP_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_EAP_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Denied'],
    "term_2_EAP_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Declined'],
    "term_2_EAP_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Pending'],
    "term_2_EAP_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_EAP_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_EAP_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_EAP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_HSAFA_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Applications'],
    "term_2_HSAFA_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Accepted'],
    "term_2_HSAFA_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_HSAFA_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Denied'],
    "term_2_HSAFA_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Declined'],
    "term_2_HSAFA_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Pending'],
    "term_2_HSAFA_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Waitlisted'],
    "term_2_HSAFA_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Enrolled'],
    "term_2_HSAFA_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSAFA_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Applications'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Accepted'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Denied'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Declined'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Pending'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_HSAD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Applications'],
    "term_2_HSAD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Accepted'],
    "term_2_HSAD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_HSAD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Denied'],
    "term_2_HSAD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Declined'],
    "term_2_HSAD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Pending'],
    "term_2_HSAD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_HSAD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Enrolled'],
    "term_2_HSAD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSAD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Applications'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Denied'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Declined'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Pending'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSAD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_HSGD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Applications'],
    "term_2_HSGD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Accepted'],
    "term_2_HSGD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_HSGD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Denied'],
    "term_2_HSGD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Declined'],
    "term_2_HSGD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Pending'],
    "term_2_HSGD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_HSGD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Enrolled'],
    "term_2_HSGD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSGD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Applications'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Denied'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Declined'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Pending'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSGD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_HSNP_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Applications'],
    "term_2_HSNP_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Accepted'],
    "term_2_HSNP_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_HSNP_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Denied'],
    "term_2_HSNP_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Declined'],
    "term_2_HSNP_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Pending'],
    "term_2_HSNP_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Waitlisted'],
    "term_2_HSNP_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Enrolled'],
    "term_2_HSNP_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSNP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Applications'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Accepted'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Denied'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Declined'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Pending'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HSNP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTELEM_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Applications'],
    "term_2_UTELEM_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Accepted'],
    "term_2_UTELEM_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTELEM_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Denied'],
    "term_2_UTELEM_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Declined'],
    "term_2_UTELEM_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Pending'],
    "term_2_UTELEM_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTELEM_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTELEM_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTELEM_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Applications'],
    "term_2_UTELEM_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Accepted'],
    "term_2_UTELEM_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_UTELEM_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Denied'],
    "term_2_UTELEM_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Declined'],
    "term_2_UTELEM_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Pending'],
    "term_2_UTELEM_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Waitlisted'],
    "term_2_UTELEM_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Enrolled'],
    "term_2_UTELEM_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTELEM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Applications'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Accepted'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Denied'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Declined'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Pending'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_UTSEC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Applications'],
    "term_2_UTSEC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Accepted'],
    "term_2_UTSEC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTSEC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Denied'],
    "term_2_UTSEC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Declined'],
    "term_2_UTSEC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Pending'],
    "term_2_UTSEC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTSEC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTSEC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTSEC_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Applications'],
    "term_2_UTSEC_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Accepted'],
    "term_2_UTSEC_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_UTSEC_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Denied'],
    "term_2_UTSEC_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Declined'],
    "term_2_UTSEC_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Pending'],
    "term_2_UTSEC_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Waitlisted'],
    "term_2_UTSEC_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Enrolled'],
    "term_2_UTSEC_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTSEC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Applications'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Accepted'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Denied'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Declined'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Pending'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_ENVTD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Applications'],
    "term_2_ENVTD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Accepted'],
    "term_2_ENVTD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ENVTD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Denied'],
    "term_2_ENVTD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Declined'],
    "term_2_ENVTD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Pending'],
    "term_2_ENVTD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ENVTD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Enrolled'],
    "term_2_ENVTD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Applications'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Denied'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Declined'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Pending'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ENVTD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Applications'],
    "term_2_ENVTD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Accepted'],
    "term_2_ENVTD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_ENVTD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Denied'],
    "term_2_ENVTD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Declined'],
    "term_2_ENVTD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Pending'],
    "term_2_ENVTD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_ENVTD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Enrolled'],
    "term_2_ENVTD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ENVTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Applications'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Denied'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Declined'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Pending'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_ENVNC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Applications'],
    "term_2_ENVNC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Accepted'],
    "term_2_ENVNC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ENVNC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Denied'],
    "term_2_ENVNC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Declined'],
    "term_2_ENVNC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Pending'],
    "term_2_ENVNC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ENVNC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Enrolled'],
    "term_2_ENVNC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Applications'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Denied'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Declined'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Pending'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ENVNC_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Applications'],
    "term_2_ENVNC_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Accepted'],
    "term_2_ENVNC_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_ENVNC_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Denied'],
    "term_2_ENVNC_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Declined'],
    "term_2_ENVNC_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Pending'],
    "term_2_ENVNC_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Waitlisted'],
    "term_2_ENVNC_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Enrolled'],
    "term_2_ENVNC_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ENVNC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Applications'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Accepted'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Denied'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Declined'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Pending'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_UTARTS_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Applications'],
    "term_2_UTARTS_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Accepted'],
    "term_2_UTARTS_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTARTS_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Denied'],
    "term_2_UTARTS_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Declined'],
    "term_2_UTARTS_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Pending'],
    "term_2_UTARTS_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTARTS_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTARTS_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTARTS_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Applications'],
    "term_2_UTARTS_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Accepted'],
    "term_2_UTARTS_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_UTARTS_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Denied'],
    "term_2_UTARTS_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Declined'],
    "term_2_UTARTS_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Pending'],
    "term_2_UTARTS_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Waitlisted'],
    "term_2_UTARTS_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Enrolled'],
    "term_2_UTARTS_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTARTS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Applications'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Accepted'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Denied'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Declined'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Pending'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Applications'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Accepted'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Denied'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Declined'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Pending'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Applications'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Accepted'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Denied'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Declined'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Pending'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Waitlisted'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Enrolled'],
    "term_2_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTCOMM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Applications'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Accepted'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Denied'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Declined'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Pending'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_UTENG_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Applications'],
    "term_2_UTENG_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Accepted'],
    "term_2_UTENG_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTENG_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Denied'],
    "term_2_UTENG_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Declined'],
    "term_2_UTENG_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Pending'],
    "term_2_UTENG_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTENG_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTENG_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTENG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTENG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Applications'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Accepted'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Denied'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Declined'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Pending'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Applications'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Accepted'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Denied'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Declined'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Pending'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Waitlisted'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Enrolled'],
    "term_2_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCGS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Applications'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Accepted'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Denied'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Declined'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Pending'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Applications'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Accepted'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Denied'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Declined'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Pending'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Waitlisted'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Enrolled'],
    "term_2_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Applications'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Accepted'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Denied'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Declined'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Pending'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Applications'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Accepted'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Denied'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Declined'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Pending'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Waitlisted'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Enrolled'],
    "term_2_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCWK_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Applications'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Accepted'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Denied'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Declined'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Pending'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_OPEN_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Applications'],
    "term_2_OPEN_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Accepted'],
    "term_2_OPEN_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_OPEN_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Denied'],
    "term_2_OPEN_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Declined'],
    "term_2_OPEN_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Pending'],
    "term_2_OPEN_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Waitlisted'],
    "term_2_OPEN_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Enrolled'],
    "term_2_OPEN_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_OPEN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Applications'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Accepted'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Denied'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Declined'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Pending'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_OPEN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Applications'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Accepted'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Denied'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Declined'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Pending'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Enrolled'],
    "term_2_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Applications'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Denied'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Declined'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Pending'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Applications'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Accepted'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Denied'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Declined'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Pending'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Waitlisted'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Enrolled'],
    "term_2_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_GOVNSC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Applications'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Accepted'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Denied'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Declined'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Pending'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_COMSCC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Applications'],
    "term_2_COMSCC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Accepted'],
    "term_2_COMSCC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_COMSCC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Denied'],
    "term_2_COMSCC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Declined'],
    "term_2_COMSCC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Pending'],
    "term_2_COMSCC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_COMSCC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Enrolled'],
    "term_2_COMSCC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_COMSCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Applications'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Denied'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Declined'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Pending'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_COMSCD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Applications'],
    "term_2_COMSCD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Accepted'],
    "term_2_COMSCD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_COMSCD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Denied'],
    "term_2_COMSCD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Declined'],
    "term_2_COMSCD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Pending'],
    "term_2_COMSCD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_COMSCD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Enrolled'],
    "term_2_COMSCD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Applications'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Denied'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Declined'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Pending'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_COMSCD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Applications'],
    "term_2_COMSCD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Accepted'],
    "term_2_COMSCD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_COMSCD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Denied'],
    "term_2_COMSCD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Declined'],
    "term_2_COMSCD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Pending'],
    "term_2_COMSCD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_COMSCD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Enrolled'],
    "term_2_COMSCD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_COMSCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Applications'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Denied'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Declined'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Pending'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_GNARTC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Applications'],
    "term_2_GNARTC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Accepted'],
    "term_2_GNARTC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_GNARTC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Denied'],
    "term_2_GNARTC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Declined'],
    "term_2_GNARTC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Pending'],
    "term_2_GNARTC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_GNARTC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Enrolled'],
    "term_2_GNARTC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GNARTC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Applications'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Denied'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Declined'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Pending'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Applications'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Accepted'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Denied'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Declined'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Pending'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Enrolled'],
    "term_2_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GNSCIC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Applications'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Denied'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Declined'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Pending'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Applications'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Accepted'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Denied'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Declined'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Pending'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Enrolled'],
    "term_2_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ANCPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Applications'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Denied'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Declined'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Pending'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_NSPDC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Applications'],
    "term_2_NSPDC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Accepted'],
    "term_2_NSPDC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_NSPDC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Denied'],
    "term_2_NSPDC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Declined'],
    "term_2_NSPDC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Pending'],
    "term_2_NSPDC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_NSPDC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Enrolled'],
    "term_2_NSPDC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_NSPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Applications'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Denied'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Declined'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Pending'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_AITS_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Applications'],
    "term_2_AITS_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Accepted'],
    "term_2_AITS_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_AITS_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Denied'],
    "term_2_AITS_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Declined'],
    "term_2_AITS_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Pending'],
    "term_2_AITS_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Waitlisted'],
    "term_2_AITS_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Enrolled'],
    "term_2_AITS_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_AITS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_AITS_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Applications'],
    "term_2_AITS_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Accepted'],
    "term_2_AITS_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_AITS_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Denied'],
    "term_2_AITS_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Declined'],
    "term_2_AITS_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Pending'],
    "term_2_AITS_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_AITS_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_AITS_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_AITS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_LINC13_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Applications'],
    "term_2_LINC13_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Accepted'],
    "term_2_LINC13_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_LINC13_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Denied'],
    "term_2_LINC13_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Declined'],
    "term_2_LINC13_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Pending'],
    "term_2_LINC13_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Waitlisted'],
    "term_2_LINC13_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Enrolled'],
    "term_2_LINC13_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_LINC13_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Applications'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Accepted'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Denied'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Declined'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Pending'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_LINC13_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_AELCC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Applications'],
    "term_2_AELCC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Accepted'],
    "term_2_AELCC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_AELCC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Denied'],
    "term_2_AELCC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Declined'],
    "term_2_AELCC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Pending'],
    "term_2_AELCC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_AELCC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Enrolled'],
    "term_2_AELCC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_AELCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Applications'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Denied'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Declined'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Pending'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_AELCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ECCERT_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Applications'],
    "term_2_ECCERT_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Accepted'],
    "term_2_ECCERT_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ECCERT_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Denied'],
    "term_2_ECCERT_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Declined'],
    "term_2_ECCERT_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Pending'],
    "term_2_ECCERT_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ECCERT_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Enrolled'],
    "term_2_ECCERT_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ECCERT_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Applications'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Denied'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Declined'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Pending'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ECDIP_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Applications'],
    "term_2_ECDIP_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Accepted'],
    "term_2_ECDIP_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ECDIP_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Denied'],
    "term_2_ECDIP_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Declined'],
    "term_2_ECDIP_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Pending'],
    "term_2_ECDIP_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ECDIP_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Enrolled'],
    "term_2_ECDIP_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Applications'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Denied'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Declined'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Pending'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ECDIP_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Applications'],
    "term_2_ECDIP_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Accepted'],
    "term_2_ECDIP_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_ECDIP_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Denied'],
    "term_2_ECDIP_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Declined'],
    "term_2_ECDIP_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Pending'],
    "term_2_ECDIP_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Waitlisted'],
    "term_2_ECDIP_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Enrolled'],
    "term_2_ECDIP_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ECDIP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Applications'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Accepted'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Denied'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Declined'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Pending'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_CYCD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Applications'],
    "term_2_CYCD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Accepted'],
    "term_2_CYCD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_CYCD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Denied'],
    "term_2_CYCD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Declined'],
    "term_2_CYCD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Pending'],
    "term_2_CYCD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_CYCD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Enrolled'],
    "term_2_CYCD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Applications'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Denied'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Declined'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Pending'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_CYCD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Applications'],
    "term_2_CYCD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Accepted'],
    "term_2_CYCD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_CYCD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Denied'],
    "term_2_CYCD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Declined'],
    "term_2_CYCD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Pending'],
    "term_2_CYCD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_CYCD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Enrolled'],
    "term_2_CYCD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_CYCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Applications'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Denied'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Declined'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Pending'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_CYCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_ECEAC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Applications'],
    "term_2_ECEAC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Accepted'],
    "term_2_ECEAC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ECEAC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Denied'],
    "term_2_ECEAC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Declined'],
    "term_2_ECEAC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Pending'],
    "term_2_ECEAC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ECEAC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Enrolled'],
    "term_2_ECEAC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ECEAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Applications'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Denied'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Declined'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Pending'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Applications'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Accepted'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Denied'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Declined'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Pending'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Enrolled'],
    "term_2_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Applications'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Denied'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Declined'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Pending'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Applications'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Accepted'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Denied'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Declined'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Pending'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Waitlisted'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Enrolled'],
    "term_2_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_SOCWKD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Applications'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Accepted'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Denied'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Declined'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Pending'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_HCAC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Applications'],
    "term_2_HCAC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Accepted'],
    "term_2_HCAC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_HCAC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Denied'],
    "term_2_HCAC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Declined'],
    "term_2_HCAC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Pending'],
    "term_2_HCAC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_HCAC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Enrolled'],
    "term_2_HCAC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HCAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Applications'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Denied'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Declined'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Pending'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_HCAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PCPC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Applications'],
    "term_2_PCPC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Accepted'],
    "term_2_PCPC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PCPC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Denied'],
    "term_2_PCPC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Declined'],
    "term_2_PCPC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Pending'],
    "term_2_PCPC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PCPC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Enrolled'],
    "term_2_PCPC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PCPC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Applications'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Denied'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Declined'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Pending'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PCPC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ACP_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Applications'],
    "term_2_ACP_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Accepted'],
    "term_2_ACP_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_ACP_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Denied'],
    "term_2_ACP_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Declined'],
    "term_2_ACP_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Pending'],
    "term_2_ACP_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Waitlisted'],
    "term_2_ACP_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Enrolled'],
    "term_2_ACP_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_ACP_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Applications'],
    "term_2_ACP_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Accepted'],
    "term_2_ACP_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_ACP_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Denied'],
    "term_2_ACP_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Declined'],
    "term_2_ACP_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Pending'],
    "term_2_ACP_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_ACP_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_ACP_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_ACP_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Applications'],
    "term_2_ACP_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Accepted'],
    "term_2_ACP_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_ACP_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Denied'],
    "term_2_ACP_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Declined'],
    "term_2_ACP_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Pending'],
    "term_2_ACP_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Waitlisted'],
    "term_2_ACP_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Enrolled'],
    "term_2_ACP_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ACP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_ACP_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Applications'],
    "term_2_ACP_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Accepted'],
    "term_2_ACP_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_ACP_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Denied'],
    "term_2_ACP_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Declined'],
    "term_2_ACP_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Pending'],
    "term_2_ACP_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_ACP_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_ACP_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_ACP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_PNR_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Applications'],
    "term_2_PNR_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Accepted'],
    "term_2_PNR_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PNR_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Denied'],
    "term_2_PNR_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Declined'],
    "term_2_PNR_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Pending'],
    "term_2_PNR_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PNR_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Enrolled'],
    "term_2_PNR_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PNR_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Applications'],
    "term_2_PNR_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PNR_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PNR_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Denied'],
    "term_2_PNR_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Declined'],
    "term_2_PNR_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Pending'],
    "term_2_PNR_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PNR_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PNR_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PNR_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Applications'],
    "term_2_PNR_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Accepted'],
    "term_2_PNR_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_PNR_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Denied'],
    "term_2_PNR_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Declined'],
    "term_2_PNR_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Pending'],
    "term_2_PNR_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Waitlisted'],
    "term_2_PNR_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Enrolled'],
    "term_2_PNR_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_PNR_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_PNR_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Applications'],
    "term_2_PNR_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Accepted'],
    "term_2_PNR_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_PNR_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Denied'],
    "term_2_PNR_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Declined'],
    "term_2_PNR_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Pending'],
    "term_2_PNR_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_PNR_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_PNR_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_PNR_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_BSCN_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Applications'],
    "term_2_BSCN_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Accepted'],
    "term_2_BSCN_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_BSCN_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Denied'],
    "term_2_BSCN_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Declined'],
    "term_2_BSCN_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Pending'],
    "term_2_BSCN_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Waitlisted'],
    "term_2_BSCN_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Enrolled'],
    "term_2_BSCN_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Applications'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Accepted'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Denied'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Declined'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Pending'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_BSCN_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Applications'],
    "term_2_BSCN_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Accepted'],
    "term_2_BSCN_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_BSCN_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Denied'],
    "term_2_BSCN_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Declined'],
    "term_2_BSCN_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Pending'],
    "term_2_BSCN_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Waitlisted'],
    "term_2_BSCN_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Enrolled'],
    "term_2_BSCN_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BSCN_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Applications'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Accepted'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Denied'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Declined'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Pending'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_BSCN_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Applications": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Applications'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Accepted": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Accepted'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Payment_Pending": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Payment_Pending'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Denied": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Denied'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Declined": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Declined'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Pending": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Pending'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Waitlisted": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Waitlisted'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Enrolled": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Enrolled'],
    "term_2_NursingYr3_DOMESTIC_Year_3_Conversion_Rate": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_DOMESTIC_Year_3_Conversion_Rate'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Applications": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Applications'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Accepted": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Accepted'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Denied": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Denied'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Declined": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Declined'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Pending": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Pending'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Waitlisted": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Waitlisted'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Enrolled": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Enrolled'],
    "term_2_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate": term_2_NursingYr3_variable_dictionary['term_2_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate'],
    "term_2_IENCN_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Applications'],
    "term_2_IENCN_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Accepted'],
    "term_2_IENCN_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_IENCN_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Denied'],
    "term_2_IENCN_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Declined'],
    "term_2_IENCN_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Pending'],
    "term_2_IENCN_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Waitlisted'],
    "term_2_IENCN_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Enrolled'],
    "term_2_IENCN_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_IENCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Applications'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Accepted'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Denied'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Declined'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Pending'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_IENCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PAELEC_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Applications'],
    "term_2_PAELEC_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Accepted'],
    "term_2_PAELEC_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PAELEC_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Denied'],
    "term_2_PAELEC_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Declined'],
    "term_2_PAELEC_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Pending'],
    "term_2_PAELEC_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PAELEC_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Enrolled'],
    "term_2_PAELEC_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAELEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Applications'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Denied'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Declined'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Pending'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PAWELD_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Applications'],
    "term_2_PAWELD_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Accepted'],
    "term_2_PAWELD_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PAWELD_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Denied'],
    "term_2_PAWELD_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Declined'],
    "term_2_PAWELD_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Pending'],
    "term_2_PAWELD_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PAWELD_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Enrolled'],
    "term_2_PAWELD_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAWELD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Applications'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Denied'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Declined'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Pending'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PAHET_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Applications'],
    "term_2_PAHET_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Accepted'],
    "term_2_PAHET_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PAHET_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Denied'],
    "term_2_PAHET_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Declined'],
    "term_2_PAHET_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Pending'],
    "term_2_PAHET_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PAHET_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Enrolled'],
    "term_2_PAHET_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAHET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Applications'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Denied'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Declined'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Pending'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAHET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Applications'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Accepted'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Denied'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Declined'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Pending'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Enrolled'],
    "term_2_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAPLUM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Applications'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Denied'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Declined'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Pending'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_POWCM4_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Applications'],
    "term_2_POWCM4_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Accepted'],
    "term_2_POWCM4_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_POWCM4_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Denied'],
    "term_2_POWCM4_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Declined'],
    "term_2_POWCM4_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Pending'],
    "term_2_POWCM4_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Waitlisted'],
    "term_2_POWCM4_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Enrolled'],
    "term_2_POWCM4_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCM4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Applications'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Accepted'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Denied'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Declined'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Pending'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_POWCM3_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Applications'],
    "term_2_POWCM3_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Accepted'],
    "term_2_POWCM3_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_POWCM3_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Denied'],
    "term_2_POWCM3_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Declined'],
    "term_2_POWCM3_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Pending'],
    "term_2_POWCM3_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Waitlisted'],
    "term_2_POWCM3_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Enrolled'],
    "term_2_POWCM3_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCM3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Applications'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Accepted'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Denied'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Declined'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Pending'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_POWCP4_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Applications'],
    "term_2_POWCP4_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Accepted'],
    "term_2_POWCP4_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_POWCP4_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Denied'],
    "term_2_POWCP4_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Declined'],
    "term_2_POWCP4_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Pending'],
    "term_2_POWCP4_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Waitlisted'],
    "term_2_POWCP4_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Enrolled'],
    "term_2_POWCP4_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCP4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Applications'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Accepted'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Denied'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Declined'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Pending'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_POWCO4_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Applications'],
    "term_2_POWCO4_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Accepted'],
    "term_2_POWCO4_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_POWCO4_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Denied'],
    "term_2_POWCO4_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Declined'],
    "term_2_POWCO4_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Pending'],
    "term_2_POWCO4_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Waitlisted'],
    "term_2_POWCO4_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Enrolled'],
    "term_2_POWCO4_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCO4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Applications'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Accepted'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Denied'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Declined'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Pending'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_POWCO3_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Applications'],
    "term_2_POWCO3_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Accepted'],
    "term_2_POWCO3_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_POWCO3_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Denied'],
    "term_2_POWCO3_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Declined'],
    "term_2_POWCO3_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Pending'],
    "term_2_POWCO3_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Waitlisted'],
    "term_2_POWCO3_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Enrolled'],
    "term_2_POWCO3_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCO3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Applications'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Accepted'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Denied'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Declined'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Pending'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PPET_DOMESTIC_Year_1_Applications": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Applications'],
    "term_2_PPET_DOMESTIC_Year_1_Accepted": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Accepted'],
    "term_2_PPET_DOMESTIC_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Payment_Pending'],
    "term_2_PPET_DOMESTIC_Year_1_Denied": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Denied'],
    "term_2_PPET_DOMESTIC_Year_1_Declined": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Declined'],
    "term_2_PPET_DOMESTIC_Year_1_Pending": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Pending'],
    "term_2_PPET_DOMESTIC_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Waitlisted'],
    "term_2_PPET_DOMESTIC_Year_1_Enrolled": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Enrolled'],
    "term_2_PPET_DOMESTIC_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_2_PPET_INTERNATIONAL_Year_1_Applications": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Applications'],
    "term_2_PPET_INTERNATIONAL_Year_1_Accepted": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Accepted'],
    "term_2_PPET_INTERNATIONAL_Year_1_Payment_Pending": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_2_PPET_INTERNATIONAL_Year_1_Denied": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Denied'],
    "term_2_PPET_INTERNATIONAL_Year_1_Declined": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Declined'],
    "term_2_PPET_INTERNATIONAL_Year_1_Pending": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Pending'],
    "term_2_PPET_INTERNATIONAL_Year_1_Waitlisted": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_2_PPET_INTERNATIONAL_Year_1_Enrolled": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Enrolled'],
    "term_2_PPET_INTERNATIONAL_Year_1_Conversion_Rate": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_2_PPET_DOMESTIC_Year_2_Applications": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Applications'],
    "term_2_PPET_DOMESTIC_Year_2_Accepted": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Accepted'],
    "term_2_PPET_DOMESTIC_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Payment_Pending'],
    "term_2_PPET_DOMESTIC_Year_2_Denied": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Denied'],
    "term_2_PPET_DOMESTIC_Year_2_Declined": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Declined'],
    "term_2_PPET_DOMESTIC_Year_2_Pending": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Pending'],
    "term_2_PPET_DOMESTIC_Year_2_Waitlisted": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Waitlisted'],
    "term_2_PPET_DOMESTIC_Year_2_Enrolled": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Enrolled'],
    "term_2_PPET_DOMESTIC_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_PPET_DOMESTIC_Year_2_Conversion_Rate'],
    "term_2_PPET_INTERNATIONAL_Year_2_Applications": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Applications'],
    "term_2_PPET_INTERNATIONAL_Year_2_Accepted": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Accepted'],
    "term_2_PPET_INTERNATIONAL_Year_2_Payment_Pending": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_2_PPET_INTERNATIONAL_Year_2_Denied": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Denied'],
    "term_2_PPET_INTERNATIONAL_Year_2_Declined": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Declined'],
    "term_2_PPET_INTERNATIONAL_Year_2_Pending": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Pending'],
    "term_2_PPET_INTERNATIONAL_Year_2_Waitlisted": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Waitlisted'],
    "term_2_PPET_INTERNATIONAL_Year_2_Enrolled": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Enrolled'],
    "term_2_PPET_INTERNATIONAL_Year_2_Conversion_Rate": term_2_variable_dictionary['term_2_PPET_INTERNATIONAL_Year_2_Conversion_Rate'],

    #Term3Yr YYYY Apps By School sheet
    "term_3_Total_Business_Administration_Applications": term_3_Total_Business_Administration_Applications,
    "term_3_Total_Business_Administration_Accepted": term_3_Total_Business_Administration_Accepted,
    "term_3_Total_Business_Administration_Payment_Pending": term_3_Total_Business_Administration_Payment_Pending,
    "term_3_Total_Business_Administration_Denied": term_3_Total_Business_Administration_Denied,
    "term_3_Total_Business_Administration_Declined": term_3_Total_Business_Administration_Declined,
    "term_3_Total_Business_Administration_Pending": term_3_Total_Business_Administration_Pending,
    "term_3_Total_Business_Administration_Waitlisted": term_3_Total_Business_Administration_Waitlisted,
    "term_3_Total_Business_Administration_Enrolled": term_3_Total_Business_Administration_Enrolled,
    "term_3_Total_Business_Administration_Conversion_Rate": term_3_Total_Business_Administration_Conversion_Rate,
    "term_3_Total_Education_and_Academic_Upgrading_Applications": term_3_Total_Education_and_Academic_Upgrading_Applications,
    "term_3_Total_Education_and_Academic_Upgrading_Accepted": term_3_Total_Education_and_Academic_Upgrading_Accepted,
    "term_3_Total_Education_and_Academic_Upgrading_Payment_Pending": term_3_Total_Education_and_Academic_Upgrading_Payment_Pending,
    "term_3_Total_Education_and_Academic_Upgrading_Denied": term_3_Total_Education_and_Academic_Upgrading_Denied,
    "term_3_Total_Education_and_Academic_Upgrading_Declined": term_3_Total_Education_and_Academic_Upgrading_Declined,
    "term_3_Total_Education_and_Academic_Upgrading_Pending": term_3_Total_Education_and_Academic_Upgrading_Pending,
    "term_3_Total_Education_and_Academic_Upgrading_Waitlisted": term_3_Total_Education_and_Academic_Upgrading_Waitlisted,
    "term_3_Total_Education_and_Academic_Upgrading_Enrolled": term_3_Total_Education_and_Academic_Upgrading_Enrolled,
    "term_3_Total_Education_and_Academic_Upgrading_Conversion_Rate": term_3_Total_Education_and_Academic_Upgrading_Conversion_Rate,
    "term_3_Total_University_Studies_and_Environmental_Science_Applications": term_3_Total_University_Studies_and_Environmental_Science_Applications,
    "term_3_Total_University_Studies_and_Environmental_Science_Accepted": term_3_Total_University_Studies_and_Environmental_Science_Accepted,
    "term_3_Total_University_Studies_and_Environmental_Science_Payment_Pending": term_3_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    "term_3_Total_University_Studies_and_Environmental_Science_Denied": term_3_Total_University_Studies_and_Environmental_Science_Denied,
    "term_3_Total_University_Studies_and_Environmental_Science_Declined": term_3_Total_University_Studies_and_Environmental_Science_Declined,
    "term_3_Total_University_Studies_and_Environmental_Science_Pending": term_3_Total_University_Studies_and_Environmental_Science_Pending,
    "term_3_Total_University_Studies_and_Environmental_Science_Waitlisted": term_3_Total_University_Studies_and_Environmental_Science_Waitlisted,
    "term_3_Total_University_Studies_and_Environmental_Science_Enrolled": term_3_Total_University_Studies_and_Environmental_Science_Enrolled,
    "term_3_Total_University_Studies_and_Environmental_Science_Conversion_Rate": term_3_Total_University_Studies_and_Environmental_Science_Conversion_Rate,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Applications": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Applications,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Denied": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Denied,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Declined": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Declined,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Pending": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Pending,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled,
    "term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate": term_3_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    "term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate": term_3_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate,
    "term_3_Total_Childhood_Studies_and_Human_Services_Applications": term_3_Total_Childhood_Studies_and_Human_Services_Applications,
    "term_3_Total_Childhood_Studies_and_Human_Services_Accepted": term_3_Total_Childhood_Studies_and_Human_Services_Accepted,
    "term_3_Total_Childhood_Studies_and_Human_Services_Payment_Pending": term_3_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    "term_3_Total_Childhood_Studies_and_Human_Services_Denied": term_3_Total_Childhood_Studies_and_Human_Services_Denied,
    "term_3_Total_Childhood_Studies_and_Human_Services_Declined": term_3_Total_Childhood_Studies_and_Human_Services_Declined,
    "term_3_Total_Childhood_Studies_and_Human_Services_Pending": term_3_Total_Childhood_Studies_and_Human_Services_Pending,
    "term_3_Total_Childhood_Studies_and_Human_Services_Waitlisted": term_3_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    "term_3_Total_Childhood_Studies_and_Human_Services_Enrolled": term_3_Total_Childhood_Studies_and_Human_Services_Enrolled,
    "term_3_Total_Childhood_Studies_and_Human_Services_Conversion_Rate": term_3_Total_Childhood_Studies_and_Human_Services_Conversion_Rate,
    "term_3_Total_Nursing_and_Allied_Health_Applications": term_3_Total_Nursing_and_Allied_Health_Applications,
    "term_3_Total_Nursing_and_Allied_Health_Accepted": term_3_Total_Nursing_and_Allied_Health_Accepted,
    "term_3_Total_Nursing_and_Allied_Health_Payment_Pending": term_3_Total_Nursing_and_Allied_Health_Payment_Pending,
    "term_3_Total_Nursing_and_Allied_Health_Denied": term_3_Total_Nursing_and_Allied_Health_Denied,
    "term_3_Total_Nursing_and_Allied_Health_Declined": term_3_Total_Nursing_and_Allied_Health_Declined,
    "term_3_Total_Nursing_and_Allied_Health_Pending": term_3_Total_Nursing_and_Allied_Health_Pending,
    "term_3_Total_Nursing_and_Allied_Health_Waitlisted": term_3_Total_Nursing_and_Allied_Health_Waitlisted,
    "term_3_Total_Nursing_and_Allied_Health_Enrolled": term_3_Total_Nursing_and_Allied_Health_Enrolled,
    "term_3_Total_Nursing_and_Allied_Health_Conversion_Rate": term_3_Total_Nursing_and_Allied_Health_Conversion_Rate,
    "term_3_Total_School_of_Health_and_Human_Services_Applications": term_3_Total_School_of_Health_and_Human_Services_Applications,
    "term_3_Total_School_of_Health_and_Human_Services_Accepted": term_3_Total_School_of_Health_and_Human_Services_Accepted,
    "term_3_Total_School_of_Health_and_Human_Services_Payment_Pending": term_3_Total_School_of_Health_and_Human_Services_Payment_Pending,
    "term_3_Total_School_of_Health_and_Human_Services_Denied": term_3_Total_School_of_Health_and_Human_Services_Denied,
    "term_3_Total_School_of_Health_and_Human_Services_Declined": term_3_Total_School_of_Health_and_Human_Services_Declined,
    "term_3_Total_School_of_Health_and_Human_Services_Pending": term_3_Total_School_of_Health_and_Human_Services_Pending,
    "term_3_Total_School_of_Health_and_Human_Services_Waitlisted": term_3_Total_School_of_Health_and_Human_Services_Waitlisted,
    "term_3_Total_School_of_Health_and_Human_Services_Enrolled": term_3_Total_School_of_Health_and_Human_Services_Enrolled,
    "term_3_Total_School_of_Health_and_Human_Services_Conversion_Rate": term_3_Total_School_of_Health_and_Human_Services_Conversion_Rate,
    "term_3_Total_Pre_employment_Applications": term_3_Total_Pre_employment_Applications,
    "term_3_Total_Pre_employment_Accepted": term_3_Total_Pre_employment_Accepted,
    "term_3_Total_Pre_employment_Payment_Pending": term_3_Total_Pre_employment_Payment_Pending,
    "term_3_Total_Pre_employment_Denied": term_3_Total_Pre_employment_Denied,
    "term_3_Total_Pre_employment_Declined": term_3_Total_Pre_employment_Declined,
    "term_3_Total_Pre_employment_Pending": term_3_Total_Pre_employment_Pending,
    "term_3_Total_Pre_employment_Waitlisted": term_3_Total_Pre_employment_Waitlisted,
    "term_3_Total_Pre_employment_Enrolled": term_3_Total_Pre_employment_Enrolled,
    "term_3_Total_Pre_employment_Conversion_Rate": term_3_Total_Pre_employment_Conversion_Rate,
    "term_3_Total_Power_Engineering_Applications": term_3_Total_Power_Engineering_Applications,
    "term_3_Total_Power_Engineering_Accepted": term_3_Total_Power_Engineering_Accepted,
    "term_3_Total_Power_Engineering_Payment_Pending": term_3_Total_Power_Engineering_Payment_Pending,
    "term_3_Total_Power_Engineering_Denied": term_3_Total_Power_Engineering_Denied,
    "term_3_Total_Power_Engineering_Declined": term_3_Total_Power_Engineering_Declined,
    "term_3_Total_Power_Engineering_Pending": term_3_Total_Power_Engineering_Pending,
    "term_3_Total_Power_Engineering_Waitlisted": term_3_Total_Power_Engineering_Waitlisted,
    "term_3_Total_Power_Engineering_Enrolled": term_3_Total_Power_Engineering_Enrolled,
    "term_3_Total_Power_Engineering_Conversion_Rate": term_3_Total_Power_Engineering_Conversion_Rate,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Applications": term_3_Total_School_of_Trades_and_Heavy_Industrial_Applications,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Accepted": term_3_Total_School_of_Trades_and_Heavy_Industrial_Accepted,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending": term_3_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Denied": term_3_Total_School_of_Trades_and_Heavy_Industrial_Denied,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Declined": term_3_Total_School_of_Trades_and_Heavy_Industrial_Declined,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Pending": term_3_Total_School_of_Trades_and_Heavy_Industrial_Pending,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted": term_3_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Enrolled": term_3_Total_School_of_Trades_and_Heavy_Industrial_Enrolled,
    "term_3_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate": term_3_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate,
    "term_3_Keyano_College_Total_Applications": term_3_Keyano_College_Total_Applications,
    "term_3_Keyano_College_Total_Accepted": term_3_Keyano_College_Total_Accepted,
    "term_3_Keyano_College_Total_Payment_Pending": term_3_Keyano_College_Total_Payment_Pending,
    "term_3_Keyano_College_Total_Denied": term_3_Keyano_College_Total_Denied,
    "term_3_Keyano_College_Total_Declined": term_3_Keyano_College_Total_Declined,
    "term_3_Keyano_College_Total_Pending": term_3_Keyano_College_Total_Pending,
    "term_3_Keyano_College_Total_Waitlisted": term_3_Keyano_College_Total_Waitlisted,
    "term_3_Keyano_College_Total_Enrolled": term_3_Keyano_College_Total_Enrolled,
    "term_3_Keyano_College_Total_Conversion_Rate": term_3_Keyano_College_Total_Conversion_Rate,
    "term_3_BABUSC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Applications'],
    "term_3_BABUSC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Accepted'],
    "term_3_BABUSC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BABUSC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Denied'],
    "term_3_BABUSC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Declined'],
    "term_3_BABUSC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Pending'],
    "term_3_BABUSC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BABUSC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Enrolled'],
    "term_3_BABUSC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BABUSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Applications'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Denied'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Declined'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Pending'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BAACTD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Applications'],
    "term_3_BAACTD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Accepted'],
    "term_3_BAACTD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BAACTD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Denied'],
    "term_3_BAACTD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Declined'],
    "term_3_BAACTD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Pending'],
    "term_3_BAACTD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BAACTD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Enrolled'],
    "term_3_BAACTD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Applications'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Denied'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Declined'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Pending'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BAACTD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Applications'],
    "term_3_BAACTD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Accepted'],
    "term_3_BAACTD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BAACTD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Denied'],
    "term_3_BAACTD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Declined'],
    "term_3_BAACTD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Pending'],
    "term_3_BAACTD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BAACTD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Enrolled'],
    "term_3_BAACTD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAACTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Applications'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Denied'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Declined'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Pending'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BABUSD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Applications'],
    "term_3_BABUSD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Accepted'],
    "term_3_BABUSD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BABUSD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Denied'],
    "term_3_BABUSD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Declined'],
    "term_3_BABUSD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Pending'],
    "term_3_BABUSD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BABUSD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Enrolled'],
    "term_3_BABUSD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Applications'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Denied'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Declined'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Pending'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BABUSD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Applications'],
    "term_3_BABUSD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Accepted'],
    "term_3_BABUSD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BABUSD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Denied'],
    "term_3_BABUSD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Declined'],
    "term_3_BABUSD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Pending'],
    "term_3_BABUSD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BABUSD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Enrolled'],
    "term_3_BABUSD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BABUSD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Applications'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Denied'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Declined'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Pending'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BAESPM_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Applications'],
    "term_3_BAESPM_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Accepted'],
    "term_3_BAESPM_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BAESPM_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Denied'],
    "term_3_BAESPM_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Declined'],
    "term_3_BAESPM_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Pending'],
    "term_3_BAESPM_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BAESPM_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Enrolled'],
    "term_3_BAESPM_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Applications'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Denied'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Declined'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Pending'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BAESPM_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Applications'],
    "term_3_BAESPM_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Accepted'],
    "term_3_BAESPM_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BAESPM_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Denied'],
    "term_3_BAESPM_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Declined'],
    "term_3_BAESPM_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Pending'],
    "term_3_BAESPM_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BAESPM_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Enrolled'],
    "term_3_BAESPM_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAESPM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Applications'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Denied'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Declined'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Pending'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Applications'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Accepted'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Denied'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Declined'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Pending'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Enrolled'],
    "term_3_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Applications'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Denied'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Declined'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Pending'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Applications'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Accepted'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Denied'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Declined'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Pending'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Enrolled'],
    "term_3_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAHRMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Applications'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Denied'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Declined'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Pending'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_GSCMD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Applications'],
    "term_3_GSCMD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Accepted'],
    "term_3_GSCMD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_GSCMD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Denied'],
    "term_3_GSCMD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Declined'],
    "term_3_GSCMD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Pending'],
    "term_3_GSCMD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_GSCMD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Enrolled'],
    "term_3_GSCMD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Applications'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Denied'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Declined'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Pending'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_GSCMD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Applications'],
    "term_3_GSCMD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Accepted'],
    "term_3_GSCMD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_GSCMD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Denied'],
    "term_3_GSCMD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Declined'],
    "term_3_GSCMD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Pending'],
    "term_3_GSCMD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_GSCMD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Enrolled'],
    "term_3_GSCMD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_GSCMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Applications'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Denied'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Declined'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Pending'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_DIGMD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Applications'],
    "term_3_DIGMD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Accepted'],
    "term_3_DIGMD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_DIGMD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Denied'],
    "term_3_DIGMD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Declined'],
    "term_3_DIGMD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Pending'],
    "term_3_DIGMD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_DIGMD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Enrolled'],
    "term_3_DIGMD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Applications'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Denied'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Declined'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Pending'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_DIGMD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Applications'],
    "term_3_DIGMD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Accepted'],
    "term_3_DIGMD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_DIGMD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Denied'],
    "term_3_DIGMD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Declined'],
    "term_3_DIGMD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Pending'],
    "term_3_DIGMD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_DIGMD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Enrolled'],
    "term_3_DIGMD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_DIGMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Applications'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Denied'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Declined'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Pending'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BAATCO_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Applications'],
    "term_3_BAATCO_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Accepted'],
    "term_3_BAATCO_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BAATCO_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Denied'],
    "term_3_BAATCO_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Declined'],
    "term_3_BAATCO_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Pending'],
    "term_3_BAATCO_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BAATCO_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Enrolled'],
    "term_3_BAATCO_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Applications'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Denied'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Declined'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Pending'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BAATCO_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Applications'],
    "term_3_BAATCO_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Accepted'],
    "term_3_BAATCO_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BAATCO_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Denied'],
    "term_3_BAATCO_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Declined'],
    "term_3_BAATCO_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Pending'],
    "term_3_BAATCO_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BAATCO_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Enrolled'],
    "term_3_BAATCO_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAATCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Applications'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Denied'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Declined'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Pending'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BABSCO_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Applications'],
    "term_3_BABSCO_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Accepted'],
    "term_3_BABSCO_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BABSCO_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Denied'],
    "term_3_BABSCO_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Declined'],
    "term_3_BABSCO_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Pending'],
    "term_3_BABSCO_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BABSCO_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Enrolled'],
    "term_3_BABSCO_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Applications'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Denied'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Declined'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Pending'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BABSCO_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Applications'],
    "term_3_BABSCO_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Accepted'],
    "term_3_BABSCO_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BABSCO_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Denied'],
    "term_3_BABSCO_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Declined'],
    "term_3_BABSCO_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Pending'],
    "term_3_BABSCO_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BABSCO_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Enrolled'],
    "term_3_BABSCO_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BABSCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Applications'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Denied'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Declined'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Pending'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_NAO_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Applications'],
    "term_3_NAO_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Accepted'],
    "term_3_NAO_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_NAO_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Denied'],
    "term_3_NAO_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Declined'],
    "term_3_NAO_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Pending'],
    "term_3_NAO_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Waitlisted'],
    "term_3_NAO_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Enrolled'],
    "term_3_NAO_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_NAO_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Applications'],
    "term_3_NAO_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Accepted'],
    "term_3_NAO_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_NAO_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Denied'],
    "term_3_NAO_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Declined'],
    "term_3_NAO_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Pending'],
    "term_3_NAO_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_NAO_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_NAO_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_NAO_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Applications'],
    "term_3_NAO_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Accepted'],
    "term_3_NAO_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_NAO_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Denied'],
    "term_3_NAO_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Declined'],
    "term_3_NAO_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Pending'],
    "term_3_NAO_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Waitlisted'],
    "term_3_NAO_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Enrolled'],
    "term_3_NAO_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_NAO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_NAO_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Applications'],
    "term_3_NAO_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Accepted'],
    "term_3_NAO_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_NAO_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Denied'],
    "term_3_NAO_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Declined'],
    "term_3_NAO_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Pending'],
    "term_3_NAO_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_NAO_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_NAO_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_NAO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_ALO_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Applications'],
    "term_3_ALO_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Accepted'],
    "term_3_ALO_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ALO_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Denied'],
    "term_3_ALO_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Declined'],
    "term_3_ALO_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Pending'],
    "term_3_ALO_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ALO_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Enrolled'],
    "term_3_ALO_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ALO_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Applications'],
    "term_3_ALO_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ALO_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ALO_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Denied'],
    "term_3_ALO_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Declined'],
    "term_3_ALO_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Pending'],
    "term_3_ALO_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ALO_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ALO_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ALO_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Applications'],
    "term_3_ALO_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Accepted'],
    "term_3_ALO_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_ALO_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Denied'],
    "term_3_ALO_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Declined'],
    "term_3_ALO_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Pending'],
    "term_3_ALO_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Waitlisted'],
    "term_3_ALO_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Enrolled'],
    "term_3_ALO_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ALO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_ALO_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Applications'],
    "term_3_ALO_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Accepted'],
    "term_3_ALO_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_ALO_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Denied'],
    "term_3_ALO_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Declined'],
    "term_3_ALO_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Pending'],
    "term_3_ALO_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_ALO_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_ALO_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ALO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Applications'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Accepted'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Denied'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Declined'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Pending'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Waitlisted'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Enrolled'],
    "term_3_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Applications'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Accepted'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Denied'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Declined'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Pending'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Applications'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Accepted'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Denied'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Declined'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Pending'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Waitlisted'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Enrolled'],
    "term_3_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_HOSMNG_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Applications'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Accepted'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Denied'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Declined'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Pending'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Applications'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Accepted'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Denied'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Declined'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Pending'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Enrolled'],
    "term_3_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAHRMC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Applications'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Denied'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Declined'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Pending'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_OFAC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Applications'],
    "term_3_OFAC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Accepted'],
    "term_3_OFAC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_OFAC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Denied'],
    "term_3_OFAC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Declined'],
    "term_3_OFAC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Pending'],
    "term_3_OFAC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_OFAC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Enrolled'],
    "term_3_OFAC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_OFAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Applications'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Denied'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Declined'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Pending'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_OFAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_EAP_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Applications'],
    "term_3_EAP_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Accepted'],
    "term_3_EAP_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_EAP_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Denied'],
    "term_3_EAP_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Declined'],
    "term_3_EAP_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Pending'],
    "term_3_EAP_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Waitlisted'],
    "term_3_EAP_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Enrolled'],
    "term_3_EAP_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_EAP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_EAP_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Applications'],
    "term_3_EAP_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Accepted'],
    "term_3_EAP_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_EAP_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Denied'],
    "term_3_EAP_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Declined'],
    "term_3_EAP_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Pending'],
    "term_3_EAP_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_EAP_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_EAP_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_EAP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_HSAFA_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Applications'],
    "term_3_HSAFA_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Accepted'],
    "term_3_HSAFA_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_HSAFA_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Denied'],
    "term_3_HSAFA_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Declined'],
    "term_3_HSAFA_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Pending'],
    "term_3_HSAFA_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Waitlisted'],
    "term_3_HSAFA_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Enrolled'],
    "term_3_HSAFA_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSAFA_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Applications'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Accepted'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Denied'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Declined'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Pending'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_HSAD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Applications'],
    "term_3_HSAD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Accepted'],
    "term_3_HSAD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_HSAD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Denied'],
    "term_3_HSAD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Declined'],
    "term_3_HSAD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Pending'],
    "term_3_HSAD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_HSAD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Enrolled'],
    "term_3_HSAD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSAD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Applications'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Denied'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Declined'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Pending'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSAD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_HSGD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Applications'],
    "term_3_HSGD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Accepted'],
    "term_3_HSGD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_HSGD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Denied'],
    "term_3_HSGD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Declined'],
    "term_3_HSGD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Pending'],
    "term_3_HSGD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_HSGD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Enrolled'],
    "term_3_HSGD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSGD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Applications'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Denied'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Declined'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Pending'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSGD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_HSNP_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Applications'],
    "term_3_HSNP_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Accepted'],
    "term_3_HSNP_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_HSNP_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Denied'],
    "term_3_HSNP_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Declined'],
    "term_3_HSNP_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Pending'],
    "term_3_HSNP_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Waitlisted'],
    "term_3_HSNP_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Enrolled'],
    "term_3_HSNP_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSNP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Applications'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Accepted'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Denied'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Declined'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Pending'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HSNP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTELEM_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Applications'],
    "term_3_UTELEM_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Accepted'],
    "term_3_UTELEM_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTELEM_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Denied'],
    "term_3_UTELEM_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Declined'],
    "term_3_UTELEM_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Pending'],
    "term_3_UTELEM_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTELEM_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTELEM_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTELEM_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Applications'],
    "term_3_UTELEM_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Accepted'],
    "term_3_UTELEM_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_UTELEM_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Denied'],
    "term_3_UTELEM_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Declined'],
    "term_3_UTELEM_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Pending'],
    "term_3_UTELEM_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Waitlisted'],
    "term_3_UTELEM_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Enrolled'],
    "term_3_UTELEM_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTELEM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Applications'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Accepted'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Denied'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Declined'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Pending'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_UTSEC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Applications'],
    "term_3_UTSEC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Accepted'],
    "term_3_UTSEC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTSEC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Denied'],
    "term_3_UTSEC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Declined'],
    "term_3_UTSEC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Pending'],
    "term_3_UTSEC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTSEC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTSEC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTSEC_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Applications'],
    "term_3_UTSEC_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Accepted'],
    "term_3_UTSEC_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_UTSEC_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Denied'],
    "term_3_UTSEC_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Declined'],
    "term_3_UTSEC_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Pending'],
    "term_3_UTSEC_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Waitlisted'],
    "term_3_UTSEC_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Enrolled'],
    "term_3_UTSEC_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTSEC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Applications'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Accepted'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Denied'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Declined'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Pending'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_ENVTD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Applications'],
    "term_3_ENVTD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Accepted'],
    "term_3_ENVTD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ENVTD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Denied'],
    "term_3_ENVTD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Declined'],
    "term_3_ENVTD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Pending'],
    "term_3_ENVTD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ENVTD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Enrolled'],
    "term_3_ENVTD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Applications'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Denied'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Declined'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Pending'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ENVTD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Applications'],
    "term_3_ENVTD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Accepted'],
    "term_3_ENVTD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_ENVTD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Denied'],
    "term_3_ENVTD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Declined'],
    "term_3_ENVTD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Pending'],
    "term_3_ENVTD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_ENVTD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Enrolled'],
    "term_3_ENVTD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ENVTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Applications'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Denied'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Declined'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Pending'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_ENVNC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Applications'],
    "term_3_ENVNC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Accepted'],
    "term_3_ENVNC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ENVNC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Denied'],
    "term_3_ENVNC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Declined'],
    "term_3_ENVNC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Pending'],
    "term_3_ENVNC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ENVNC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Enrolled'],
    "term_3_ENVNC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Applications'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Denied'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Declined'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Pending'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ENVNC_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Applications'],
    "term_3_ENVNC_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Accepted'],
    "term_3_ENVNC_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_ENVNC_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Denied'],
    "term_3_ENVNC_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Declined'],
    "term_3_ENVNC_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Pending'],
    "term_3_ENVNC_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Waitlisted'],
    "term_3_ENVNC_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Enrolled'],
    "term_3_ENVNC_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ENVNC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Applications'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Accepted'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Denied'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Declined'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Pending'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_UTARTS_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Applications'],
    "term_3_UTARTS_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Accepted'],
    "term_3_UTARTS_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTARTS_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Denied'],
    "term_3_UTARTS_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Declined'],
    "term_3_UTARTS_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Pending'],
    "term_3_UTARTS_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTARTS_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTARTS_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTARTS_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Applications'],
    "term_3_UTARTS_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Accepted'],
    "term_3_UTARTS_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_UTARTS_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Denied'],
    "term_3_UTARTS_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Declined'],
    "term_3_UTARTS_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Pending'],
    "term_3_UTARTS_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Waitlisted'],
    "term_3_UTARTS_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Enrolled'],
    "term_3_UTARTS_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTARTS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Applications'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Accepted'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Denied'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Declined'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Pending'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Applications'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Accepted'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Denied'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Declined'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Pending'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Applications'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Accepted'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Denied'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Declined'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Pending'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Waitlisted'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Enrolled'],
    "term_3_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTCOMM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Applications'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Accepted'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Denied'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Declined'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Pending'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_UTENG_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Applications'],
    "term_3_UTENG_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Accepted'],
    "term_3_UTENG_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTENG_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Denied'],
    "term_3_UTENG_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Declined'],
    "term_3_UTENG_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Pending'],
    "term_3_UTENG_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTENG_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTENG_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTENG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTENG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Applications'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Accepted'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Denied'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Declined'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Pending'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Applications'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Accepted'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Denied'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Declined'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Pending'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Waitlisted'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Enrolled'],
    "term_3_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCGS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Applications'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Accepted'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Denied'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Declined'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Pending'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Applications'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Accepted'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Denied'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Declined'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Pending'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Waitlisted'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Enrolled'],
    "term_3_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Applications'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Accepted'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Denied'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Declined'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Pending'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Applications'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Accepted'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Denied'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Declined'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Pending'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Waitlisted'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Enrolled'],
    "term_3_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCWK_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Applications'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Accepted'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Denied'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Declined'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Pending'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_OPEN_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Applications'],
    "term_3_OPEN_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Accepted'],
    "term_3_OPEN_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_OPEN_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Denied'],
    "term_3_OPEN_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Declined'],
    "term_3_OPEN_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Pending'],
    "term_3_OPEN_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Waitlisted'],
    "term_3_OPEN_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Enrolled'],
    "term_3_OPEN_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_OPEN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Applications'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Accepted'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Denied'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Declined'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Pending'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_OPEN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Applications'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Accepted'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Denied'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Declined'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Pending'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Enrolled'],
    "term_3_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Applications'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Denied'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Declined'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Pending'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Applications'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Accepted'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Denied'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Declined'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Pending'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Waitlisted'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Enrolled'],
    "term_3_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_GOVNSC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Applications'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Accepted'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Denied'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Declined'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Pending'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_COMSCC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Applications'],
    "term_3_COMSCC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Accepted'],
    "term_3_COMSCC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_COMSCC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Denied'],
    "term_3_COMSCC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Declined'],
    "term_3_COMSCC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Pending'],
    "term_3_COMSCC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_COMSCC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Enrolled'],
    "term_3_COMSCC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_COMSCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Applications'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Denied'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Declined'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Pending'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_COMSCD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Applications'],
    "term_3_COMSCD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Accepted'],
    "term_3_COMSCD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_COMSCD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Denied'],
    "term_3_COMSCD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Declined'],
    "term_3_COMSCD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Pending'],
    "term_3_COMSCD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_COMSCD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Enrolled'],
    "term_3_COMSCD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Applications'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Denied'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Declined'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Pending'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_COMSCD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Applications'],
    "term_3_COMSCD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Accepted'],
    "term_3_COMSCD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_COMSCD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Denied'],
    "term_3_COMSCD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Declined'],
    "term_3_COMSCD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Pending'],
    "term_3_COMSCD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_COMSCD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Enrolled'],
    "term_3_COMSCD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_COMSCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Applications'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Denied'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Declined'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Pending'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_GNARTC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Applications'],
    "term_3_GNARTC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Accepted'],
    "term_3_GNARTC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_GNARTC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Denied'],
    "term_3_GNARTC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Declined'],
    "term_3_GNARTC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Pending'],
    "term_3_GNARTC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_GNARTC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Enrolled'],
    "term_3_GNARTC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GNARTC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Applications'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Denied'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Declined'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Pending'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Applications'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Accepted'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Denied'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Declined'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Pending'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Enrolled'],
    "term_3_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GNSCIC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Applications'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Denied'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Declined'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Pending'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Applications'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Accepted'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Denied'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Declined'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Pending'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Enrolled'],
    "term_3_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ANCPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Applications'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Denied'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Declined'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Pending'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_NSPDC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Applications'],
    "term_3_NSPDC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Accepted'],
    "term_3_NSPDC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_NSPDC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Denied'],
    "term_3_NSPDC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Declined'],
    "term_3_NSPDC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Pending'],
    "term_3_NSPDC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_NSPDC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Enrolled'],
    "term_3_NSPDC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_NSPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Applications'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Denied'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Declined'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Pending'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_AITS_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Applications'],
    "term_3_AITS_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Accepted'],
    "term_3_AITS_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_AITS_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Denied'],
    "term_3_AITS_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Declined'],
    "term_3_AITS_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Pending'],
    "term_3_AITS_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Waitlisted'],
    "term_3_AITS_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Enrolled'],
    "term_3_AITS_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_AITS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_AITS_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Applications'],
    "term_3_AITS_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Accepted'],
    "term_3_AITS_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_AITS_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Denied'],
    "term_3_AITS_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Declined'],
    "term_3_AITS_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Pending'],
    "term_3_AITS_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_AITS_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_AITS_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_AITS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_LINC13_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Applications'],
    "term_3_LINC13_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Accepted'],
    "term_3_LINC13_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_LINC13_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Denied'],
    "term_3_LINC13_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Declined'],
    "term_3_LINC13_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Pending'],
    "term_3_LINC13_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Waitlisted'],
    "term_3_LINC13_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Enrolled'],
    "term_3_LINC13_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_LINC13_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Applications'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Accepted'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Denied'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Declined'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Pending'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_LINC13_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_AELCC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Applications'],
    "term_3_AELCC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Accepted'],
    "term_3_AELCC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_AELCC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Denied'],
    "term_3_AELCC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Declined'],
    "term_3_AELCC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Pending'],
    "term_3_AELCC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_AELCC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Enrolled'],
    "term_3_AELCC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_AELCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Applications'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Denied'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Declined'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Pending'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_AELCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ECCERT_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Applications'],
    "term_3_ECCERT_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Accepted'],
    "term_3_ECCERT_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ECCERT_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Denied'],
    "term_3_ECCERT_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Declined'],
    "term_3_ECCERT_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Pending'],
    "term_3_ECCERT_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ECCERT_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Enrolled'],
    "term_3_ECCERT_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ECCERT_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Applications'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Denied'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Declined'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Pending'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ECDIP_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Applications'],
    "term_3_ECDIP_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Accepted'],
    "term_3_ECDIP_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ECDIP_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Denied'],
    "term_3_ECDIP_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Declined'],
    "term_3_ECDIP_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Pending'],
    "term_3_ECDIP_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ECDIP_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Enrolled'],
    "term_3_ECDIP_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Applications'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Denied'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Declined'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Pending'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ECDIP_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Applications'],
    "term_3_ECDIP_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Accepted'],
    "term_3_ECDIP_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_ECDIP_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Denied'],
    "term_3_ECDIP_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Declined'],
    "term_3_ECDIP_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Pending'],
    "term_3_ECDIP_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Waitlisted'],
    "term_3_ECDIP_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Enrolled'],
    "term_3_ECDIP_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ECDIP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Applications'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Accepted'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Denied'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Declined'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Pending'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_CYCD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Applications'],
    "term_3_CYCD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Accepted'],
    "term_3_CYCD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_CYCD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Denied'],
    "term_3_CYCD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Declined'],
    "term_3_CYCD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Pending'],
    "term_3_CYCD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_CYCD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Enrolled'],
    "term_3_CYCD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Applications'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Denied'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Declined'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Pending'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_CYCD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Applications'],
    "term_3_CYCD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Accepted'],
    "term_3_CYCD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_CYCD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Denied'],
    "term_3_CYCD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Declined'],
    "term_3_CYCD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Pending'],
    "term_3_CYCD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_CYCD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Enrolled'],
    "term_3_CYCD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_CYCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Applications'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Denied'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Declined'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Pending'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_CYCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_ECEAC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Applications'],
    "term_3_ECEAC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Accepted'],
    "term_3_ECEAC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ECEAC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Denied'],
    "term_3_ECEAC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Declined'],
    "term_3_ECEAC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Pending'],
    "term_3_ECEAC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ECEAC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Enrolled'],
    "term_3_ECEAC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ECEAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Applications'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Denied'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Declined'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Pending'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Applications'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Accepted'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Denied'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Declined'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Pending'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Enrolled'],
    "term_3_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Applications'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Denied'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Declined'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Pending'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Applications'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Accepted'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Denied'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Declined'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Pending'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Waitlisted'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Enrolled'],
    "term_3_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_SOCWKD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Applications'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Accepted'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Denied'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Declined'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Pending'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_HCAC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Applications'],
    "term_3_HCAC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Accepted'],
    "term_3_HCAC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_HCAC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Denied'],
    "term_3_HCAC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Declined'],
    "term_3_HCAC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Pending'],
    "term_3_HCAC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_HCAC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Enrolled'],
    "term_3_HCAC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HCAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Applications'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Denied'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Declined'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Pending'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_HCAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PCPC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Applications'],
    "term_3_PCPC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Accepted'],
    "term_3_PCPC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PCPC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Denied'],
    "term_3_PCPC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Declined'],
    "term_3_PCPC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Pending'],
    "term_3_PCPC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PCPC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Enrolled'],
    "term_3_PCPC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PCPC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Applications'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Denied'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Declined'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Pending'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PCPC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ACP_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Applications'],
    "term_3_ACP_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Accepted'],
    "term_3_ACP_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_ACP_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Denied'],
    "term_3_ACP_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Declined'],
    "term_3_ACP_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Pending'],
    "term_3_ACP_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Waitlisted'],
    "term_3_ACP_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Enrolled'],
    "term_3_ACP_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_ACP_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Applications'],
    "term_3_ACP_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Accepted'],
    "term_3_ACP_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_ACP_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Denied'],
    "term_3_ACP_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Declined'],
    "term_3_ACP_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Pending'],
    "term_3_ACP_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_ACP_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_ACP_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_ACP_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Applications'],
    "term_3_ACP_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Accepted'],
    "term_3_ACP_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_ACP_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Denied'],
    "term_3_ACP_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Declined'],
    "term_3_ACP_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Pending'],
    "term_3_ACP_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Waitlisted'],
    "term_3_ACP_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Enrolled'],
    "term_3_ACP_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ACP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_ACP_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Applications'],
    "term_3_ACP_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Accepted'],
    "term_3_ACP_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_ACP_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Denied'],
    "term_3_ACP_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Declined'],
    "term_3_ACP_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Pending'],
    "term_3_ACP_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_ACP_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_ACP_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_ACP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_PNR_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Applications'],
    "term_3_PNR_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Accepted'],
    "term_3_PNR_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PNR_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Denied'],
    "term_3_PNR_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Declined'],
    "term_3_PNR_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Pending'],
    "term_3_PNR_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PNR_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Enrolled'],
    "term_3_PNR_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PNR_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Applications'],
    "term_3_PNR_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PNR_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PNR_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Denied'],
    "term_3_PNR_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Declined'],
    "term_3_PNR_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Pending'],
    "term_3_PNR_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PNR_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PNR_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PNR_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Applications'],
    "term_3_PNR_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Accepted'],
    "term_3_PNR_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_PNR_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Denied'],
    "term_3_PNR_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Declined'],
    "term_3_PNR_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Pending'],
    "term_3_PNR_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Waitlisted'],
    "term_3_PNR_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Enrolled'],
    "term_3_PNR_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_PNR_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_PNR_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Applications'],
    "term_3_PNR_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Accepted'],
    "term_3_PNR_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_PNR_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Denied'],
    "term_3_PNR_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Declined'],
    "term_3_PNR_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Pending'],
    "term_3_PNR_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_PNR_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_PNR_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_PNR_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_BSCN_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Applications'],
    "term_3_BSCN_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Accepted'],
    "term_3_BSCN_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_BSCN_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Denied'],
    "term_3_BSCN_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Declined'],
    "term_3_BSCN_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Pending'],
    "term_3_BSCN_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Waitlisted'],
    "term_3_BSCN_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Enrolled'],
    "term_3_BSCN_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Applications'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Accepted'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Denied'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Declined'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Pending'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_BSCN_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Applications'],
    "term_3_BSCN_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Accepted'],
    "term_3_BSCN_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_BSCN_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Denied'],
    "term_3_BSCN_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Declined'],
    "term_3_BSCN_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Pending'],
    "term_3_BSCN_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Waitlisted'],
    "term_3_BSCN_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Enrolled'],
    "term_3_BSCN_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BSCN_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Applications'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Accepted'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Denied'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Declined'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Pending'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_BSCN_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Applications": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Applications'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Accepted": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Accepted'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Payment_Pending": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Payment_Pending'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Denied": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Denied'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Declined": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Declined'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Pending": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Pending'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Waitlisted": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Waitlisted'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Enrolled": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Enrolled'],
    "term_3_NursingYr3_DOMESTIC_Year_3_Conversion_Rate": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_DOMESTIC_Year_3_Conversion_Rate'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Applications": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Applications'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Accepted": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Accepted'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Denied": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Denied'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Declined": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Declined'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Pending": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Pending'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Waitlisted": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Waitlisted'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Enrolled": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Enrolled'],
    "term_3_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate": term_3_NursingYr3_variable_dictionary['term_3_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate'],
    "term_3_IENCN_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Applications'],
    "term_3_IENCN_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Accepted'],
    "term_3_IENCN_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_IENCN_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Denied'],
    "term_3_IENCN_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Declined'],
    "term_3_IENCN_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Pending'],
    "term_3_IENCN_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Waitlisted'],
    "term_3_IENCN_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Enrolled'],
    "term_3_IENCN_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_IENCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Applications'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Accepted'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Denied'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Declined'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Pending'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_IENCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PAELEC_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Applications'],
    "term_3_PAELEC_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Accepted'],
    "term_3_PAELEC_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PAELEC_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Denied'],
    "term_3_PAELEC_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Declined'],
    "term_3_PAELEC_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Pending'],
    "term_3_PAELEC_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PAELEC_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Enrolled'],
    "term_3_PAELEC_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAELEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Applications'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Denied'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Declined'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Pending'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PAWELD_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Applications'],
    "term_3_PAWELD_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Accepted'],
    "term_3_PAWELD_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PAWELD_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Denied'],
    "term_3_PAWELD_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Declined'],
    "term_3_PAWELD_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Pending'],
    "term_3_PAWELD_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PAWELD_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Enrolled'],
    "term_3_PAWELD_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAWELD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Applications'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Denied'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Declined'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Pending'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PAHET_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Applications'],
    "term_3_PAHET_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Accepted'],
    "term_3_PAHET_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PAHET_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Denied'],
    "term_3_PAHET_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Declined'],
    "term_3_PAHET_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Pending'],
    "term_3_PAHET_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PAHET_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Enrolled'],
    "term_3_PAHET_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAHET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Applications'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Denied'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Declined'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Pending'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAHET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Applications'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Accepted'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Denied'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Declined'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Pending'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Enrolled'],
    "term_3_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAPLUM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Applications'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Denied'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Declined'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Pending'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_POWCM4_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Applications'],
    "term_3_POWCM4_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Accepted'],
    "term_3_POWCM4_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_POWCM4_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Denied'],
    "term_3_POWCM4_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Declined'],
    "term_3_POWCM4_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Pending'],
    "term_3_POWCM4_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Waitlisted'],
    "term_3_POWCM4_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Enrolled'],
    "term_3_POWCM4_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCM4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Applications'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Accepted'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Denied'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Declined'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Pending'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_POWCM3_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Applications'],
    "term_3_POWCM3_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Accepted'],
    "term_3_POWCM3_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_POWCM3_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Denied'],
    "term_3_POWCM3_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Declined'],
    "term_3_POWCM3_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Pending'],
    "term_3_POWCM3_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Waitlisted'],
    "term_3_POWCM3_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Enrolled'],
    "term_3_POWCM3_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCM3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Applications'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Accepted'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Denied'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Declined'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Pending'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_POWCP4_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Applications'],
    "term_3_POWCP4_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Accepted'],
    "term_3_POWCP4_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_POWCP4_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Denied'],
    "term_3_POWCP4_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Declined'],
    "term_3_POWCP4_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Pending'],
    "term_3_POWCP4_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Waitlisted'],
    "term_3_POWCP4_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Enrolled'],
    "term_3_POWCP4_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCP4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Applications'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Accepted'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Denied'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Declined'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Pending'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_POWCO4_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Applications'],
    "term_3_POWCO4_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Accepted'],
    "term_3_POWCO4_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_POWCO4_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Denied'],
    "term_3_POWCO4_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Declined'],
    "term_3_POWCO4_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Pending'],
    "term_3_POWCO4_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Waitlisted'],
    "term_3_POWCO4_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Enrolled'],
    "term_3_POWCO4_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCO4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Applications'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Accepted'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Denied'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Declined'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Pending'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_POWCO3_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Applications'],
    "term_3_POWCO3_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Accepted'],
    "term_3_POWCO3_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_POWCO3_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Denied'],
    "term_3_POWCO3_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Declined'],
    "term_3_POWCO3_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Pending'],
    "term_3_POWCO3_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Waitlisted'],
    "term_3_POWCO3_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Enrolled'],
    "term_3_POWCO3_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCO3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Applications'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Accepted'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Denied'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Declined'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Pending'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PPET_DOMESTIC_Year_1_Applications": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Applications'],
    "term_3_PPET_DOMESTIC_Year_1_Accepted": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Accepted'],
    "term_3_PPET_DOMESTIC_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Payment_Pending'],
    "term_3_PPET_DOMESTIC_Year_1_Denied": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Denied'],
    "term_3_PPET_DOMESTIC_Year_1_Declined": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Declined'],
    "term_3_PPET_DOMESTIC_Year_1_Pending": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Pending'],
    "term_3_PPET_DOMESTIC_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Waitlisted'],
    "term_3_PPET_DOMESTIC_Year_1_Enrolled": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Enrolled'],
    "term_3_PPET_DOMESTIC_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_3_PPET_INTERNATIONAL_Year_1_Applications": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Applications'],
    "term_3_PPET_INTERNATIONAL_Year_1_Accepted": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Accepted'],
    "term_3_PPET_INTERNATIONAL_Year_1_Payment_Pending": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_3_PPET_INTERNATIONAL_Year_1_Denied": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Denied'],
    "term_3_PPET_INTERNATIONAL_Year_1_Declined": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Declined'],
    "term_3_PPET_INTERNATIONAL_Year_1_Pending": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Pending'],
    "term_3_PPET_INTERNATIONAL_Year_1_Waitlisted": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_3_PPET_INTERNATIONAL_Year_1_Enrolled": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Enrolled'],
    "term_3_PPET_INTERNATIONAL_Year_1_Conversion_Rate": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_3_PPET_DOMESTIC_Year_2_Applications": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Applications'],
    "term_3_PPET_DOMESTIC_Year_2_Accepted": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Accepted'],
    "term_3_PPET_DOMESTIC_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Payment_Pending'],
    "term_3_PPET_DOMESTIC_Year_2_Denied": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Denied'],
    "term_3_PPET_DOMESTIC_Year_2_Declined": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Declined'],
    "term_3_PPET_DOMESTIC_Year_2_Pending": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Pending'],
    "term_3_PPET_DOMESTIC_Year_2_Waitlisted": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Waitlisted'],
    "term_3_PPET_DOMESTIC_Year_2_Enrolled": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Enrolled'],
    "term_3_PPET_DOMESTIC_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_PPET_DOMESTIC_Year_2_Conversion_Rate'],
    "term_3_PPET_INTERNATIONAL_Year_2_Applications": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Applications'],
    "term_3_PPET_INTERNATIONAL_Year_2_Accepted": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Accepted'],
    "term_3_PPET_INTERNATIONAL_Year_2_Payment_Pending": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_3_PPET_INTERNATIONAL_Year_2_Denied": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Denied'],
    "term_3_PPET_INTERNATIONAL_Year_2_Declined": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Declined'],
    "term_3_PPET_INTERNATIONAL_Year_2_Pending": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Pending'],
    "term_3_PPET_INTERNATIONAL_Year_2_Waitlisted": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Waitlisted'],
    "term_3_PPET_INTERNATIONAL_Year_2_Enrolled": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Enrolled'],
    "term_3_PPET_INTERNATIONAL_Year_2_Conversion_Rate": term_3_variable_dictionary['term_3_PPET_INTERNATIONAL_Year_2_Conversion_Rate'],

    #Term4Yr YYYY Apps By School sheet
    "term_4_Total_Business_Administration_Applications": term_4_Total_Business_Administration_Applications,
    "term_4_Total_Business_Administration_Accepted": term_4_Total_Business_Administration_Accepted,
    "term_4_Total_Business_Administration_Payment_Pending": term_4_Total_Business_Administration_Payment_Pending,
    "term_4_Total_Business_Administration_Denied": term_4_Total_Business_Administration_Denied,
    "term_4_Total_Business_Administration_Declined": term_4_Total_Business_Administration_Declined,
    "term_4_Total_Business_Administration_Pending": term_4_Total_Business_Administration_Pending,
    "term_4_Total_Business_Administration_Waitlisted": term_4_Total_Business_Administration_Waitlisted,
    "term_4_Total_Business_Administration_Enrolled": term_4_Total_Business_Administration_Enrolled,
    "term_4_Total_Business_Administration_Conversion_Rate": term_4_Total_Business_Administration_Conversion_Rate,
    "term_4_Total_Education_and_Academic_Upgrading_Applications": term_4_Total_Education_and_Academic_Upgrading_Applications,
    "term_4_Total_Education_and_Academic_Upgrading_Accepted": term_4_Total_Education_and_Academic_Upgrading_Accepted,
    "term_4_Total_Education_and_Academic_Upgrading_Payment_Pending": term_4_Total_Education_and_Academic_Upgrading_Payment_Pending,
    "term_4_Total_Education_and_Academic_Upgrading_Denied": term_4_Total_Education_and_Academic_Upgrading_Denied,
    "term_4_Total_Education_and_Academic_Upgrading_Declined": term_4_Total_Education_and_Academic_Upgrading_Declined,
    "term_4_Total_Education_and_Academic_Upgrading_Pending": term_4_Total_Education_and_Academic_Upgrading_Pending,
    "term_4_Total_Education_and_Academic_Upgrading_Waitlisted": term_4_Total_Education_and_Academic_Upgrading_Waitlisted,
    "term_4_Total_Education_and_Academic_Upgrading_Enrolled": term_4_Total_Education_and_Academic_Upgrading_Enrolled,
    "term_4_Total_Education_and_Academic_Upgrading_Conversion_Rate": term_4_Total_Education_and_Academic_Upgrading_Conversion_Rate,
    "term_4_Total_University_Studies_and_Environmental_Science_Applications": term_4_Total_University_Studies_and_Environmental_Science_Applications,
    "term_4_Total_University_Studies_and_Environmental_Science_Accepted": term_4_Total_University_Studies_and_Environmental_Science_Accepted,
    "term_4_Total_University_Studies_and_Environmental_Science_Payment_Pending": term_4_Total_University_Studies_and_Environmental_Science_Payment_Pending,
    "term_4_Total_University_Studies_and_Environmental_Science_Denied": term_4_Total_University_Studies_and_Environmental_Science_Denied,
    "term_4_Total_University_Studies_and_Environmental_Science_Declined": term_4_Total_University_Studies_and_Environmental_Science_Declined,
    "term_4_Total_University_Studies_and_Environmental_Science_Pending": term_4_Total_University_Studies_and_Environmental_Science_Pending,
    "term_4_Total_University_Studies_and_Environmental_Science_Waitlisted": term_4_Total_University_Studies_and_Environmental_Science_Waitlisted,
    "term_4_Total_University_Studies_and_Environmental_Science_Enrolled": term_4_Total_University_Studies_and_Environmental_Science_Enrolled,
    "term_4_Total_University_Studies_and_Environmental_Science_Conversion_Rate": term_4_Total_University_Studies_and_Environmental_Science_Conversion_Rate,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Applications": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Applications,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Accepted,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Payment_Pending,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Denied": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Denied,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Declined": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Declined,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Pending": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Pending,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Waitlisted,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Enrolled,
    "term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate": term_4_Total_Language_Instruction_for_Newcomers_to_Canada_Conversion_Rate,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Applications,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Accepted,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Payment_Pending,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Denied,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Declined,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Pending,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Waitlisted,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Enrolled,
    "term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate": term_4_Total_School_of_University_Studies_Career_Programs_and_Academic_Upgrading_Conversion_Rate,
    "term_4_Total_Childhood_Studies_and_Human_Services_Applications": term_4_Total_Childhood_Studies_and_Human_Services_Applications,
    "term_4_Total_Childhood_Studies_and_Human_Services_Accepted": term_4_Total_Childhood_Studies_and_Human_Services_Accepted,
    "term_4_Total_Childhood_Studies_and_Human_Services_Payment_Pending": term_4_Total_Childhood_Studies_and_Human_Services_Payment_Pending,
    "term_4_Total_Childhood_Studies_and_Human_Services_Denied": term_4_Total_Childhood_Studies_and_Human_Services_Denied,
    "term_4_Total_Childhood_Studies_and_Human_Services_Declined": term_4_Total_Childhood_Studies_and_Human_Services_Declined,
    "term_4_Total_Childhood_Studies_and_Human_Services_Pending": term_4_Total_Childhood_Studies_and_Human_Services_Pending,
    "term_4_Total_Childhood_Studies_and_Human_Services_Waitlisted": term_4_Total_Childhood_Studies_and_Human_Services_Waitlisted,
    "term_4_Total_Childhood_Studies_and_Human_Services_Enrolled": term_4_Total_Childhood_Studies_and_Human_Services_Enrolled,
    "term_4_Total_Childhood_Studies_and_Human_Services_Conversion_Rate": term_4_Total_Childhood_Studies_and_Human_Services_Conversion_Rate,
    "term_4_Total_Nursing_and_Allied_Health_Applications": term_4_Total_Nursing_and_Allied_Health_Applications,
    "term_4_Total_Nursing_and_Allied_Health_Accepted": term_4_Total_Nursing_and_Allied_Health_Accepted,
    "term_4_Total_Nursing_and_Allied_Health_Payment_Pending": term_4_Total_Nursing_and_Allied_Health_Payment_Pending,
    "term_4_Total_Nursing_and_Allied_Health_Denied": term_4_Total_Nursing_and_Allied_Health_Denied,
    "term_4_Total_Nursing_and_Allied_Health_Declined": term_4_Total_Nursing_and_Allied_Health_Declined,
    "term_4_Total_Nursing_and_Allied_Health_Pending": term_4_Total_Nursing_and_Allied_Health_Pending,
    "term_4_Total_Nursing_and_Allied_Health_Waitlisted": term_4_Total_Nursing_and_Allied_Health_Waitlisted,
    "term_4_Total_Nursing_and_Allied_Health_Enrolled": term_4_Total_Nursing_and_Allied_Health_Enrolled,
    "term_4_Total_Nursing_and_Allied_Health_Conversion_Rate": term_4_Total_Nursing_and_Allied_Health_Conversion_Rate,
    "term_4_Total_School_of_Health_and_Human_Services_Applications": term_4_Total_School_of_Health_and_Human_Services_Applications,
    "term_4_Total_School_of_Health_and_Human_Services_Accepted": term_4_Total_School_of_Health_and_Human_Services_Accepted,
    "term_4_Total_School_of_Health_and_Human_Services_Payment_Pending": term_4_Total_School_of_Health_and_Human_Services_Payment_Pending,
    "term_4_Total_School_of_Health_and_Human_Services_Denied": term_4_Total_School_of_Health_and_Human_Services_Denied,
    "term_4_Total_School_of_Health_and_Human_Services_Declined": term_4_Total_School_of_Health_and_Human_Services_Declined,
    "term_4_Total_School_of_Health_and_Human_Services_Pending": term_4_Total_School_of_Health_and_Human_Services_Pending,
    "term_4_Total_School_of_Health_and_Human_Services_Waitlisted": term_4_Total_School_of_Health_and_Human_Services_Waitlisted,
    "term_4_Total_School_of_Health_and_Human_Services_Enrolled": term_4_Total_School_of_Health_and_Human_Services_Enrolled,
    "term_4_Total_School_of_Health_and_Human_Services_Conversion_Rate": term_4_Total_School_of_Health_and_Human_Services_Conversion_Rate,
    "term_4_Total_Pre_employment_Applications": term_4_Total_Pre_employment_Applications,
    "term_4_Total_Pre_employment_Accepted": term_4_Total_Pre_employment_Accepted,
    "term_4_Total_Pre_employment_Payment_Pending": term_4_Total_Pre_employment_Payment_Pending,
    "term_4_Total_Pre_employment_Denied": term_4_Total_Pre_employment_Denied,
    "term_4_Total_Pre_employment_Declined": term_4_Total_Pre_employment_Declined,
    "term_4_Total_Pre_employment_Pending": term_4_Total_Pre_employment_Pending,
    "term_4_Total_Pre_employment_Waitlisted": term_4_Total_Pre_employment_Waitlisted,
    "term_4_Total_Pre_employment_Enrolled": term_4_Total_Pre_employment_Enrolled,
    "term_4_Total_Pre_employment_Conversion_Rate": term_4_Total_Pre_employment_Conversion_Rate,
    "term_4_Total_Power_Engineering_Applications": term_4_Total_Power_Engineering_Applications,
    "term_4_Total_Power_Engineering_Accepted": term_4_Total_Power_Engineering_Accepted,
    "term_4_Total_Power_Engineering_Payment_Pending": term_4_Total_Power_Engineering_Payment_Pending,
    "term_4_Total_Power_Engineering_Denied": term_4_Total_Power_Engineering_Denied,
    "term_4_Total_Power_Engineering_Declined": term_4_Total_Power_Engineering_Declined,
    "term_4_Total_Power_Engineering_Pending": term_4_Total_Power_Engineering_Pending,
    "term_4_Total_Power_Engineering_Waitlisted": term_4_Total_Power_Engineering_Waitlisted,
    "term_4_Total_Power_Engineering_Enrolled": term_4_Total_Power_Engineering_Enrolled,
    "term_4_Total_Power_Engineering_Conversion_Rate": term_4_Total_Power_Engineering_Conversion_Rate,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Applications": term_4_Total_School_of_Trades_and_Heavy_Industrial_Applications,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Accepted": term_4_Total_School_of_Trades_and_Heavy_Industrial_Accepted,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending": term_4_Total_School_of_Trades_and_Heavy_Industrial_Payment_Pending,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Denied": term_4_Total_School_of_Trades_and_Heavy_Industrial_Denied,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Declined": term_4_Total_School_of_Trades_and_Heavy_Industrial_Declined,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Pending": term_4_Total_School_of_Trades_and_Heavy_Industrial_Pending,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted": term_4_Total_School_of_Trades_and_Heavy_Industrial_Waitlisted,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Enrolled": term_4_Total_School_of_Trades_and_Heavy_Industrial_Enrolled,
    "term_4_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate": term_4_Total_School_of_Trades_and_Heavy_Industrial_Conversion_Rate,
    "term_4_Keyano_College_Total_Applications": term_4_Keyano_College_Total_Applications,
    "term_4_Keyano_College_Total_Accepted": term_4_Keyano_College_Total_Accepted,
    "term_4_Keyano_College_Total_Payment_Pending": term_4_Keyano_College_Total_Payment_Pending,
    "term_4_Keyano_College_Total_Denied": term_4_Keyano_College_Total_Denied,
    "term_4_Keyano_College_Total_Declined": term_4_Keyano_College_Total_Declined,
    "term_4_Keyano_College_Total_Pending": term_4_Keyano_College_Total_Pending,
    "term_4_Keyano_College_Total_Waitlisted": term_4_Keyano_College_Total_Waitlisted,
    "term_4_Keyano_College_Total_Enrolled": term_4_Keyano_College_Total_Enrolled,
    "term_4_Keyano_College_Total_Conversion_Rate": term_4_Keyano_College_Total_Conversion_Rate,
    "term_4_BABUSC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Applications'],
    "term_4_BABUSC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Accepted'],
    "term_4_BABUSC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BABUSC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Denied'],
    "term_4_BABUSC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Declined'],
    "term_4_BABUSC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Pending'],
    "term_4_BABUSC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BABUSC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Enrolled'],
    "term_4_BABUSC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BABUSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Applications'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Denied'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Declined'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Pending'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BABUSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BAACTD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Applications'],
    "term_4_BAACTD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Accepted'],
    "term_4_BAACTD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BAACTD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Denied'],
    "term_4_BAACTD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Declined'],
    "term_4_BAACTD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Pending'],
    "term_4_BAACTD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BAACTD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Enrolled'],
    "term_4_BAACTD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Applications'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Denied'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Declined'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Pending'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BAACTD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Applications'],
    "term_4_BAACTD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Accepted'],
    "term_4_BAACTD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BAACTD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Denied'],
    "term_4_BAACTD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Declined'],
    "term_4_BAACTD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Pending'],
    "term_4_BAACTD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BAACTD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Enrolled'],
    "term_4_BAACTD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAACTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Applications'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Denied'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Declined'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Pending'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAACTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BABUSD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Applications'],
    "term_4_BABUSD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Accepted'],
    "term_4_BABUSD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BABUSD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Denied'],
    "term_4_BABUSD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Declined'],
    "term_4_BABUSD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Pending'],
    "term_4_BABUSD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BABUSD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Enrolled'],
    "term_4_BABUSD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Applications'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Denied'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Declined'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Pending'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BABUSD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Applications'],
    "term_4_BABUSD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Accepted'],
    "term_4_BABUSD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BABUSD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Denied'],
    "term_4_BABUSD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Declined'],
    "term_4_BABUSD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Pending'],
    "term_4_BABUSD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BABUSD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Enrolled'],
    "term_4_BABUSD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BABUSD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Applications'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Denied'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Declined'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Pending'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BABUSD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BAESPM_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Applications'],
    "term_4_BAESPM_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Accepted'],
    "term_4_BAESPM_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BAESPM_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Denied'],
    "term_4_BAESPM_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Declined'],
    "term_4_BAESPM_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Pending'],
    "term_4_BAESPM_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BAESPM_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Enrolled'],
    "term_4_BAESPM_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Applications'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Denied'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Declined'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Pending'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BAESPM_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Applications'],
    "term_4_BAESPM_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Accepted'],
    "term_4_BAESPM_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BAESPM_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Denied'],
    "term_4_BAESPM_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Declined'],
    "term_4_BAESPM_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Pending'],
    "term_4_BAESPM_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BAESPM_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Enrolled'],
    "term_4_BAESPM_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAESPM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Applications'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Denied'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Declined'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Pending'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAESPM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Applications'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Accepted'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Denied'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Declined'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Pending'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Enrolled'],
    "term_4_BAHRMD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Applications'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Denied'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Declined'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Pending'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Applications'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Accepted'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Denied'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Declined'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Pending'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Enrolled'],
    "term_4_BAHRMD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAHRMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Applications'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Denied'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Declined'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Pending'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAHRMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_GSCMD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Applications'],
    "term_4_GSCMD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Accepted'],
    "term_4_GSCMD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_GSCMD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Denied'],
    "term_4_GSCMD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Declined'],
    "term_4_GSCMD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Pending'],
    "term_4_GSCMD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_GSCMD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Enrolled'],
    "term_4_GSCMD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Applications'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Denied'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Declined'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Pending'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_GSCMD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Applications'],
    "term_4_GSCMD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Accepted'],
    "term_4_GSCMD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_GSCMD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Denied'],
    "term_4_GSCMD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Declined'],
    "term_4_GSCMD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Pending'],
    "term_4_GSCMD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_GSCMD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Enrolled'],
    "term_4_GSCMD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_GSCMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Applications'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Denied'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Declined'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Pending'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_GSCMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_DIGMD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Applications'],
    "term_4_DIGMD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Accepted'],
    "term_4_DIGMD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_DIGMD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Denied'],
    "term_4_DIGMD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Declined'],
    "term_4_DIGMD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Pending'],
    "term_4_DIGMD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_DIGMD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Enrolled'],
    "term_4_DIGMD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Applications'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Denied'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Declined'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Pending'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_DIGMD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Applications'],
    "term_4_DIGMD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Accepted'],
    "term_4_DIGMD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_DIGMD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Denied'],
    "term_4_DIGMD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Declined'],
    "term_4_DIGMD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Pending'],
    "term_4_DIGMD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_DIGMD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Enrolled'],
    "term_4_DIGMD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_DIGMD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Applications'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Denied'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Declined'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Pending'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_DIGMD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BAATCO_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Applications'],
    "term_4_BAATCO_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Accepted'],
    "term_4_BAATCO_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BAATCO_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Denied'],
    "term_4_BAATCO_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Declined'],
    "term_4_BAATCO_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Pending'],
    "term_4_BAATCO_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BAATCO_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Enrolled'],
    "term_4_BAATCO_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Applications'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Denied'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Declined'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Pending'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BAATCO_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Applications'],
    "term_4_BAATCO_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Accepted'],
    "term_4_BAATCO_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BAATCO_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Denied'],
    "term_4_BAATCO_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Declined'],
    "term_4_BAATCO_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Pending'],
    "term_4_BAATCO_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BAATCO_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Enrolled'],
    "term_4_BAATCO_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAATCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Applications'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Denied'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Declined'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Pending'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BAATCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BABSCO_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Applications'],
    "term_4_BABSCO_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Accepted'],
    "term_4_BABSCO_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BABSCO_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Denied'],
    "term_4_BABSCO_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Declined'],
    "term_4_BABSCO_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Pending'],
    "term_4_BABSCO_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BABSCO_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Enrolled'],
    "term_4_BABSCO_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Applications'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Denied'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Declined'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Pending'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BABSCO_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Applications'],
    "term_4_BABSCO_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Accepted'],
    "term_4_BABSCO_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BABSCO_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Denied'],
    "term_4_BABSCO_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Declined'],
    "term_4_BABSCO_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Pending'],
    "term_4_BABSCO_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BABSCO_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Enrolled'],
    "term_4_BABSCO_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BABSCO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Applications'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Denied'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Declined'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Pending'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BABSCO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_NAO_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Applications'],
    "term_4_NAO_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Accepted'],
    "term_4_NAO_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_NAO_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Denied'],
    "term_4_NAO_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Declined'],
    "term_4_NAO_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Pending'],
    "term_4_NAO_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Waitlisted'],
    "term_4_NAO_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Enrolled'],
    "term_4_NAO_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_NAO_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Applications'],
    "term_4_NAO_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Accepted'],
    "term_4_NAO_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_NAO_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Denied'],
    "term_4_NAO_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Declined'],
    "term_4_NAO_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Pending'],
    "term_4_NAO_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_NAO_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_NAO_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_NAO_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Applications'],
    "term_4_NAO_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Accepted'],
    "term_4_NAO_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_NAO_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Denied'],
    "term_4_NAO_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Declined'],
    "term_4_NAO_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Pending'],
    "term_4_NAO_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Waitlisted'],
    "term_4_NAO_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Enrolled'],
    "term_4_NAO_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_NAO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_NAO_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Applications'],
    "term_4_NAO_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Accepted'],
    "term_4_NAO_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_NAO_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Denied'],
    "term_4_NAO_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Declined'],
    "term_4_NAO_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Pending'],
    "term_4_NAO_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_NAO_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_NAO_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_NAO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_ALO_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Applications'],
    "term_4_ALO_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Accepted'],
    "term_4_ALO_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ALO_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Denied'],
    "term_4_ALO_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Declined'],
    "term_4_ALO_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Pending'],
    "term_4_ALO_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ALO_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Enrolled'],
    "term_4_ALO_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ALO_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Applications'],
    "term_4_ALO_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ALO_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ALO_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Denied'],
    "term_4_ALO_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Declined'],
    "term_4_ALO_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Pending'],
    "term_4_ALO_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ALO_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ALO_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ALO_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Applications'],
    "term_4_ALO_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Accepted'],
    "term_4_ALO_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_ALO_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Denied'],
    "term_4_ALO_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Declined'],
    "term_4_ALO_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Pending'],
    "term_4_ALO_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Waitlisted'],
    "term_4_ALO_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Enrolled'],
    "term_4_ALO_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ALO_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_ALO_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Applications'],
    "term_4_ALO_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Accepted'],
    "term_4_ALO_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_ALO_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Denied'],
    "term_4_ALO_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Declined'],
    "term_4_ALO_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Pending'],
    "term_4_ALO_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_ALO_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_ALO_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ALO_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Applications'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Accepted'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Denied'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Declined'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Pending'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Waitlisted'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Enrolled'],
    "term_4_HOSMNG_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Applications'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Accepted'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Denied'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Declined'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Pending'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Applications'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Accepted'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Denied'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Declined'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Pending'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Waitlisted'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Enrolled'],
    "term_4_HOSMNG_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_HOSMNG_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Applications'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Accepted'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Denied'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Declined'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Pending'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_HOSMNG_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Applications'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Accepted'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Denied'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Declined'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Pending'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Enrolled'],
    "term_4_BAHRMC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAHRMC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Applications'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Denied'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Declined'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Pending'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BAHRMC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_OFAC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Applications'],
    "term_4_OFAC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Accepted'],
    "term_4_OFAC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_OFAC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Denied'],
    "term_4_OFAC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Declined'],
    "term_4_OFAC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Pending'],
    "term_4_OFAC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_OFAC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Enrolled'],
    "term_4_OFAC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_OFAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Applications'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Denied'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Declined'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Pending'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_OFAC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_OFAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_EAP_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Applications'],
    "term_4_EAP_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Accepted'],
    "term_4_EAP_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_EAP_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Denied'],
    "term_4_EAP_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Declined'],
    "term_4_EAP_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Pending'],
    "term_4_EAP_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Waitlisted'],
    "term_4_EAP_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Enrolled'],
    "term_4_EAP_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_EAP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_EAP_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Applications'],
    "term_4_EAP_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Accepted'],
    "term_4_EAP_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_EAP_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Denied'],
    "term_4_EAP_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Declined'],
    "term_4_EAP_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Pending'],
    "term_4_EAP_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_EAP_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_EAP_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_EAP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_HSAFA_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Applications'],
    "term_4_HSAFA_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Accepted'],
    "term_4_HSAFA_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_HSAFA_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Denied'],
    "term_4_HSAFA_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Declined'],
    "term_4_HSAFA_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Pending'],
    "term_4_HSAFA_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Waitlisted'],
    "term_4_HSAFA_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Enrolled'],
    "term_4_HSAFA_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSAFA_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Applications'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Accepted'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Denied'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Declined'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Pending'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSAFA_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_HSAD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Applications'],
    "term_4_HSAD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Accepted'],
    "term_4_HSAD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_HSAD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Denied'],
    "term_4_HSAD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Declined'],
    "term_4_HSAD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Pending'],
    "term_4_HSAD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_HSAD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Enrolled'],
    "term_4_HSAD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSAD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Applications'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Denied'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Declined'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Pending'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_HSAD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSAD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_HSGD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Applications'],
    "term_4_HSGD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Accepted'],
    "term_4_HSGD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_HSGD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Denied'],
    "term_4_HSGD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Declined'],
    "term_4_HSGD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Pending'],
    "term_4_HSGD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_HSGD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Enrolled'],
    "term_4_HSGD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSGD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Applications'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Denied'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Declined'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Pending'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_HSGD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSGD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_HSNP_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Applications'],
    "term_4_HSNP_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Accepted'],
    "term_4_HSNP_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_HSNP_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Denied'],
    "term_4_HSNP_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Declined'],
    "term_4_HSNP_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Pending'],
    "term_4_HSNP_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Waitlisted'],
    "term_4_HSNP_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Enrolled'],
    "term_4_HSNP_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSNP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Applications'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Accepted'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Denied'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Declined'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Pending'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_HSNP_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HSNP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTELEM_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Applications'],
    "term_4_UTELEM_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Accepted'],
    "term_4_UTELEM_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTELEM_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Denied'],
    "term_4_UTELEM_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Declined'],
    "term_4_UTELEM_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Pending'],
    "term_4_UTELEM_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTELEM_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTELEM_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTELEM_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Applications'],
    "term_4_UTELEM_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Accepted'],
    "term_4_UTELEM_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_UTELEM_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Denied'],
    "term_4_UTELEM_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Declined'],
    "term_4_UTELEM_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Pending'],
    "term_4_UTELEM_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Waitlisted'],
    "term_4_UTELEM_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Enrolled'],
    "term_4_UTELEM_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTELEM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Applications'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Accepted'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Denied'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Declined'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Pending'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTELEM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_UTSEC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Applications'],
    "term_4_UTSEC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Accepted'],
    "term_4_UTSEC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTSEC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Denied'],
    "term_4_UTSEC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Declined'],
    "term_4_UTSEC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Pending'],
    "term_4_UTSEC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTSEC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTSEC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTSEC_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Applications'],
    "term_4_UTSEC_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Accepted'],
    "term_4_UTSEC_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_UTSEC_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Denied'],
    "term_4_UTSEC_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Declined'],
    "term_4_UTSEC_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Pending'],
    "term_4_UTSEC_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Waitlisted'],
    "term_4_UTSEC_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Enrolled'],
    "term_4_UTSEC_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTSEC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Applications'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Accepted'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Denied'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Declined'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Pending'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTSEC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_ENVTD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Applications'],
    "term_4_ENVTD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Accepted'],
    "term_4_ENVTD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ENVTD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Denied'],
    "term_4_ENVTD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Declined'],
    "term_4_ENVTD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Pending'],
    "term_4_ENVTD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ENVTD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Enrolled'],
    "term_4_ENVTD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Applications'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Denied'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Declined'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Pending'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ENVTD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Applications'],
    "term_4_ENVTD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Accepted'],
    "term_4_ENVTD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_ENVTD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Denied'],
    "term_4_ENVTD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Declined'],
    "term_4_ENVTD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Pending'],
    "term_4_ENVTD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_ENVTD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Enrolled'],
    "term_4_ENVTD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ENVTD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Applications'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Denied'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Declined'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Pending'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ENVTD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_ENVNC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Applications'],
    "term_4_ENVNC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Accepted'],
    "term_4_ENVNC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ENVNC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Denied'],
    "term_4_ENVNC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Declined'],
    "term_4_ENVNC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Pending'],
    "term_4_ENVNC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ENVNC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Enrolled'],
    "term_4_ENVNC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Applications'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Denied'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Declined'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Pending'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ENVNC_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Applications'],
    "term_4_ENVNC_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Accepted'],
    "term_4_ENVNC_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_ENVNC_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Denied'],
    "term_4_ENVNC_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Declined'],
    "term_4_ENVNC_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Pending'],
    "term_4_ENVNC_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Waitlisted'],
    "term_4_ENVNC_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Enrolled'],
    "term_4_ENVNC_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ENVNC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Applications'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Accepted'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Denied'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Declined'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Pending'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ENVNC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_UTARTS_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Applications'],
    "term_4_UTARTS_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Accepted'],
    "term_4_UTARTS_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTARTS_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Denied'],
    "term_4_UTARTS_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Declined'],
    "term_4_UTARTS_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Pending'],
    "term_4_UTARTS_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTARTS_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTARTS_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTARTS_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Applications'],
    "term_4_UTARTS_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Accepted'],
    "term_4_UTARTS_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_UTARTS_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Denied'],
    "term_4_UTARTS_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Declined'],
    "term_4_UTARTS_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Pending'],
    "term_4_UTARTS_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Waitlisted'],
    "term_4_UTARTS_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Enrolled'],
    "term_4_UTARTS_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTARTS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Applications'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Accepted'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Denied'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Declined'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Pending'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTARTS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Applications'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Accepted'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Denied'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Declined'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Pending'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTCOMM_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Applications'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Accepted'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Denied'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Declined'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Pending'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Waitlisted'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Enrolled'],
    "term_4_UTCOMM_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTCOMM_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Applications'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Accepted'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Denied'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Declined'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Pending'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTCOMM_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_UTENG_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Applications'],
    "term_4_UTENG_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Accepted'],
    "term_4_UTENG_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTENG_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Denied'],
    "term_4_UTENG_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Declined'],
    "term_4_UTENG_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Pending'],
    "term_4_UTENG_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTENG_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTENG_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTENG_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTENG_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTENG_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Applications'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Accepted'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Denied'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Declined'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Pending'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTSCGS_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Applications'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Accepted'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Denied'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Declined'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Pending'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Waitlisted'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Enrolled'],
    "term_4_UTSCGS_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCGS_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Applications'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Accepted'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Denied'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Declined'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Pending'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCGS_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Applications'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Accepted'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Denied'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Declined'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Pending'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Waitlisted'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Enrolled'],
    "term_4_UTSCWK_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Applications'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Accepted'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Denied'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Declined'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Pending'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Applications'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Accepted'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Denied'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Declined'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Pending'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Waitlisted'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Enrolled'],
    "term_4_UTSCWK_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCWK_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Applications'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Accepted'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Denied'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Declined'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Pending'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_UTSCWK_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_OPEN_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Applications'],
    "term_4_OPEN_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Accepted'],
    "term_4_OPEN_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_OPEN_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Denied'],
    "term_4_OPEN_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Declined'],
    "term_4_OPEN_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Pending'],
    "term_4_OPEN_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Waitlisted'],
    "term_4_OPEN_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Enrolled'],
    "term_4_OPEN_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_OPEN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Applications'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Accepted'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Denied'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Declined'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Pending'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_OPEN_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_OPEN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Applications'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Accepted'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Denied'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Declined'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Pending'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Enrolled'],
    "term_4_GOVNSC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Applications'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Denied'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Declined'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Pending'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Applications'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Accepted'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Denied'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Declined'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Pending'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Waitlisted'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Enrolled'],
    "term_4_GOVNSC_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_GOVNSC_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Applications'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Accepted'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Denied'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Declined'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Pending'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_GOVNSC_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_COMSCC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Applications'],
    "term_4_COMSCC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Accepted'],
    "term_4_COMSCC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_COMSCC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Denied'],
    "term_4_COMSCC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Declined'],
    "term_4_COMSCC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Pending'],
    "term_4_COMSCC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_COMSCC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Enrolled'],
    "term_4_COMSCC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_COMSCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Applications'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Denied'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Declined'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Pending'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_COMSCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_COMSCD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Applications'],
    "term_4_COMSCD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Accepted'],
    "term_4_COMSCD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_COMSCD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Denied'],
    "term_4_COMSCD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Declined'],
    "term_4_COMSCD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Pending'],
    "term_4_COMSCD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_COMSCD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Enrolled'],
    "term_4_COMSCD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Applications'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Denied'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Declined'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Pending'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_COMSCD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Applications'],
    "term_4_COMSCD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Accepted'],
    "term_4_COMSCD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_COMSCD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Denied'],
    "term_4_COMSCD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Declined'],
    "term_4_COMSCD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Pending'],
    "term_4_COMSCD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_COMSCD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Enrolled'],
    "term_4_COMSCD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_COMSCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Applications'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Denied'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Declined'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Pending'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_COMSCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_GNARTC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Applications'],
    "term_4_GNARTC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Accepted'],
    "term_4_GNARTC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_GNARTC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Denied'],
    "term_4_GNARTC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Declined'],
    "term_4_GNARTC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Pending'],
    "term_4_GNARTC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_GNARTC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Enrolled'],
    "term_4_GNARTC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GNARTC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Applications'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Denied'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Declined'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Pending'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GNARTC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Applications'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Accepted'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Denied'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Declined'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Pending'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Enrolled'],
    "term_4_GNSCIC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GNSCIC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Applications'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Denied'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Declined'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Pending'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_GNSCIC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Applications'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Accepted'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Denied'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Declined'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Pending'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Enrolled'],
    "term_4_ANCPDC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ANCPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Applications'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Denied'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Declined'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Pending'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ANCPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_NSPDC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Applications'],
    "term_4_NSPDC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Accepted'],
    "term_4_NSPDC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_NSPDC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Denied'],
    "term_4_NSPDC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Declined'],
    "term_4_NSPDC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Pending'],
    "term_4_NSPDC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_NSPDC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Enrolled'],
    "term_4_NSPDC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_NSPDC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Applications'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Denied'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Declined'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Pending'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_NSPDC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_AITS_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Applications'],
    "term_4_AITS_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Accepted'],
    "term_4_AITS_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_AITS_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Denied'],
    "term_4_AITS_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Declined'],
    "term_4_AITS_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Pending'],
    "term_4_AITS_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Waitlisted'],
    "term_4_AITS_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Enrolled'],
    "term_4_AITS_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_AITS_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_AITS_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Applications'],
    "term_4_AITS_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Accepted'],
    "term_4_AITS_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_AITS_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Denied'],
    "term_4_AITS_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Declined'],
    "term_4_AITS_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Pending'],
    "term_4_AITS_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_AITS_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_AITS_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_AITS_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_LINC13_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Applications'],
    "term_4_LINC13_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Accepted'],
    "term_4_LINC13_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_LINC13_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Denied'],
    "term_4_LINC13_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Declined'],
    "term_4_LINC13_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Pending'],
    "term_4_LINC13_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Waitlisted'],
    "term_4_LINC13_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Enrolled'],
    "term_4_LINC13_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_LINC13_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Applications'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Accepted'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Denied'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Declined'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Pending'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_LINC13_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_LINC13_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_AELCC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Applications'],
    "term_4_AELCC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Accepted'],
    "term_4_AELCC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_AELCC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Denied'],
    "term_4_AELCC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Declined'],
    "term_4_AELCC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Pending'],
    "term_4_AELCC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_AELCC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Enrolled'],
    "term_4_AELCC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_AELCC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Applications'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Denied'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Declined'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Pending'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_AELCC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_AELCC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ECCERT_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Applications'],
    "term_4_ECCERT_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Accepted'],
    "term_4_ECCERT_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ECCERT_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Denied'],
    "term_4_ECCERT_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Declined'],
    "term_4_ECCERT_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Pending'],
    "term_4_ECCERT_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ECCERT_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Enrolled'],
    "term_4_ECCERT_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ECCERT_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Applications'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Denied'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Declined'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Pending'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ECCERT_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ECDIP_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Applications'],
    "term_4_ECDIP_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Accepted'],
    "term_4_ECDIP_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ECDIP_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Denied'],
    "term_4_ECDIP_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Declined'],
    "term_4_ECDIP_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Pending'],
    "term_4_ECDIP_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ECDIP_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Enrolled'],
    "term_4_ECDIP_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Applications'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Denied'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Declined'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Pending'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ECDIP_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Applications'],
    "term_4_ECDIP_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Accepted'],
    "term_4_ECDIP_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_ECDIP_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Denied'],
    "term_4_ECDIP_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Declined'],
    "term_4_ECDIP_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Pending'],
    "term_4_ECDIP_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Waitlisted'],
    "term_4_ECDIP_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Enrolled'],
    "term_4_ECDIP_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ECDIP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Applications'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Accepted'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Denied'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Declined'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Pending'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ECDIP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_CYCD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Applications'],
    "term_4_CYCD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Accepted'],
    "term_4_CYCD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_CYCD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Denied'],
    "term_4_CYCD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Declined'],
    "term_4_CYCD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Pending'],
    "term_4_CYCD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_CYCD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Enrolled'],
    "term_4_CYCD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Applications'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Denied'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Declined'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Pending'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_CYCD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_CYCD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Applications'],
    "term_4_CYCD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Accepted'],
    "term_4_CYCD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_CYCD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Denied'],
    "term_4_CYCD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Declined'],
    "term_4_CYCD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Pending'],
    "term_4_CYCD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_CYCD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Enrolled'],
    "term_4_CYCD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_CYCD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Applications'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Denied'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Declined'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Pending'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_CYCD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_CYCD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_ECEAC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Applications'],
    "term_4_ECEAC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Accepted'],
    "term_4_ECEAC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ECEAC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Denied'],
    "term_4_ECEAC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Declined'],
    "term_4_ECEAC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Pending'],
    "term_4_ECEAC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ECEAC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Enrolled'],
    "term_4_ECEAC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ECEAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Applications'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Denied'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Declined'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Pending'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ECEAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Applications'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Accepted'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Denied'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Declined'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Pending'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Enrolled'],
    "term_4_SOCWKD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Applications'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Denied'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Declined'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Pending'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Applications'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Accepted'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Denied'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Declined'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Pending'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Waitlisted'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Enrolled'],
    "term_4_SOCWKD_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_SOCWKD_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Applications'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Accepted'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Denied'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Declined'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Pending'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_SOCWKD_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_HCAC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Applications'],
    "term_4_HCAC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Accepted'],
    "term_4_HCAC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_HCAC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Denied'],
    "term_4_HCAC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Declined'],
    "term_4_HCAC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Pending'],
    "term_4_HCAC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_HCAC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Enrolled'],
    "term_4_HCAC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HCAC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Applications'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Denied'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Declined'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Pending'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_HCAC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_HCAC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PCPC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Applications'],
    "term_4_PCPC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Accepted'],
    "term_4_PCPC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PCPC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Denied'],
    "term_4_PCPC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Declined'],
    "term_4_PCPC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Pending'],
    "term_4_PCPC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PCPC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Enrolled'],
    "term_4_PCPC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PCPC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Applications'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Denied'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Declined'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Pending'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PCPC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PCPC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ACP_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Applications'],
    "term_4_ACP_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Accepted'],
    "term_4_ACP_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_ACP_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Denied'],
    "term_4_ACP_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Declined'],
    "term_4_ACP_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Pending'],
    "term_4_ACP_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Waitlisted'],
    "term_4_ACP_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Enrolled'],
    "term_4_ACP_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_ACP_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Applications'],
    "term_4_ACP_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Accepted'],
    "term_4_ACP_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_ACP_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Denied'],
    "term_4_ACP_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Declined'],
    "term_4_ACP_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Pending'],
    "term_4_ACP_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_ACP_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_ACP_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_ACP_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Applications'],
    "term_4_ACP_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Accepted'],
    "term_4_ACP_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_ACP_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Denied'],
    "term_4_ACP_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Declined'],
    "term_4_ACP_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Pending'],
    "term_4_ACP_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Waitlisted'],
    "term_4_ACP_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Enrolled'],
    "term_4_ACP_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ACP_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_ACP_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Applications'],
    "term_4_ACP_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Accepted'],
    "term_4_ACP_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_ACP_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Denied'],
    "term_4_ACP_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Declined'],
    "term_4_ACP_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Pending'],
    "term_4_ACP_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_ACP_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_ACP_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_ACP_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_PNR_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Applications'],
    "term_4_PNR_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Accepted'],
    "term_4_PNR_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PNR_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Denied'],
    "term_4_PNR_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Declined'],
    "term_4_PNR_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Pending'],
    "term_4_PNR_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PNR_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Enrolled'],
    "term_4_PNR_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PNR_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Applications'],
    "term_4_PNR_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PNR_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PNR_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Denied'],
    "term_4_PNR_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Declined'],
    "term_4_PNR_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Pending'],
    "term_4_PNR_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PNR_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PNR_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PNR_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Applications'],
    "term_4_PNR_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Accepted'],
    "term_4_PNR_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_PNR_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Denied'],
    "term_4_PNR_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Declined'],
    "term_4_PNR_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Pending'],
    "term_4_PNR_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Waitlisted'],
    "term_4_PNR_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Enrolled'],
    "term_4_PNR_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_PNR_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_PNR_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Applications'],
    "term_4_PNR_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Accepted'],
    "term_4_PNR_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_PNR_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Denied'],
    "term_4_PNR_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Declined'],
    "term_4_PNR_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Pending'],
    "term_4_PNR_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_PNR_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_PNR_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_PNR_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_BSCN_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Applications'],
    "term_4_BSCN_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Accepted'],
    "term_4_BSCN_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_BSCN_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Denied'],
    "term_4_BSCN_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Declined'],
    "term_4_BSCN_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Pending'],
    "term_4_BSCN_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Waitlisted'],
    "term_4_BSCN_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Enrolled'],
    "term_4_BSCN_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Applications'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Accepted'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Denied'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Declined'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Pending'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_BSCN_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_BSCN_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Applications'],
    "term_4_BSCN_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Accepted'],
    "term_4_BSCN_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_BSCN_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Denied'],
    "term_4_BSCN_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Declined'],
    "term_4_BSCN_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Pending'],
    "term_4_BSCN_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Waitlisted'],
    "term_4_BSCN_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Enrolled'],
    "term_4_BSCN_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BSCN_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Applications'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Accepted'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Denied'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Declined'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Pending'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_BSCN_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_BSCN_INTERNATIONAL_Year_2_Conversion_Rate'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Applications": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Applications'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Accepted": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Accepted'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Payment_Pending": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Payment_Pending'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Denied": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Denied'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Declined": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Declined'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Pending": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Pending'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Waitlisted": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Waitlisted'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Enrolled": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Enrolled'],
    "term_4_NursingYr3_DOMESTIC_Year_3_Conversion_Rate": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_DOMESTIC_Year_3_Conversion_Rate'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Applications": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Applications'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Accepted": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Accepted'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Payment_Pending'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Denied": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Denied'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Declined": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Declined'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Pending": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Pending'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Waitlisted": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Waitlisted'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Enrolled": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Enrolled'],
    "term_4_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate": term_4_NursingYr3_variable_dictionary['term_4_NursingYr3_INTERNATIONAL_Year_3_Conversion_Rate'],
    "term_4_IENCN_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Applications'],
    "term_4_IENCN_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Accepted'],
    "term_4_IENCN_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_IENCN_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Denied'],
    "term_4_IENCN_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Declined'],
    "term_4_IENCN_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Pending'],
    "term_4_IENCN_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Waitlisted'],
    "term_4_IENCN_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Enrolled'],
    "term_4_IENCN_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_IENCN_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Applications'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Accepted'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Denied'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Declined'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Pending'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_IENCN_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_IENCN_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PAELEC_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Applications'],
    "term_4_PAELEC_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Accepted'],
    "term_4_PAELEC_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PAELEC_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Denied'],
    "term_4_PAELEC_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Declined'],
    "term_4_PAELEC_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Pending'],
    "term_4_PAELEC_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PAELEC_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Enrolled'],
    "term_4_PAELEC_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAELEC_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Applications'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Denied'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Declined'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Pending'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAELEC_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PAWELD_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Applications'],
    "term_4_PAWELD_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Accepted'],
    "term_4_PAWELD_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PAWELD_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Denied'],
    "term_4_PAWELD_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Declined'],
    "term_4_PAWELD_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Pending'],
    "term_4_PAWELD_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PAWELD_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Enrolled'],
    "term_4_PAWELD_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAWELD_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Applications'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Denied'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Declined'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Pending'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAWELD_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PAHET_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Applications'],
    "term_4_PAHET_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Accepted'],
    "term_4_PAHET_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PAHET_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Denied'],
    "term_4_PAHET_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Declined'],
    "term_4_PAHET_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Pending'],
    "term_4_PAHET_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PAHET_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Enrolled'],
    "term_4_PAHET_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAHET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Applications'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Denied'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Declined'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Pending'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PAHET_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAHET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Applications'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Accepted'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Denied'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Declined'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Pending'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Enrolled'],
    "term_4_PAPLUM_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAPLUM_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Applications'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Denied'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Declined'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Pending'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PAPLUM_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_POWCM4_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Applications'],
    "term_4_POWCM4_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Accepted'],
    "term_4_POWCM4_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_POWCM4_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Denied'],
    "term_4_POWCM4_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Declined'],
    "term_4_POWCM4_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Pending'],
    "term_4_POWCM4_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Waitlisted'],
    "term_4_POWCM4_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Enrolled'],
    "term_4_POWCM4_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCM4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Applications'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Accepted'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Denied'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Declined'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Pending'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCM4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_POWCM3_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Applications'],
    "term_4_POWCM3_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Accepted'],
    "term_4_POWCM3_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_POWCM3_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Denied'],
    "term_4_POWCM3_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Declined'],
    "term_4_POWCM3_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Pending'],
    "term_4_POWCM3_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Waitlisted'],
    "term_4_POWCM3_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Enrolled'],
    "term_4_POWCM3_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCM3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Applications'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Accepted'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Denied'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Declined'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Pending'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCM3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_POWCP4_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Applications'],
    "term_4_POWCP4_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Accepted'],
    "term_4_POWCP4_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_POWCP4_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Denied'],
    "term_4_POWCP4_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Declined'],
    "term_4_POWCP4_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Pending'],
    "term_4_POWCP4_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Waitlisted'],
    "term_4_POWCP4_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Enrolled'],
    "term_4_POWCP4_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCP4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Applications'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Accepted'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Denied'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Declined'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Pending'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCP4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_POWCO4_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Applications'],
    "term_4_POWCO4_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Accepted'],
    "term_4_POWCO4_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_POWCO4_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Denied'],
    "term_4_POWCO4_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Declined'],
    "term_4_POWCO4_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Pending'],
    "term_4_POWCO4_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Waitlisted'],
    "term_4_POWCO4_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Enrolled'],
    "term_4_POWCO4_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCO4_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Applications'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Accepted'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Denied'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Declined'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Pending'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCO4_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_POWCO3_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Applications'],
    "term_4_POWCO3_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Accepted'],
    "term_4_POWCO3_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_POWCO3_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Denied'],
    "term_4_POWCO3_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Declined'],
    "term_4_POWCO3_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Pending'],
    "term_4_POWCO3_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Waitlisted'],
    "term_4_POWCO3_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Enrolled'],
    "term_4_POWCO3_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCO3_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Applications'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Accepted'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Denied'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Declined'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Pending'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_POWCO3_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PPET_DOMESTIC_Year_1_Applications": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Applications'],
    "term_4_PPET_DOMESTIC_Year_1_Accepted": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Accepted'],
    "term_4_PPET_DOMESTIC_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Payment_Pending'],
    "term_4_PPET_DOMESTIC_Year_1_Denied": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Denied'],
    "term_4_PPET_DOMESTIC_Year_1_Declined": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Declined'],
    "term_4_PPET_DOMESTIC_Year_1_Pending": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Pending'],
    "term_4_PPET_DOMESTIC_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Waitlisted'],
    "term_4_PPET_DOMESTIC_Year_1_Enrolled": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Enrolled'],
    "term_4_PPET_DOMESTIC_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_1_Conversion_Rate'],
    "term_4_PPET_INTERNATIONAL_Year_1_Applications": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Applications'],
    "term_4_PPET_INTERNATIONAL_Year_1_Accepted": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Accepted'],
    "term_4_PPET_INTERNATIONAL_Year_1_Payment_Pending": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Payment_Pending'],
    "term_4_PPET_INTERNATIONAL_Year_1_Denied": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Denied'],
    "term_4_PPET_INTERNATIONAL_Year_1_Declined": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Declined'],
    "term_4_PPET_INTERNATIONAL_Year_1_Pending": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Pending'],
    "term_4_PPET_INTERNATIONAL_Year_1_Waitlisted": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Waitlisted'],
    "term_4_PPET_INTERNATIONAL_Year_1_Enrolled": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Enrolled'],
    "term_4_PPET_INTERNATIONAL_Year_1_Conversion_Rate": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_1_Conversion_Rate'],
    "term_4_PPET_DOMESTIC_Year_2_Applications": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Applications'],
    "term_4_PPET_DOMESTIC_Year_2_Accepted": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Accepted'],
    "term_4_PPET_DOMESTIC_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Payment_Pending'],
    "term_4_PPET_DOMESTIC_Year_2_Denied": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Denied'],
    "term_4_PPET_DOMESTIC_Year_2_Declined": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Declined'],
    "term_4_PPET_DOMESTIC_Year_2_Pending": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Pending'],
    "term_4_PPET_DOMESTIC_Year_2_Waitlisted": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Waitlisted'],
    "term_4_PPET_DOMESTIC_Year_2_Enrolled": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Enrolled'],
    "term_4_PPET_DOMESTIC_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_PPET_DOMESTIC_Year_2_Conversion_Rate'],
    "term_4_PPET_INTERNATIONAL_Year_2_Applications": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Applications'],
    "term_4_PPET_INTERNATIONAL_Year_2_Accepted": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Accepted'],
    "term_4_PPET_INTERNATIONAL_Year_2_Payment_Pending": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Payment_Pending'],
    "term_4_PPET_INTERNATIONAL_Year_2_Denied": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Denied'],
    "term_4_PPET_INTERNATIONAL_Year_2_Declined": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Declined'],
    "term_4_PPET_INTERNATIONAL_Year_2_Pending": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Pending'],
    "term_4_PPET_INTERNATIONAL_Year_2_Waitlisted": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Waitlisted'],
    "term_4_PPET_INTERNATIONAL_Year_2_Enrolled": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Enrolled'],
    "term_4_PPET_INTERNATIONAL_Year_2_Conversion_Rate": term_4_variable_dictionary['term_4_PPET_INTERNATIONAL_Year_2_Conversion_Rate'],
}

# Regular expression to detect {{variable}} patterns
pattern = re.compile(r"{{(.*?)}}")

# Loop through all worksheets and replace variables
for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if isinstance(cell.value, str):
                matches = pattern.findall(cell.value)
                for match in matches:
                    if match in variable_map:
                        replacement = variable_map[match]
                        if cell.value.strip() == f"{{{{{match}}}}}":  # Entire cell is just the variable
                            cell.value = replacement  # Assign number directly if numeric
                        else:
                            cell.value = cell.value.replace(f"{{{{{match}}}}}", str(replacement))

# Rename sheets by replacing placeholders in their names
for ws in wb.worksheets:
    matches = pattern.findall(ws.title)
    new_title = ws.title
    for match in matches:
        if match in variable_map:
            new_title = new_title.replace(f"{{{{{match}}}}}", str(variable_map[match]))
    if new_title != ws.title:
        ws.title = new_title

# Convert text to numbers in column P for specific sheets
target_sheets = [f"{term_1_term_year_current} Apps By School", f"{term_2_term_year_current} Apps By School", f"{term_3_term_year_current} Apps By School", f"{term_4_term_year_current} Apps By School"]
column_p_index = openpyxl.utils.column_index_from_string('P')

for sheet_name in target_sheets:
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for row in ws.iter_rows(min_col=column_p_index, max_col=column_p_index):
            cell = row[0]
            if isinstance(cell.value, str) and cell.value.strip().replace(',', '').replace('.', '', 1).isdigit():
                try:
                    cell.value = float(cell.value.replace(',', '')) if '.' in cell.value else int(cell.value.replace(',', ''))
                except ValueError:
                    pass  # Skip if conversion fails

# Define output folder, create one if it doesn't exist
output_folder = "Output Files"
os.makedirs(output_folder, exist_ok=True)

# Save to new file
output_filename = f"KC {variable_current_academic_year_file_name} Applications KPI as of {variable_Extraction_date}.xlsx"
output_path = os.path.join(output_folder, output_filename)
wb.save(output_path)


In [28]:
# Reopen the final output workbook and hide column D (Program Code) on the Apps by School sheets
# Investigate how to delete column D without breaking the formatting later
# Also investigate how to fix hyperlinks on Glossary sheet
wb = openpyxl.load_workbook(output_path)

column_d_letter = 'D'

for sheet_name in target_sheets:
    if sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        ws.column_dimensions[column_d_letter].hidden = True  # Hide column D

# Add hyperlinks to Summary!C26:C29 if they match sheet names
summary_ws = wb["Summary"]
link_cells = ["C26", "C27", "C28", "C29"]

for cell_ref in link_cells:
    cell = summary_ws[cell_ref]
    sheet_name = cell.value
    if isinstance(sheet_name, str) and sheet_name in wb.sheetnames:
        # Add single quotes around the sheet name to handle spaces
        cell.hyperlink = f"#'{sheet_name}'!A1"

# Save changes
wb.save(output_path)


In [29]:
term_2_variable_dictionary

{'term_2_BABUSC_DOMESTIC_Year_1_Applications': 5,
 'term_2_BABUSC_DOMESTIC_Year_1_Accepted': 1,
 'term_2_BABUSC_DOMESTIC_Year_1_Payment_Pending': None,
 'term_2_BABUSC_DOMESTIC_Year_1_Denied': None,
 'term_2_BABUSC_DOMESTIC_Year_1_Declined': None,
 'term_2_BABUSC_DOMESTIC_Year_1_Pending': 3,
 'term_2_BABUSC_DOMESTIC_Year_1_Waitlisted': None,
 'term_2_BABUSC_DOMESTIC_Year_1_Enrolled': 1,
 'term_2_BABUSC_DOMESTIC_Year_1_Conversion_Rate': 1.0,
 'term_2_BABUSC_DOMESTIC_Year_2_Applications': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Accepted': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Payment_Pending': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Denied': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Declined': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Pending': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Waitlisted': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Enrolled': None,
 'term_2_BABUSC_DOMESTIC_Year_2_Conversion_Rate': None,
 'term_2_BABUSC_DOMESTIC_Year_3_Applications': None,
 'term_2_BABUSC_DOMESTIC_Year_3_Accepted': N